<a href="https://colab.research.google.com/github/gluo88/misc/blob/main/notebook/finance.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# References


**hosted web page at Hugging Face**

* https://huggingface.co/spaces/gluo88/stock_performance


**Core references**
* yfinance https://pypi.org/project/yfinance/,
 * tutorial: https://aroussi.com/post/python-yahoo-finance
* pandas tutorial https://www.w3schools.com/python/pandas/default.asp

**an unofficial Yahoo Finance API - yahooquery**
* https://yahooquery.dpguthrie.com/
* https://github.com/dpguthrie/yahooquery

**Retired Yahoo! Query Language (YQL)**
* https://en.wikipedia.org/wiki/Yahoo!_Query_Language

**Misc**
* https://sourceforge.net/software/stock-analysis/free-version/
* https://wallethacks.com/free-stock-analysis-software-tools/
* How to Use Python and Pandas with Yahoo Finance API https://saturncloud.io/blog/how-to-use-python-and-pandas-with-yahoo-finance-api/
* PyPortfolioOpt https://pyportfolioopt.readthedocs.io/en/latest/UserGuide.html
* https://nyc3.digitaloceanspaces.com/roboticview/media/jupyter/Yahoo_Finance_Stock_Analysis_1.html?AWSAccessKeyId=DO00FWAZBR62EWAWDFYF&Signature=dBz8W5movGymEYR5kw5gggByzL8%3D&Expires=1705115798


# Installation

In [1]:
!pip install gradio
#!pip install --upgrade gradio
#!pip install gradio==3.50
!pip install yfinance
!pip install datetime
!pip install pytz
!pip install pandas
#!pip install --upgrade pandas
!pip install numpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.9/16.9 MB 47.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.1/92.1 kB 14.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.9/307.9 kB 23.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.5/138.5 kB 20.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 43.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 18.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.8/77.8 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 kB 11.3 MB/s eta 0:00:00
  Created wheel for ffmpy: filename=ffmpy-0.3.2-py3-none-any.whl si

# gradio deploy

In [ ]:
#!pip install gradio
!gradio deploy


Need 'write' access token to create a Spaces repo.

    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) Y
Token is valid (permission: write).
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub.
Run the following command in your 

# Example 9  annual,  trailing, cumumlative, and CAGR  returns  (restructured from Example 8)

In [ ]:
'''
Example 9 for using yfinance

Calculate annual, trailing, cumumlative, and CAGR returns for multiple stocks.
* The start date can be an arbitrary date.  The default is the current date.
* annual return is displayed from the default current day, or an arbitrary given
  day (except for Feb 29 for leap year)
  For leap years, use Feb 28 to replace Feb 29 as simplification & approximation
* trailing, cumumlative returns are currently displayed from the month boundary (last day of Month)
  prior to the given date.
* However, trailing, cumumlative returns can be displayed
  from any date, which can be not at the month boundary (last day of Month),
  by minor change of setting calculation_end_date_for_others_str = calculation_end_date_str.
  prior to the given date in the function "calculation_response(message, history)"

Author: Gang Luo
'''
script_version = '(2024-03-04.2)'
import gradio as gr
import yfinance as yf
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import pytz
DEBUG_ENABLED = True
#==============================================================================

print_yearly_total_return = True
num_years_calculation=52   # total years for calculation

# Define a list of years to calculate the trailing returns, cumulative returns, and so on
# remove the row of current year row since it is not a full year.
years_list = [1, 2, 3, 5, 10, 15, 20, 25, 30, 40, 50, 60]

# Set the stock tickers list
tickers_lists = [["qqq","hxq.to","spy", "vfv.to","xiu.to", "xbb.to","xcb.to","xhb.to"], #0  checking ETF
    ["qqq","spy", "vfv.to", "vgg.to", "zlu.to", "xiu.to", "vdy.to", "xfn.to", "ry.to", "td.to", "na.to",
      "slf.to", "gwo.to", "bce.to", "t.to", "rci-b.to", "enb.to", "trp.to", "zlb.to", "cp.to"], #1 main monitoring list
    ["qqq","spy","vfv.to", "xiu.to", "zeb.to", "xfn.to", "na.to","ry.to", "bmo.to","bns.to", "td.to", "cm.to", "cwb.to",
      "slf.to", "gwo.to", "bce.to", "t.to", "rci-b.to", "enb.to", "trp.to", "xdv.to","cdz.to","vdy.to", "xdiv.to"],  #2   financial  ETF & stocks
    ["^ndx","qqq","^GSPC","spy", "vfv.to", "vgg.to","zlu.to","msft","AAPL","goog","AMZN","NVDA","meta","tsla","BRK-A","shop.to","hxq.to"],   #3  US mega stocks + risky shopfy
    ["^DJI","dia","^GSPC","spy","voo","ivv", "tpu-u.to","vfv.to", "zsp.to","hxs.to","tpu.to","xus.to", "xsp.to",
      "^IXIC","^ndx", "qqq","hxq.to","^GSPTSE","xic.to","xiu.to", "HXT.TO", "TTP.TO","ZCN.TO", "xfn.to", "xit.to"], #4   indexes and index ETFs
    ["dia","^DJI","^GSPC","spy","vfv.to", "zsp.to","hxs.to","xus.to", "xsp.to",
      "^IXIC","qqq","hxq.to","^GSPTSE","xic.to","xiu.to", "HXT.TO", "xfn.to"], #5   indexes and typical index ETFs
    ["^IXIC","^ndx","ONEQ","CIBR","QQJG", "qqq", "tqqq", "spy", "vfv.to", "HXQ.to", "ZQQ.to", "XQQ.to", "QQC.to", "ZNQ.TO",
         "xiu.to", "xit.to"],  #6   Nasdaq ETF and TSX IT ETF
    ["qqq","tqqq","sqqq", "QLD", "spy", "spxu", "upro", "sso", "spxl","tecl"], #7 leveraged ETFs
    ["^IXIC","^DJI","^GSPC","^GSPTSE"], #8 testing
    ["vfv.to","spy"] #9 testing
]

#==============================================================================
# Part 1:
#  retrieve daily adjusted close prices of a list of tickers from yahoo finance
#  Generate the year-end adjusted close prices
#  return year-end adjusted close prices, and  daily adjusted close prices
def stock_prices_df(tickers_list, end_date_str):
    tickers_list_upper = [ticker.upper() for ticker in tickers_list]
    tickers_str = ", ".join(tickers_list_upper)
    try:
        '''
        'try' statement for handlingy the exception error  for yf.download
        '''
        # Download the historical data
        data = yf.download(tickers_str, period="max")
    except:
        return  pd.DataFrame()
    else:
        data_adj_close = data['Adj Close']

    # Filter out rows with dates newer than calculation_end_date
    data_adj_close = data_adj_close[data_adj_close.index <= end_date_str]
    #print("\nDebug- stock_prices_df\n", data_adj_close)

    # Rearrange columns based on the order in tickers_list_upper
    if len(tickers_list)>1:
        data_adj_close = data_adj_close.reindex(columns=tickers_list_upper)

    # needed this when having only a single ticker in the ticker list
    if len(tickers_list_upper)==1:
        data_adj_close = pd.DataFrame(data_adj_close)
        data_adj_close.rename(columns={'Adj Close': tickers_list_upper[0]}, inplace=True)

    data_adj_close.columns = map(str.lower, data_adj_close.columns) # must after  pd.DataFrame(data_adj_close)
    data_adj_close_year_end = data_adj_close.resample('A').ffill().round(2) # must before index changed to date

    data_adj_close.index=data_adj_close.index.date
    data_adj_close_year_end.index=data_adj_close_year_end.index.date

    last_date = data_adj_close.index[-1]
    data_adj_close_year_end = data_adj_close_year_end.rename(index={last_date: end_date_str})

    return data_adj_close_year_end, data_adj_close

#==============================================================================
# Part 2:  Calculate annual returns at year end, and at any given day (by calculation_end_date_str)
#
# annual return calculation can start at any given day
def get_annual_returns_anyday_df(daily_adj_close_df, calculation_end_date_str):

    calculation_end_date=pd.to_datetime(calculation_end_date_str).tz_localize('America/New_York')

    # Create a DataFrame with a complete date range
    date_range = pd.date_range(start=daily_adj_close_df.index.min(), end=daily_adj_close_df.index.max(), freq='D')

    complete_stock_history = pd.DataFrame(index=date_range)
    # Merge the complete DataFrame with the original stock_history
    complete_stock_history = complete_stock_history.merge(daily_adj_close_df, how='left', left_index=True, right_index=True)
    complete_stock_history = complete_stock_history.ffill()  # fill the newy added rows with previous day value
    '''
    Filter out the rows that matches the month and date of calculation_end_date, which are the ends of
    annual periods from the calculation_end_date.
    '''
    # Filter out rows with dates newer than calculation_end_date
    #filtered_stock_history = complete_stock_history[complete_stock_history.index <= calculation_end_date]
    # note" daily_adj_close_df satisfys daily_adj_close_df.index <= calculation_end_date
    filtered_stock_history = complete_stock_history
    #print(filtered_stock_history)
    target_month=filtered_stock_history.index.max().month
    target_day=filtered_stock_history.index.max().day
    #print("target_month", target_month, "target_day",target_day, "start_year", filtered_stock_history.index.max().year)
    annual_returns = filtered_stock_history[(filtered_stock_history.index.month == target_month)
           & (filtered_stock_history.index.day ==target_day)]
    annual_returns_percent = annual_returns.pct_change().dropna(how='all')

    annual_returns_df = pd.DataFrame(annual_returns_percent)
    #print("\ndebug-annual_returns_df\n", annual_returns_df)
    return annual_returns_df

# annual return calculation can start at year end
def get_annual_returns_year_end_df(data_adj_close_df, calculation_end_date_str):
    annual_returns_percent = data_adj_close_df.pct_change().dropna(how='all')
    return annual_returns_percent

#==============================================================================
# Part 3: calculate the annualized trailing total return from the data generated in step 1 & display
# Define a function to calculate the annualized trailing total return for a given number of years
def get_trailing_return(ticker, data, years):
    # Get the total return values for the last n years
    trailing_data = data[ticker].tail(years)
    # Check if there are empty values within years
    if trailing_data.isna().any():
        return np.nan
    # Check if there are valid total return values for all years
    if len(trailing_data) == years:
        # Convert the percentage strings to numeric values
        trailing_data = trailing_data.astype(str).str.replace('%', '').astype(float)
        """ Calculate the annualized trailing total return using the formula from Investopedia[^1^][1]:
            Annualized Return = [(1 + r1) * (1 + r2) * ... * (1 + rn)]^(1/n) - 1
            Where r1, r2, ..., rn are the total return values for each year                    """
        annualized_trailing_return = (trailing_data + 1).prod() ** (1 / years) - 1

        # Format the result as a percentage with two decimal places
        annualized_trailing_return = annualized_trailing_return * 100
        annualized_trailing_return = annualized_trailing_return.round(2)
        return annualized_trailing_return
    else:
        return np.nan

# Define a function to Loop through the list and print the trailing returns for each num_years
def get_trailing_return_column(ticker, annual_returns_df):
    trailing_return_column = {}
    for num_years in years_list:
        # Check if the ticker data is available in all_tickers_returns_df
        if ticker in annual_returns_df.columns:
            # using data from step 1, avoiding get_annual_returns_df(ticker) for less traffic from yahoo server
            data = annual_returns_df[[ticker]]
            trailing_return = get_trailing_return(ticker, data, num_years)
            trailing_return_column[f"{num_years}-Year"] = trailing_return
        else:
            print(f"Data not available for {ticker}. Skipping.")
            trailing_return_column[f"{num_years}-Year"] = np.nan
    return trailing_return_column

# Create an empty DataFrame to store all tickers' trailing returns
def get_trailing_return_all(annual_returns_df):
    all_tickers_trailing_returns_df = pd.DataFrame(index=years_list)
    tickers=annual_returns_df.columns.tolist()
    # Loop through each ticker in the list
    for ticker in tickers:
        trailing_returns = get_trailing_return_column(ticker, annual_returns_df)
        # Add the trailing returns to the DataFrame
        all_tickers_trailing_returns_df[ticker] = pd.Series(trailing_returns).values
    return all_tickers_trailing_returns_df

#==============================================================================
# Part 4: calculate the cumulative return from the data (all_tickers_returns_df) generated in part 1 & display
#  Define a function to calculate the cumulative return for a given number of years from a ticker
def get_cumulative_return(ticker, data, years):
    # Calculate the cumulative return
    cumulative_return = (1 + data[ticker]).rolling(window=years).apply(lambda x: x.prod(), raw=True) - 1
    return cumulative_return

# Define a function to Loop through the list and return the cumulative returns for each num_years
def get_cumulative_return_column(ticker, annual_returns_df):
    cumulative_returns = {}
    for years in years_list:
        # Calculate the cumulative return for the given number of years
        cumulative_return = get_cumulative_return(ticker, annual_returns_df, years)
        # Get the last value, which is the cumulative return up to the current year
        cumulative_returns[years] = cumulative_return.iloc[-1]
    return cumulative_returns

def get_cumulative_return_all(annual_returns_df):
    # Create an empty DataFrame with years_list as the index for cumulative  returns
    all_tickers_cumulative_returns_df = pd.DataFrame(index=years_list)
    tickers=annual_returns_df.columns.tolist()
    # Loop through each ticker in the list
    for ticker in tickers:
        cumulative_returns = get_cumulative_return_column(ticker, annual_returns_df)
        # Add the trailing returns to the DataFrame
        all_tickers_cumulative_returns_df[ticker] = pd.Series(cumulative_returns).values
    return all_tickers_cumulative_returns_df

#==============================================================================
# Part 5: calculate the  CAGR (Compound Annual Growth Rate) from the data
# in all_tickers_cumulative_returns_df generated earlier & display
# Define a function to calculate the CAGR from the cumulative value and the years
def calculate_cagr(value, years):
    # Otherwise, calculate the CAGR using the formula
    cagr = (value + 1) ** (1 / np.array(years)) - 1
    #print("debug-cagr\n", cagr, "end")
    return cagr

# Define a function to format the Float64Index values into percentage strings
def format_to_percentage(value):
    # If any element in the value array is not null, format it as a percentage string with two decimal places
    if np.any(pd.notnull(value)):
        return f"{value:.2f}%"
    # Otherwise, return None
    return None

def get_cagr_return_all(all_tickers_cumulative_returns_df):
    # Apply the calculate_cagr function to each column of the DataFrame
    all_tickers_cagrs_df = all_tickers_cumulative_returns_df.apply(lambda x: calculate_cagr(x, x.index), axis=0)
    return all_tickers_cagrs_df

#==============================================================================
# Part 6:
# single ticker's Prices, Returns,Dividends, good for verifying whether "Adj Close" is correct.
'''
    Calculate and display: yearly dividendSum, 'Close' & 'Adj Close' prices,
    Return(by 'Close' price), total return(by 'Adj Close' price),
    CalReturn(total return by 'Close' price and "dividendSum).
    Note: CalReturn from  is expected to be nearly same as  total return,
           when the 'Adj Close' price is correct.
'''
def get_yearly_single_stock_data(ticker):
    stock = yf.Ticker(ticker)
    #-------- mainly for downloading 'Dividends'
    history = stock.history(period="max")
    dividend_history=history['Dividends']
    dividend_history.index=dividend_history.index.date

    #-------- mainly for downloading 'Close','Adj Close'
    dld_history=yf.download(ticker, period="max")
    dld_history=dld_history[['Close','Adj Close']]
    dld_history.rename(columns={'Adj Close': 'AdjClose'}, inplace=True)
    date_range = pd.date_range(start=dld_history.index.min(), end=dld_history.index.max(), freq='D')
    complete_history = pd.DataFrame(index=date_range)

    # Merge the complete DataFrame with the original stock_history
    complete_history = complete_history.merge(dld_history, how='left', left_index=True, right_index=True)
    complete_history[['Close','AdjClose']] = complete_history[['Close','AdjClose']].ffill().round(3)

    # Merge dividend into complete_history
    complete_history = complete_history.merge(dividend_history, how='left', left_index=True, right_index=True)
    # replace all NaN values in the 'Dividends' column with 0.0
    complete_history['Dividends'] = complete_history['Dividends'].fillna(0.0).round(3)

    complete_history['Year']=complete_history.index.year
    complete_history['Date']=complete_history.index
    yearly_data = complete_history.groupby('Year').agg({'Date': 'last', 'Close': 'last', 'AdjClose': 'last','Dividends': 'sum'})
    yearly_data.rename(columns={'Dividends': 'DivSum'}, inplace=True)

    # calculating 'Return' and 'TotalReturn'
    yearly_data['DivRatio']=yearly_data['DivSum'] / yearly_data['Close']
    yearly_data['Return']=yearly_data['Close'].pct_change()
    yearly_data['TotalReturn']=yearly_data['AdjClose'].pct_change()

    '''
    The CalReturn column is the yearly total return calculated from un-adjusted "Close" prices and yearly "dividend sum",
    which is expected to be equal to the total return that is calculated from "AdjClose" prices
    '''
    yearly_data['CalReturn'] = (yearly_data['Close'] + yearly_data['DivSum']) / yearly_data['Close'].shift(1) - 1
    # set the display format
    yearly_data[['DivRatio','Return','TotalReturn','CalReturn']] = yearly_data[['DivRatio','Return','TotalReturn','CalReturn']].mul(100).round(2)
    '''
    #yearly_data[['DivRatio','Return', 'TotalReturn', 'CalReturn']] = yearly_data[['DivRatio','Return', 'TotalReturn', 'CalReturn']].applymap("{:.2f}%".format)
    yearly_data[['DivRatio','Return', 'TotalReturn', 'CalReturn']]= \
     yearly_data[['DivRatio','Return', 'TotalReturn', 'CalReturn']].applymap(lambda x: f"{x:.2f}%" if not pd.isna(x) else "NaN")
    '''
    # Use .applymap() and lambda to format the values as percentage strings only if they are not NaN
    yearly_data[['DivRatio','Return', 'TotalReturn', 'CalReturn']]= \
         yearly_data[['DivRatio','Return', 'TotalReturn', 'CalReturn']].applymap(lambda x: f"{x:.2f}%" if not pd.isna(x) else x)

    # 'Date' column is no longer required
    yearly_data.drop('Date', axis=1, inplace=True)
    return yearly_data

#==============================================================================
# Part 7: utility functions
# get the last trading day of S&P 500 in string format
def get_last_trading_day():
    # Get today's date, use .strftime("%Y-%m-%d") to convert to a string
    today_date_str=datetime.now(pytz.timezone('America/New_York')).date().strftime("%Y-%m-%d")
    stock = yf.Ticker("^GSPC") # S&P 500 (^GSPC) ticker
    #  search and see yfinance_BUG_1 NOTE in this file
    history_df=stock.history(period="max", end=today_date_str)["Close"]
    last_trading_day_str = history_df.index.max().date().strftime("%Y-%m-%d")
    return last_trading_day_str

def str_to_integer(integer_str):
    try:
        integer_number = int(integer_str)
        return integer_number
    except ValueError:
        return -1

# validate the date string
def is_valid_date(date_string):
    try:
        # Attempt to parse the date string
        datetime.strptime(date_string, "%Y-%m-%d")
        return True
    except ValueError:
        # Raised when the date string is not in the expected format
        return False

def date_label_conversion_strip_time(all_tickers_returns_df, calculation_end_date_str):
    all_tickers_returns_df.index=all_tickers_returns_df.index.date
    all_tickers_returns_df.index.name='date'
    # print("debug get_annual_returns_tickers_df", all_tickers_returns_df)
    # Convert calculation_end_date_str to a datetime object, replace the index's mon/day portion of date
    end_date_datetime_obj = datetime.strptime(calculation_end_date_str, "%Y-%m-%d")
    all_tickers_returns_df.index = all_tickers_returns_df.index.map(
      lambda x: x.replace(month=end_date_datetime_obj.month,
      day=end_date_datetime_obj.day))
    return all_tickers_returns_df

#==============================================================================
# Part 8: gradio handling - Input command handling and display in web page

help_info_str="Input Formats:\n  \
            1. ticker list....................Example:  spy vfv.to xiu.to xic.to xfn.to ry.to \n \
            2. One of default ticker list, a number between 1 and 7....Example:   0, or 1, ...,7 \n \
            3. CalculationEndDate as prefix.  Example:   2020-12-31 2 \n \
            .........................................2020-12-31 spy vfv.to xiu.to xic.to xfn.to ry.to \n \
            4. single ticker: Dividend/Close/AdjClose/Return/TotalReturn/CalReturn(by Close/Dividends).  @1 spy \n \
            note: daily adjusted close data are from Yahoo Finance. "

# Main Handling Process
def calculation_response(message):
    # if there is no input, display help information
    if message=="":
        return help_info_str

    tickers=message.split()

    # ******************************************************************************
    #  processing web input parameters
    #  set calculation_end_date_str, and tickers

    #---------------------------------------------------------
    # single stock ticker - detailed information
    if (tickers[0] == "@1"):
        tickers.pop(0) # remove the first string which is "@1"
        if len(tickers)==0:
            ticker = 'spy' # default ticker = spy
        else:
            ticker=tickers[0]
        output_string=f"\n {ticker}\n"
        output_dataframe0=get_yearly_single_stock_data(ticker)
        output_html=output_string + output_dataframe0.to_html()
        return output_html

    #----------------------------------------------------------
    # Get today's date, use .strftime("%Y-%m-%d") to convert to a string
    #calculation_end_date_str=datetime.now(pytz.timezone('America/New_York')).date().strftime("%Y-%m-%d")
    calculation_end_date_str = get_last_trading_day()
    # Check whether the first str is date for calculation end date
    if is_valid_date(tickers[0]):
        calculation_end_date_str = tickers[0] # reset calculation_end_date_str
        tickers.pop(0) # remove the first string which is the date

    #............ For display trailing and cumulative returns at month_boundary_date
    # Assuming calculation_end_date_str contains the date string '2024-01-03'
    calculation_end_date = datetime.strptime(calculation_end_date_str, '%Y-%m-%d')
    # Calculate the first day of the current month
    first_day_of_month = calculation_end_date.replace(day=1)
    # Calculate the last day of the month
    last_day_of_month = (calculation_end_date.replace(day=1) + timedelta(days=32)).replace(day=1) - timedelta(days=1)
    # Calculate the last day of the previous month
    last_day_of_previous_month = first_day_of_month - timedelta(days=1)
    # Check if the original date is the last day of the month
    if (calculation_end_date == last_day_of_month):
        calculation_end_date_month_boundary_date_str=calculation_end_date_str
    else:
        calculation_end_date_month_boundary_date_str=last_day_of_previous_month.strftime('%Y-%m-%d')
    # calculation_end_date_for_others are for trailing and cumulative returns
    calculation_end_date_for_others_str=calculation_end_date_month_boundary_date_str

    '''  Handling Feb 29 of leap years.
    For leap years, to simiplify the calculation,  Feb 28 will be used to replace Feb 29 for
    for calculating returns.
    Therefore, if calculation_end_date_for_others_str is Feb 29, then replace 29 to 28 of calculation_end_date_for_others_str
    '''
    leap_year=False
    if (
        calculation_end_date_for_others_str[-5:] == '02-29'
    ):
        calculation_end_date_for_others_str = calculation_end_date_for_others_str[:-2] + '28'
        leap_year=True
    #................End

    # Check whether numebr 0, 1, 2, .. is selected for using a default ticker list
    integer_value=str_to_integer(tickers[0])
    if (integer_value >= 0 and integer_value <len(tickers_lists)):
        tickers=tickers_lists[integer_value]

    # if no tickers were set, display help information
    if len(tickers)==0:
        return help_info_str
    tmp_ticker_list=tickers
    tickers = [ticker.lower() for ticker in tmp_ticker_list]

    #*********************************************************************************
    # Calculating year-end prices, Annual, Trailing, Cumulative, and CAGR returns & generating html for display
    #
    # list of year-end prices of stocks
    output_string1= f"\nAdj Close Prices ($)\n"
    data_adj_close_year_end_df, data_adj_close_df = stock_prices_df(tickers, calculation_end_date_str)
    output_dataframe= data_adj_close_year_end_df
    output_html1=output_string1 + output_dataframe.to_html()

    #  Annual Total Return
    output_string = f"\nAnnual Total Return (%) as {calculation_end_date_str}\n"
    #output_dataframe = get_annual_returns_tickers_year_boundary_df(tickers, calculation_end_date_str)
    output_dataframe = get_annual_returns_year_end_df(data_adj_close_year_end_df, calculation_end_date_str)
    output_dataframe = output_dataframe.dropna(how='all')
    output_dataframe = output_dataframe.round(4)*100
    #output_dataframe.index=output_dataframe.index.date
    # Assuming your DataFrame is named output_dataframe
    last_date = output_dataframe.index[-1]
    output_dataframe = output_dataframe.rename(index={last_date: calculation_end_date_str})
    # Convert the DataFrame to HTML, Combine the expected string outputs
    output_html2 = output_string + output_dataframe.to_html()
    print("\ndebug2  output_dataframe\n", output_dataframe)

    # annual_returns  - at any given day, for calculating trailing and cumulative returns, not to be displayed
    #annual_returns_dataframe=get_annual_returns_tickers_df(tickers, calculation_end_date_for_others_str)
    annual_returns_dataframe=get_annual_returns_anyday_df(data_adj_close_df, calculation_end_date_str)
    print("\ndebug2-T  annual_returns_dataframe\n", annual_returns_dataframe)

    # Trailing Return
    if (leap_year):
        output_string3 = f"\nTrailing Total Return (%) as {calculation_end_date_for_others_str} (leap year: Feb 29 replaced by Feb 28 for approximation)\n"
    else:
        output_string3 = f"\nTrailing Total Return (%) as {calculation_end_date_for_others_str}\n"
    output_dataframe3=get_trailing_return_all(annual_returns_dataframe)
    output_dataframe3 = output_dataframe3.dropna(how='all')
    # Insert an empty to align the ticker symbols with annual return display
    output_dataframe3.insert(0, "--------", "      ")
    output_dataframe3.index.name="years"
    output_html3=output_string3 + output_dataframe3.to_html()
    #print("\ndebug3\n", output_string3, output_dataframe3)
    # Cumulative Return
    output_string4 = f"\nCumulative Return (%) as {calculation_end_date_for_others_str}\n"
    cumulative_return_all_dataframe=get_cumulative_return_all(annual_returns_dataframe)
    cumulative_return_all_dataframe = cumulative_return_all_dataframe.dropna(how='all')
    output_dataframe4=cumulative_return_all_dataframe.round(4)*100
    output_dataframe4.index.name="years"
    output_html4=output_string4 + output_dataframe4.to_html()

    # CAGR Return
    output_string5 = f"\nCompound Annual Growth Rate (CAGR) (%) as {calculation_end_date_for_others_str}\n"
    output_dataframe5=get_cagr_return_all (cumulative_return_all_dataframe)
    output_dataframe5=output_dataframe5.round(4)*100
    output_html5=output_string5 + output_dataframe5.to_html()
    output_html = output_html1 + output_html2 + output_html3 + output_html4
    return  output_html

# Gradio Web interface
with gr.Blocks() as web_block:

    chatbot = gr.Chatbot(height="500px")
    # Create a row element for the Textbox and Clear button
    with gr.Row():
        #msg = gr.Textbox(label="stock tickers input", scale=2, min_width=380)
        msg = gr.Textbox(show_label=False, scale=2, min_width=380)
        clear = gr.ClearButton([msg, chatbot], scale=0, min_width=50)

    def respond(message, chat_history):
        bot_message = calculation_response(message)
        chat_history.append((message, bot_message))
        return "", chat_history

    msg.submit(respond, # function
     [msg, chatbot],  # inputs of the function
     [msg, chatbot]   # outputs of the function
               )
web_block.launch()
#web_block.launch(debug=True)

# test cases:
# calculation_response("4")
#calculation_response("SPY")



## backup -example 9

In [ ]:
'''
Example 9 for using yfinance

Calculate annual, trailing, cumumlative, and CAGR returns for multiple stocks.
* The start date can be an arbitrary date.  The default is the current date.
* annual return is displayed from the default current day, or an arbitrary given
  day (except for Feb 29 for leap year)
  For leap years, use Feb 28 to replace Feb 29 as simplification & approximation
* trailing, cumumlative returns are currently displayed from the month boundary (last day of Month)
  prior to the given date.
* However, trailing, cumumlative returns can be displayed
  from any date, which can be not at the month boundary (last day of Month),
  by minor change of setting calculation_end_date_for_others_str = calculation_end_date_str.
  prior to the given date in the function "calculation_response(message, history)"

Author: Gang Luo
'''
script_version = '(2024-03-04.2)'
import gradio as gr
import yfinance as yf
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import pytz
DEBUG_ENABLED = True
#==============================================================================

print_yearly_total_return = True
num_years_calculation=52   # total years for calculation

# Define a list of years to calculate the trailing returns, cumulative returns, and so on
# remove the row of current year row since it is not a full year.
years_list = [1, 2, 3, 5, 10, 15, 20, 25, 30, 40, 50, 60]

# Set the stock tickers list
tickers_lists = [["qqq","hxq.to","spy", "vfv.to","xiu.to", "xbb.to","xcb.to","xhb.to"], #0  checking ETF
    ["qqq","spy", "vfv.to", "vgg.to", "zlu.to", "xiu.to", "vdy.to", "xfn.to", "ry.to", "td.to", "na.to",
      "slf.to", "gwo.to", "bce.to", "t.to", "rci-b.to", "enb.to", "trp.to", "zlb.to", "cp.to"], #1 main monitoring list
    ["qqq","spy","vfv.to", "xiu.to", "zeb.to", "xfn.to", "na.to","ry.to", "bmo.to","bns.to", "td.to", "cm.to", "cwb.to",
      "slf.to", "gwo.to", "bce.to", "t.to", "rci-b.to", "enb.to", "trp.to", "xdv.to","cdz.to","vdy.to", "xdiv.to"],  #2   financial  ETF & stocks
    ["^ndx","qqq","^GSPC","spy", "vfv.to", "vgg.to","zlu.to","msft","AAPL","goog","AMZN","NVDA","meta","tsla","BRK-A","shop.to","hxq.to"],   #3  US mega stocks + risky shopfy
    ["^DJI","dia","^GSPC","spy","voo","ivv", "tpu-u.to","vfv.to", "zsp.to","hxs.to","tpu.to","xus.to", "xsp.to",
      "^IXIC","^ndx", "qqq","hxq.to","^GSPTSE","xic.to","xiu.to", "HXT.TO", "TTP.TO","ZCN.TO", "xfn.to", "xit.to"], #4   indexes and index ETFs
    ["dia","^DJI","^GSPC","spy","vfv.to", "zsp.to","hxs.to","xus.to", "xsp.to",
      "^IXIC","qqq","hxq.to","^GSPTSE","xic.to","xiu.to", "HXT.TO", "xfn.to"], #5   indexes and typical index ETFs
    ["^IXIC","^ndx","ONEQ","CIBR","QQJG", "qqq", "tqqq", "spy", "vfv.to", "HXQ.to", "ZQQ.to", "XQQ.to", "QQC.to", "ZNQ.TO",
         "xiu.to", "xit.to"],  #6   Nasdaq ETF and TSX IT ETF
    ["qqq","tqqq","sqqq", "QLD", "spy", "spxu", "upro", "sso", "spxl","tecl"], #7 leveraged ETFs
    ["^IXIC","^DJI","^GSPC","^GSPTSE"], #8 testing
    ["vfv.to","spy"] #9 testing
]

#==============================================================================
# Part 1:
#  retrieve daily adjusted close prices of a list of tickers from yahoo finance
#  Generate the year-end adjusted close prices
#  return year-end adjusted close prices, and  daily adjusted close prices
def stock_prices_df(tickers_list, end_date_str):
    tickers_list_upper = [ticker.upper() for ticker in tickers_list]
    tickers_str = ", ".join(tickers_list_upper)
    try:
        '''
        'try' statement for handlingy the exception error  for yf.download
        '''
        # Download the historical data
        data = yf.download(tickers_str, period="max")
    except:
        return  pd.DataFrame()
    else:
        data_adj_close = data['Adj Close']

    # Filter out rows with dates newer than calculation_end_date
    data_adj_close = data_adj_close[data_adj_close.index <= end_date_str]
    #print("\nDebug- stock_prices_df\n", data_adj_close)

    # Rearrange columns based on the order in tickers_list_upper
    if len(tickers_list)>1:
        data_adj_close = data_adj_close.reindex(columns=tickers_list_upper)

    # needed this when having only a single ticker in the ticker list
    if len(tickers_list_upper)==1:
        data_adj_close = pd.DataFrame(data_adj_close)
        data_adj_close.rename(columns={'Adj Close': tickers_list_upper[0]}, inplace=True)

    data_adj_close.columns = map(str.lower, data_adj_close.columns) # must after  pd.DataFrame(data_adj_close)
    data_adj_close_year_end = data_adj_close.resample('A').ffill().round(2) # must before index changed to date

    data_adj_close.index=data_adj_close.index.date
    data_adj_close_year_end.index=data_adj_close_year_end.index.date

    last_date = data_adj_close.index[-1]
    data_adj_close_year_end = data_adj_close_year_end.rename(index={last_date: end_date_str})

    return data_adj_close_year_end, data_adj_close

#==============================================================================
# Part 2:  Calculate annual returns at year end, and at any given day (by calculation_end_date_str)
#
# annual return calculation can start at any given day
def get_annual_returns_anyday_df(daily_adj_close_df, calculation_end_date_str):

    calculation_end_date=pd.to_datetime(calculation_end_date_str).tz_localize('America/New_York')

    # Create a DataFrame with a complete date range
    date_range = pd.date_range(start=daily_adj_close_df.index.min(), end=daily_adj_close_df.index.max(), freq='D')

    complete_stock_history = pd.DataFrame(index=date_range)
    # Merge the complete DataFrame with the original stock_history
    complete_stock_history = complete_stock_history.merge(daily_adj_close_df, how='left', left_index=True, right_index=True)
    complete_stock_history = complete_stock_history.ffill()  # fill the newy added rows with previous day value
    '''
    Filter out the rows that matches the month and date of calculation_end_date, which are the ends of
    annual periods from the calculation_end_date.
    '''
    # Filter out rows with dates newer than calculation_end_date
    #filtered_stock_history = complete_stock_history[complete_stock_history.index <= calculation_end_date]
    # note" daily_adj_close_df satisfys daily_adj_close_df.index <= calculation_end_date
    filtered_stock_history = complete_stock_history
    #print(filtered_stock_history)
    target_month=filtered_stock_history.index.max().month
    target_day=filtered_stock_history.index.max().day
    #print("target_month", target_month, "target_day",target_day, "start_year", filtered_stock_history.index.max().year)
    annual_returns = filtered_stock_history[(filtered_stock_history.index.month == target_month)
           & (filtered_stock_history.index.day ==target_day)]
    annual_returns_percent = annual_returns.pct_change().dropna(how='all')

    annual_returns_df = pd.DataFrame(annual_returns_percent)
    #print("\ndebug-annual_returns_df\n", annual_returns_df)
    return annual_returns_df

# annual return calculation can start at year end
def get_annual_returns_year_end_df(data_adj_close_df, calculation_end_date_str):
    annual_returns_percent = data_adj_close_df.pct_change().dropna(how='all')
    return annual_returns_percent

#==============================================================================
# Part 3: calculate the annualized trailing total return from the data generated in step 1 & display
# Define a function to calculate the annualized trailing total return for a given number of years
def get_trailing_return(ticker, data, years):
    # Get the total return values for the last n years
    trailing_data = data[ticker].tail(years)
    # Check if there are empty values within years
    if trailing_data.isna().any():
        return np.nan
    # Check if there are valid total return values for all years
    if len(trailing_data) == years:
        # Convert the percentage strings to numeric values
        trailing_data = trailing_data.astype(str).str.replace('%', '').astype(float)
        """ Calculate the annualized trailing total return using the formula from Investopedia[^1^][1]:
            Annualized Return = [(1 + r1) * (1 + r2) * ... * (1 + rn)]^(1/n) - 1
            Where r1, r2, ..., rn are the total return values for each year                    """
        annualized_trailing_return = (trailing_data + 1).prod() ** (1 / years) - 1

        # Format the result as a percentage with two decimal places
        annualized_trailing_return = annualized_trailing_return * 100
        annualized_trailing_return = annualized_trailing_return.round(2)
        return annualized_trailing_return
    else:
        return np.nan

# Define a function to Loop through the list and print the trailing returns for each num_years
def get_trailing_return_column(ticker, annual_returns_df):
    trailing_return_column = {}
    for num_years in years_list:
        # Check if the ticker data is available in all_tickers_returns_df
        if ticker in annual_returns_df.columns:
            # using data from step 1, avoiding get_annual_returns_df(ticker) for less traffic from yahoo server
            data = annual_returns_df[[ticker]]
            trailing_return = get_trailing_return(ticker, data, num_years)
            trailing_return_column[f"{num_years}-Year"] = trailing_return
        else:
            print(f"Data not available for {ticker}. Skipping.")
            trailing_return_column[f"{num_years}-Year"] = np.nan
    return trailing_return_column

# Create an empty DataFrame to store all tickers' trailing returns
def get_trailing_return_all(annual_returns_df):
    all_tickers_trailing_returns_df = pd.DataFrame(index=years_list)
    tickers=annual_returns_df.columns.tolist()
    # Loop through each ticker in the list
    for ticker in tickers:
        trailing_returns = get_trailing_return_column(ticker, annual_returns_df)
        # Add the trailing returns to the DataFrame
        all_tickers_trailing_returns_df[ticker] = pd.Series(trailing_returns).values
    return all_tickers_trailing_returns_df

#==============================================================================
# Part 4: calculate the cumulative return from the data (all_tickers_returns_df) generated in part 1 & display
#  Define a function to calculate the cumulative return for a given number of years from a ticker
def get_cumulative_return(ticker, data, years):
    # Calculate the cumulative return
    cumulative_return = (1 + data[ticker]).rolling(window=years).apply(lambda x: x.prod(), raw=True) - 1
    return cumulative_return

# Define a function to Loop through the list and return the cumulative returns for each num_years
def get_cumulative_return_column(ticker, annual_returns_df):
    cumulative_returns = {}
    for years in years_list:
        # Calculate the cumulative return for the given number of years
        cumulative_return = get_cumulative_return(ticker, annual_returns_df, years)
        # Get the last value, which is the cumulative return up to the current year
        cumulative_returns[years] = cumulative_return.iloc[-1]
    return cumulative_returns

def get_cumulative_return_all(annual_returns_df):
    # Create an empty DataFrame with years_list as the index for cumulative  returns
    all_tickers_cumulative_returns_df = pd.DataFrame(index=years_list)
    tickers=annual_returns_df.columns.tolist()
    # Loop through each ticker in the list
    for ticker in tickers:
        cumulative_returns = get_cumulative_return_column(ticker, annual_returns_df)
        # Add the trailing returns to the DataFrame
        all_tickers_cumulative_returns_df[ticker] = pd.Series(cumulative_returns).values
    return all_tickers_cumulative_returns_df

#==============================================================================
# Part 5: calculate the  CAGR (Compound Annual Growth Rate) from the data
# in all_tickers_cumulative_returns_df generated earlier & display
# Define a function to calculate the CAGR from the cumulative value and the years
def calculate_cagr(value, years):
    # Otherwise, calculate the CAGR using the formula
    cagr = (value + 1) ** (1 / np.array(years)) - 1
    #print("debug-cagr\n", cagr, "end")
    return cagr

# Define a function to format the Float64Index values into percentage strings
def format_to_percentage(value):
    # If any element in the value array is not null, format it as a percentage string with two decimal places
    if np.any(pd.notnull(value)):
        return f"{value:.2f}%"
    # Otherwise, return None
    return None

def get_cagr_return_all(all_tickers_cumulative_returns_df):
    # Apply the calculate_cagr function to each column of the DataFrame
    all_tickers_cagrs_df = all_tickers_cumulative_returns_df.apply(lambda x: calculate_cagr(x, x.index), axis=0)
    return all_tickers_cagrs_df

#==============================================================================
# Part 6:
# single ticker's Prices, Returns,Dividends, good for verifying whether "Adj Close" is correct.
'''
    Calculate and display: yearly dividendSum, 'Close' & 'Adj Close' prices,
    Return(by 'Close' price), total return(by 'Adj Close' price),
    CalReturn(total return by 'Close' price and "dividendSum).
    Note: CalReturn from  is expected to be nearly same as  total return,
           when the 'Adj Close' price is correct.
'''
def get_yearly_single_stock_data(ticker):
    stock = yf.Ticker(ticker)
    #-------- mainly for downloading 'Dividends'
    history = stock.history(period="max")
    dividend_history=history['Dividends']
    dividend_history.index=dividend_history.index.date

    #-------- mainly for downloading 'Close','Adj Close'
    dld_history=yf.download(ticker, period="max")
    dld_history=dld_history[['Close','Adj Close']]
    dld_history.rename(columns={'Adj Close': 'AdjClose'}, inplace=True)
    date_range = pd.date_range(start=dld_history.index.min(), end=dld_history.index.max(), freq='D')
    complete_history = pd.DataFrame(index=date_range)

    # Merge the complete DataFrame with the original stock_history
    complete_history = complete_history.merge(dld_history, how='left', left_index=True, right_index=True)
    complete_history[['Close','AdjClose']] = complete_history[['Close','AdjClose']].ffill().round(3)

    # Merge dividend into complete_history
    complete_history = complete_history.merge(dividend_history, how='left', left_index=True, right_index=True)
    # replace all NaN values in the 'Dividends' column with 0.0
    complete_history['Dividends'] = complete_history['Dividends'].fillna(0.0).round(3)

    complete_history['Year']=complete_history.index.year
    complete_history['Date']=complete_history.index
    yearly_data = complete_history.groupby('Year').agg({'Date': 'last', 'Close': 'last', 'AdjClose': 'last','Dividends': 'sum'})
    yearly_data.rename(columns={'Dividends': 'DivSum'}, inplace=True)

    # calculating 'Return' and 'TotalReturn'
    yearly_data['DivRatio']=yearly_data['DivSum'] / yearly_data['Close']
    yearly_data['Return']=yearly_data['Close'].pct_change()
    yearly_data['TotalReturn']=yearly_data['AdjClose'].pct_change()

    '''
    The CalReturn column is the yearly total return calculated from un-adjusted "Close" prices and yearly "dividend sum",
    which is expected to be equal to the total return that is calculated from "AdjClose" prices
    '''
    yearly_data['CalReturn'] = (yearly_data['Close'] + yearly_data['DivSum']) / yearly_data['Close'].shift(1) - 1
    # set the display format
    yearly_data[['DivRatio','Return','TotalReturn','CalReturn']] = yearly_data[['DivRatio','Return','TotalReturn','CalReturn']].mul(100).round(2)
    '''
    #yearly_data[['DivRatio','Return', 'TotalReturn', 'CalReturn']] = yearly_data[['DivRatio','Return', 'TotalReturn', 'CalReturn']].applymap("{:.2f}%".format)
    yearly_data[['DivRatio','Return', 'TotalReturn', 'CalReturn']]= \
     yearly_data[['DivRatio','Return', 'TotalReturn', 'CalReturn']].applymap(lambda x: f"{x:.2f}%" if not pd.isna(x) else "NaN")
    '''
    # Use .applymap() and lambda to format the values as percentage strings only if they are not NaN
    yearly_data[['DivRatio','Return', 'TotalReturn', 'CalReturn']]= \
         yearly_data[['DivRatio','Return', 'TotalReturn', 'CalReturn']].applymap(lambda x: f"{x:.2f}%" if not pd.isna(x) else x)

    # 'Date' column is no longer required
    yearly_data.drop('Date', axis=1, inplace=True)
    return yearly_data

#==============================================================================
# Part 7: utility functions
# get the last trading day of S&P 500 in string format
def get_last_trading_day():
    # Get today's date, use .strftime("%Y-%m-%d") to convert to a string
    today_date_str=datetime.now(pytz.timezone('America/New_York')).date().strftime("%Y-%m-%d")
    stock = yf.Ticker("^GSPC") # S&P 500 (^GSPC) ticker
    #  search and see yfinance_BUG_1 NOTE in this file
    history_df=stock.history(period="max", end=today_date_str)["Close"]
    last_trading_day_str = history_df.index.max().date().strftime("%Y-%m-%d")
    return last_trading_day_str

def str_to_integer(integer_str):
    try:
        integer_number = int(integer_str)
        return integer_number
    except ValueError:
        return -1

# validate the date string
def is_valid_date(date_string):
    try:
        # Attempt to parse the date string
        datetime.strptime(date_string, "%Y-%m-%d")
        return True
    except ValueError:
        # Raised when the date string is not in the expected format
        return False

def date_label_conversion_strip_time(all_tickers_returns_df, calculation_end_date_str):
    all_tickers_returns_df.index=all_tickers_returns_df.index.date
    all_tickers_returns_df.index.name='date'
    # print("debug get_annual_returns_tickers_df", all_tickers_returns_df)
    # Convert calculation_end_date_str to a datetime object, replace the index's mon/day portion of date
    end_date_datetime_obj = datetime.strptime(calculation_end_date_str, "%Y-%m-%d")
    all_tickers_returns_df.index = all_tickers_returns_df.index.map(
      lambda x: x.replace(month=end_date_datetime_obj.month,
      day=end_date_datetime_obj.day))
    return all_tickers_returns_df

#==============================================================================
# Part 8: gradio handling - Input command handling and display in web page

help_info_str="Input Formats:\n  \
            1. ticker list....................Example:  spy vfv.to xiu.to xic.to xfn.to ry.to \n \
            2. One of default ticker list, a number between 1 and 7....Example:   0, or 1, ...,7 \n \
            3. CalculationEndDate as prefix.  Example:   2020-12-31 2 \n \
            .........................................2020-12-31 spy vfv.to xiu.to xic.to xfn.to ry.to \n \
            4. single ticker: Dividend/Close/AdjClose/Return/TotalReturn/CalReturn(by Close/Dividends).  @1 spy \n \
            note: daily adjusted close data are from Yahoo Finance. "

# Main Handling Process
def calculation_response(message):
    # if there is no input, display help information
    if message=="":
        return help_info_str

    tickers=message.split()

    # ******************************************************************************
    #  processing web input parameters
    #  set calculation_end_date_str, and tickers

    #---------------------------------------------------------
    # single stock ticker - detailed information
    if (tickers[0] == "@1"):
        tickers.pop(0) # remove the first string which is "@1"
        if len(tickers)==0:
            ticker = 'spy' # default ticker = spy
        else:
            ticker=tickers[0]
        output_string=f"\n {ticker}\n"
        output_dataframe0=get_yearly_single_stock_data(ticker)
        output_html=output_string + output_dataframe0.to_html()
        return output_html

    #----------------------------------------------------------
    # Get today's date, use .strftime("%Y-%m-%d") to convert to a string
    #calculation_end_date_str=datetime.now(pytz.timezone('America/New_York')).date().strftime("%Y-%m-%d")
    calculation_end_date_str = get_last_trading_day()
    # Check whether the first str is date for calculation end date
    if is_valid_date(tickers[0]):
        calculation_end_date_str = tickers[0] # reset calculation_end_date_str
        tickers.pop(0) # remove the first string which is the date

    #............ For display trailing and cumulative returns at month_boundary_date
    # Assuming calculation_end_date_str contains the date string '2024-01-03'
    calculation_end_date = datetime.strptime(calculation_end_date_str, '%Y-%m-%d')
    # Calculate the first day of the current month
    first_day_of_month = calculation_end_date.replace(day=1)
    # Calculate the last day of the month
    last_day_of_month = (calculation_end_date.replace(day=1) + timedelta(days=32)).replace(day=1) - timedelta(days=1)
    # Calculate the last day of the previous month
    last_day_of_previous_month = first_day_of_month - timedelta(days=1)
    # Check if the original date is the last day of the month
    if (calculation_end_date == last_day_of_month):
        calculation_end_date_month_boundary_date_str=calculation_end_date_str
    else:
        calculation_end_date_month_boundary_date_str=last_day_of_previous_month.strftime('%Y-%m-%d')
    # calculation_end_date_for_others are for trailing and cumulative returns
    calculation_end_date_for_others_str=calculation_end_date_month_boundary_date_str

    '''  Handling Feb 29 of leap years.
    For leap years, to simiplify the calculation,  Feb 28 will be used to replace Feb 29 for
    for calculating returns.
    Therefore, if calculation_end_date_for_others_str is Feb 29, then replace 29 to 28 of calculation_end_date_for_others_str
    '''
    leap_year=False
    if (
        calculation_end_date_for_others_str[-5:] == '02-29'
    ):
        calculation_end_date_for_others_str = calculation_end_date_for_others_str[:-2] + '28'
        leap_year=True
    #................End

    # Check whether numebr 0, 1, 2, .. is selected for using a default ticker list
    integer_value=str_to_integer(tickers[0])
    if (integer_value >= 0 and integer_value <len(tickers_lists)):
        tickers=tickers_lists[integer_value]

    # if no tickers were set, display help information
    if len(tickers)==0:
        return help_info_str
    tmp_ticker_list=tickers
    tickers = [ticker.lower() for ticker in tmp_ticker_list]

    #*********************************************************************************
    # Calculating year-end prices, Annual, Trailing, Cumulative, and CAGR returns & generating html for display
    #
    # list of year-end prices of stocks
    output_string1= f"\nAdj Close Prices ($)\n"
    data_adj_close_year_end_df, data_adj_close_df = stock_prices_df(tickers, calculation_end_date_str)
    output_dataframe= data_adj_close_year_end_df
    output_html1=output_string1 + output_dataframe.to_html()

    #  Annual Total Return
    output_string = f"\nAnnual Total Return (%) as {calculation_end_date_str}\n"
    #output_dataframe = get_annual_returns_tickers_year_boundary_df(tickers, calculation_end_date_str)
    output_dataframe = get_annual_returns_year_end_df(data_adj_close_year_end_df, calculation_end_date_str)
    output_dataframe = output_dataframe.dropna(how='all')
    output_dataframe = output_dataframe.round(4)*100
    #output_dataframe.index=output_dataframe.index.date
    # Assuming your DataFrame is named output_dataframe
    last_date = output_dataframe.index[-1]
    output_dataframe = output_dataframe.rename(index={last_date: calculation_end_date_str})
    # Convert the DataFrame to HTML, Combine the expected string outputs
    output_html2 = output_string + output_dataframe.to_html()
    print("\ndebug2  output_dataframe\n", output_dataframe)

    # annual_returns  - at any given day, for calculating trailing and cumulative returns, not to be displayed
    #annual_returns_dataframe=get_annual_returns_tickers_df(tickers, calculation_end_date_for_others_str)
    annual_returns_dataframe=get_annual_returns_anyday_df(data_adj_close_df, calculation_end_date_str)
    print("\ndebug2-T  annual_returns_dataframe\n", annual_returns_dataframe)

    # Trailing Return
    if (leap_year):
        output_string3 = f"\nTrailing Total Return (%) as {calculation_end_date_for_others_str} (leap year: Feb 29 replaced by Feb 28 for approximation)\n"
    else:
        output_string3 = f"\nTrailing Total Return (%) as {calculation_end_date_for_others_str}\n"
    output_dataframe3=get_trailing_return_all(annual_returns_dataframe)
    output_dataframe3 = output_dataframe3.dropna(how='all')
    # Insert an empty to align the ticker symbols with annual return display
    output_dataframe3.insert(0, "--------", "      ")
    output_dataframe3.index.name="years"
    output_html3=output_string3 + output_dataframe3.to_html()
    #print("\ndebug3\n", output_string3, output_dataframe3)
    # Cumulative Return
    output_string4 = f"\nCumulative Return (%) as {calculation_end_date_for_others_str}\n"
    cumulative_return_all_dataframe=get_cumulative_return_all(annual_returns_dataframe)
    cumulative_return_all_dataframe = cumulative_return_all_dataframe.dropna(how='all')
    output_dataframe4=cumulative_return_all_dataframe.round(4)*100
    output_dataframe4.index.name="years"
    output_html4=output_string4 + output_dataframe4.to_html()

    # CAGR Return
    output_string5 = f"\nCompound Annual Growth Rate (CAGR) (%) as {calculation_end_date_for_others_str}\n"
    output_dataframe5=get_cagr_return_all (cumulative_return_all_dataframe)
    output_dataframe5=output_dataframe5.round(4)*100
    output_html5=output_string5 + output_dataframe5.to_html()
    output_html = output_html1 + output_html2 + output_html3 + output_html4
    return  output_html

# Gradio Web interface
with gr.Blocks() as web_block:

    chatbot = gr.Chatbot(height="500px")
    # Create a row element for the Textbox and Clear button
    with gr.Row():
        #msg = gr.Textbox(label="stock tickers input", scale=2, min_width=380)
        msg = gr.Textbox(show_label=False, scale=2, min_width=380)
        clear = gr.ClearButton([msg, chatbot], scale=0, min_width=50)

    def respond(message, chat_history):
        bot_message = calculation_response(message)
        chat_history.append((message, bot_message))
        return "", chat_history

    msg.submit(respond, # function
     [msg, chatbot],  # inputs of the function
     [msg, chatbot]   # outputs of the function
               )
web_block.launch()
#web_block.launch(debug=True)

# test cases:
# calculation_response("4")
#calculation_response("SPY")



# Example 8 = Example 7 improvement- annualy for arbitrary start date (multiple stocks) annual,  trailing, cumumlative, and CAGR  returns (not neccesry only for calender year boundary )


In [ ]:
'''
Example 8 for using yfinance

Calculate annual, trailing, cumumlative, and CAGR returns for multiple stocks.
* The start date can be an arbitrary date.  The default is the current date.
* annual return is displayed from the default current day, or an arbitrary given
  day (except for Feb 29 for leap year)
  For leap years, use Feb 28 to replace Feb 29 as simplification & approximation
* trailing, cumumlative returns are currently displayed from the month boundary (last day of Month)
  prior to the given date.
* However, trailing, cumumlative returns can be displayed
  from any date, which can be not at the month boundary (last day of Month),
  by minor change of setting calculation_end_date_for_others_str = calculation_end_date_str.
  prior to the given date in the function "calculation_response(message, history)"

Author: Gang Luo
'''
script_version = '(2024-03-02.1)'
import gradio as gr
import yfinance as yf
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import pytz
#==============================================================================

print_yearly_total_return = True
num_years_calculation=52   # total years for calculation

# Define a list of years to calculate the trailing returns, cumulative returns, and so on
# remove the row of current year row since it is not a full year.
years_list = [1, 2, 3, 5, 10, 15, 20, 25, 30, 40, 50, 60]

# Set the stock tickers list
tickers_lists = [["qqq","hxq.to","spy", "vfv.to","xiu.to", "xbb.to","xcb.to","xhb.to"], #0  checking ETF
    ["qqq","spy", "vfv.to", "vgg.to", "zlu.to", "xiu.to", "vdy.to", "xfn.to", "ry.to", "td.to", "na.to",
      "slf.to", "gwo.to", "bce.to", "t.to", "rci-b.to", "enb.to", "trp.to", "zlb.to", "cp.to"], #1 main monitoring list
    ["qqq","spy","vfv.to", "xiu.to", "zeb.to", "xfn.to", "na.to","ry.to", "bmo.to","bns.to", "td.to", "cm.to", "cwb.to",
      "slf.to", "gwo.to", "bce.to", "t.to", "rci-b.to", "enb.to", "trp.to", "xdv.to","cdz.to","vdy.to", "xdiv.to"],  #2   financial  ETF & stocks
    ["^ndx","qqq","^GSPC","spy", "vfv.to", "vgg.to","zlu.to","msft","AAPL","goog","AMZN","NVDA","meta","tsla","BRK-A","shop.to","hxq.to"],   #3  US mega stocks + risky shopfy
    ["^DJI","dia","^GSPC","spy","voo","ivv", "tpu-u.to","vfv.to", "zsp.to","hxs.to","tpu.to","xus.to", "xsp.to",
      "^IXIC","^ndx", "qqq","hxq.to","^GSPTSE","xic.to","xiu.to", "HXT.TO", "TTP.TO","ZCN.TO", "xfn.to", "xit.to"], #4   indexes and index ETFs
    ["dia","^DJI","^GSPC","spy","vfv.to", "zsp.to","hxs.to","xus.to", "xsp.to",
      "^IXIC","qqq","hxq.to","^GSPTSE","xic.to","xiu.to", "HXT.TO", "xfn.to"], #5   indexes and typical index ETFs
    ["^IXIC","^ndx","ONEQ","CIBR","QQJG", "qqq", "tqqq", "spy", "vfv.to", "HXQ.to", "ZQQ.to", "XQQ.to", "QQC.to", "ZNQ.TO",
         "xiu.to", "xit.to"],  #6   Nasdaq ETF and TSX IT ETF
    ["qqq","tqqq","sqqq", "QLD", "spy", "spxu", "upro", "sso", "spxl","tecl"], #7 leveraged ETFs
    ["^IXIC","^DJI","^GSPC","^GSPTSE"] #8 testing
]

#==============================================================================
# Part 1: fetch retrieve yearly total returns by yfinance & display
# Function to fetch data from yfinance and extract yearly total returns#
# annual return calculation can start at any given day
def get_annual_returns_df(ticker, calculation_end_date_str):
    # Get the historical data for the given ticker
    stock = yf.Ticker(ticker)
    calculation_end_date=pd.to_datetime(calculation_end_date_str).tz_localize('America/New_York')
    try:
        '''
        'try' statement for handlingy the exception error of stock.history that a ticker is not yet at stock market,
        For example, "shop.to" is not there in 2012
        '''
        stock_history=stock.history(period="max")["Close"]
        '''
        Between the start and end days in stock_history variable, there are some missing days where there are no corresponding rows.
        Add rows of  missing  days such that the values of column "Close" are set to be the value of the closest earlier day's
        value, by using date_range to create full range without any missing date.
        '''
        # Create a DataFrame with a complete date range
        date_range = pd.date_range(start=stock_history.index.min(), end=stock_history.index.max(), freq='D')
        complete_stock_history = pd.DataFrame(index=date_range)
        # Merge the complete DataFrame with the original stock_history
        complete_stock_history = complete_stock_history.merge(stock_history, how='left', left_index=True, right_index=True)
        complete_stock_history['Close'] = complete_stock_history['Close'].ffill()  # fill the newy added rows with previous day value
        '''
        Filter out the rows that matches the month and date of calculation_end_date, which are the ends of
        annual periods from the calculation_end_date.
        '''
        # Filter out rows with dates newer than calculation_end_date
        filtered_stock_history = complete_stock_history[complete_stock_history.index <= calculation_end_date]
        #print(filtered_stock_history)
        target_month=filtered_stock_history.index.max().month
        target_day=filtered_stock_history.index.max().day
        #print("target_month", target_month, "target_day",target_day, "start_year", filtered_stock_history.index.max().year)
        annual_returns = filtered_stock_history[(filtered_stock_history.index.month == target_month)
           & (filtered_stock_history.index.day ==target_day)]
        annual_returns_percent = annual_returns.pct_change().dropna()
    except:
        return  pd.DataFrame()
    else:
        annual_returns_df = pd.DataFrame(annual_returns_percent, columns=['Close'])
        annual_returns_df.rename(columns={'Close': ticker}, inplace=True)
        return annual_returns_df

# Function to fetch data from yfinance and extract yearly total returns
# annual return calculation starts at only yaer end boundary, i.e, Dec 31,
# by resample('A')
def get_annual_returns_year_boundary_df(ticker, calculation_end_date_str):
    # Get the historical data for the given ticker
    stock = yf.Ticker(ticker)
    calculation_end_date = datetime.strptime(calculation_end_date_str, "%Y-%m-%d")
    calculation_start_date_str = (calculation_end_date
                - timedelta(days=num_years_calculation * 365)).strftime("%Y-%m-%d")

    try:
        '''
        1.  'try' statement for handlingy the exception error of stock.history that a ticker is not yet at stock market,
             For example, "shop.to" is not there in 2012
        2. The row with the latest day from .history(.., end='end_day_date') is the day prior to end_day_date.  Therefore,
           let end=the expected end day plus one day.
        '''
        calculation_end_date_plus_1day_str =  (calculation_end_date + timedelta(days=1)).strftime("%Y-%m-%d")
        annual_returns_history=stock.history(start=calculation_start_date_str,end=calculation_end_date_plus_1day_str)["Close"]

        #print("debug get_annual_returns_df ", ticker, annual_returns_history)
        # For 'A', 'Y', see https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html#offset-aliases
        ffilled_history=annual_returns = annual_returns_history.resample('A').ffill()
        #print(ffilled_history)
        annual_returns = ffilled_history.pct_change().dropna()
        #annual_returns = annual_returns_history.resample('A').ffill().pct_change().dropna()
        #print("debug get_annual_returns_df after resample()", ticker, calculation_end_date, "\n", annual_returns)
    except:
        return  pd.DataFrame()
    else:
        annual_returns_df = pd.DataFrame(annual_returns, columns=['Close'])
        annual_returns_df.rename(columns={'Close': ticker}, inplace=True)
        return annual_returns_df

#----------------------------------------------------------------------------------
# handling a list of tickers by calling the functions (either get_annual_returns_df
# get_annual_returns_year_boundary_df) that handle single tickers
def get_annual_returns_tickers_common_df(tickers, calculation_end_date_str, annual_returns_func_df):
    # Create an empty DataFrame to store all tickers' total returns
    all_tickers_returns_df = pd.DataFrame()

    # Loop through each ticker in the list
    for ticker in tickers:
        ticker_returns_df = annual_returns_func_df(ticker, calculation_end_date_str)
        if not ticker_returns_df.empty:
            if all_tickers_returns_df.empty:
                all_tickers_returns_df = ticker_returns_df
            else:
                '''
                When running in  huggingface, pd.concat changed the index order of ticker_returns_df
                when ticker_returns_df has more rows than all_tickers_returns_df. However, it is ok
                running in colab.  Therefore, use pd.merge to replace pd.concat.
                all_tickers_returns_df = pd.concat([all_tickers_returns_df, ticker_returns_df],axis=1,join='outer')  # Concatenate DataFrames
                all_tickers_returns_df.sort_index() # index may be changed when running in huggingface
                '''
                all_tickers_returns_df = pd.merge(all_tickers_returns_df, ticker_returns_df,
                        left_index=True, right_index=True, how='outer')
        else:
            # New column with NaN values
            new_column_name = ticker
            new_column_values = [None] * len(all_tickers_returns_df)
            new_column = pd.DataFrame({new_column_name: new_column_values}, index=all_tickers_returns_df.index)
            # Concatenate the new column to the original DataFrame
            all_tickers_returns_df = pd.concat([all_tickers_returns_df, new_column], axis=1)
    #return date_label_conversion_strip_time(all_tickers_returns_df, calculation_end_date_str)
    return all_tickers_returns_df

def get_annual_returns_tickers_df(tickers, calculation_end_date_str):
    return get_annual_returns_tickers_common_df(tickers, calculation_end_date_str,
                                                get_annual_returns_df)

def get_annual_returns_tickers_year_boundary_df(tickers, calculation_end_date_str):
    return get_annual_returns_tickers_common_df(tickers, calculation_end_date_str,
                                                get_annual_returns_year_boundary_df)

#==============================================================================
# Part 2: calculate the annualized trailing total return from the data generated in step 1 & display
# Define a function to calculate the annualized trailing total return for a given number of years
def get_trailing_return(ticker, data, years):
    # Get the total return values for the last n years
    trailing_data = data[ticker].tail(years)
    # Check if there are empty values within years
    if trailing_data.isna().any():
        return np.nan
    # Check if there are valid total return values for all years
    if len(trailing_data) == years:
        # Convert the percentage strings to numeric values
        trailing_data = trailing_data.astype(str).str.replace('%', '').astype(float)
        """ Calculate the annualized trailing total return using the formula from Investopedia[^1^][1]:
            Annualized Return = [(1 + r1) * (1 + r2) * ... * (1 + rn)]^(1/n) - 1
            Where r1, r2, ..., rn are the total return values for each year                    """
        annualized_trailing_return = (trailing_data + 1).prod() ** (1 / years) - 1

        # Format the result as a percentage with two decimal places
        annualized_trailing_return = annualized_trailing_return * 100
        annualized_trailing_return = annualized_trailing_return.round(2)
        return annualized_trailing_return
    else:
        return np.nan

# Define a function to Loop through the list and print the trailing returns for each num_years
def get_trailing_return_column(ticker, annual_returns_df):
    trailing_return_column = {}
    for num_years in years_list:
        # Check if the ticker data is available in all_tickers_returns_df
        if ticker in annual_returns_df.columns:
            # using data from step 1, avoiding get_annual_returns_df(ticker) for less traffic from yahoo server
            data = annual_returns_df[[ticker]]
            trailing_return = get_trailing_return(ticker, data, num_years)
            trailing_return_column[f"{num_years}-Year"] = trailing_return
        else:
            print(f"Data not available for {ticker}. Skipping.")
            trailing_return_column[f"{num_years}-Year"] = np.nan
    return trailing_return_column

# Create an empty DataFrame to store all tickers' trailing returns
def get_trailing_return_all(tickers, annual_returns_df):
    all_tickers_trailing_returns_df = pd.DataFrame(index=years_list)

    # Loop through each ticker in the list
    for ticker in tickers:
        trailing_returns = get_trailing_return_column(ticker, annual_returns_df)
        # Add the trailing returns to the DataFrame
        all_tickers_trailing_returns_df[ticker] = pd.Series(trailing_returns).values
    all_tickers_trailing_returns_df.columns = map(str.lower, all_tickers_trailing_returns_df.columns)
    return all_tickers_trailing_returns_df

#==============================================================================
# Part 3: calculate the cumulative return from the data (all_tickers_returns_df) generated in part 1 & display
#  Define a function to calculate the cumulative return for a given number of years from a ticker
def get_cumulative_return(ticker, data, years):
    # Calculate the cumulative return
    cumulative_return = (1 + data[ticker]).rolling(window=years).apply(lambda x: x.prod(), raw=True) - 1
    return cumulative_return

# Define a function to Loop through the list and return the cumulative returns for each num_years
def get_cumulative_return_column(ticker, annual_returns_df):
    cumulative_returns = {}
    for years in years_list:
        # Calculate the cumulative return for the given number of years
        cumulative_return = get_cumulative_return(ticker, annual_returns_df, years)
        # Get the last value, which is the cumulative return up to the current year
        cumulative_returns[years] = cumulative_return.iloc[-1]
    return cumulative_returns

def get_cumulative_return_all(tickers, annual_returns_df):
    # Create an empty DataFrame with years_list as the index for cumulative  returns
    all_tickers_cumulative_returns_df = pd.DataFrame(index=years_list)
    # Loop through each ticker in the list
    for ticker in tickers:
        cumulative_returns = get_cumulative_return_column(ticker, annual_returns_df)
        # Add the trailing returns to the DataFrame
        all_tickers_cumulative_returns_df[ticker] = pd.Series(cumulative_returns).values
    all_tickers_cumulative_returns_df.columns = map(str.lower, all_tickers_cumulative_returns_df.columns)
    return all_tickers_cumulative_returns_df

#==============================================================================
# Part 4: calculate the  CAGR (Compound Annual Growth Rate) from the data
# in all_tickers_cumulative_returns_df generated earlier & display
# Define a function to calculate the CAGR from the cumulative value and the years
def calculate_cagr(value, years):
    # Otherwise, calculate the CAGR using the formula
    cagr = (value + 1) ** (1 / np.array(years)) - 1
    #print("debug-cagr\n", cagr, "end")
    return cagr

# Define a function to format the Float64Index values into percentage strings
def format_to_percentage(value):
    # If any element in the value array is not null, format it as a percentage string with two decimal places
    if np.any(pd.notnull(value)):
        return f"{value:.2f}%"
    # Otherwise, return None
    return None

def get_cagr_return_all(all_tickers_cumulative_returns_df):
    # Apply the calculate_cagr function to each column of the DataFrame
    all_tickers_cagrs_df = all_tickers_cumulative_returns_df.apply(lambda x: calculate_cagr(x, x.index), axis=0)
    return all_tickers_cagrs_df

#==============================================================================
# Part 5: utility functions
# get the last trading day of S&P 500 in string format
def get_last_trading_day():
    # Get today's date, use .strftime("%Y-%m-%d") to convert to a string
    today_date_str=datetime.now(pytz.timezone('America/New_York')).date().strftime("%Y-%m-%d")
    stock = yf.Ticker("^GSPC") # S&P 500 (^GSPC) ticker
    #  search and see yfinance_BUG_1 NOTE in this file
    history_df=stock.history(period="max", end=today_date_str)["Close"]
    last_trading_day_str = history_df.index.max().date().strftime("%Y-%m-%d")
    return last_trading_day_str

def str_to_integer(integer_str):
    try:
        integer_number = int(integer_str)
        return integer_number
    except ValueError:
        return -1

# validate the date string
def is_valid_date(date_string):
    try:
        # Attempt to parse the date string
        datetime.strptime(date_string, "%Y-%m-%d")
        return True
    except ValueError:
        # Raised when the date string is not in the expected format
        return False

def date_label_conversion_strip_time(all_tickers_returns_df, calculation_end_date_str):
    all_tickers_returns_df.index=all_tickers_returns_df.index.date
    all_tickers_returns_df.index.name='date'
    # print("debug get_annual_returns_tickers_df", all_tickers_returns_df)
    # Convert calculation_end_date_str to a datetime object, replace the index's mon/day portion of date
    end_date_datetime_obj = datetime.strptime(calculation_end_date_str, "%Y-%m-%d")
    all_tickers_returns_df.index = all_tickers_returns_df.index.map(
      lambda x: x.replace(month=end_date_datetime_obj.month,
      day=end_date_datetime_obj.day))
    return all_tickers_returns_df

#==============================================================================
# Part 6:
# single ticker's Prices, Returns,Dividends, good for verifying whether "Adj Close" is correct.
'''
    Calculate and display: yearly dividendSum, 'Close' & 'Adj Close' prices,
    Return(by 'Close' price), total return(by 'Adj Close' price),
    CalReturn(total return by 'Close' price and "dividendSum).
    Note: CalReturn from  is expected to be nearly same as  total return,
           when the 'Adj Close' price is correct.
'''
def get_yearly_single_stock_data(ticker):
    stock = yf.Ticker(ticker)
    #-------- mainly for downloading 'Dividends'
    history = stock.history(period="max")
    dividend_history=history['Dividends']
    dividend_history.index=dividend_history.index.date

    #-------- mainly for downloading 'Close','Adj Close'
    dld_history=yf.download(ticker, period="max")
    dld_history=dld_history[['Close','Adj Close']]
    dld_history.rename(columns={'Adj Close': 'AdjClose'}, inplace=True)
    date_range = pd.date_range(start=dld_history.index.min(), end=dld_history.index.max(), freq='D')
    complete_history = pd.DataFrame(index=date_range)

    # Merge the complete DataFrame with the original stock_history
    complete_history = complete_history.merge(dld_history, how='left', left_index=True, right_index=True)
    complete_history[['Close','AdjClose']] = complete_history[['Close','AdjClose']].ffill().round(3)

    # Merge dividend into complete_history
    complete_history = complete_history.merge(dividend_history, how='left', left_index=True, right_index=True)
    # replace all NaN values in the 'Dividends' column with 0.0
    complete_history['Dividends'] = complete_history['Dividends'].fillna(0.0).round(3)

    complete_history['Year']=complete_history.index.year
    complete_history['Date']=complete_history.index
    yearly_data = complete_history.groupby('Year').agg({'Date': 'last', 'Close': 'last', 'AdjClose': 'last','Dividends': 'sum'})
    yearly_data.rename(columns={'Dividends': 'DivSum'}, inplace=True)

    # calculating 'Return' and 'TotalReturn'
    yearly_data['DivRatio']=yearly_data['DivSum'] / yearly_data['Close']
    yearly_data['Return']=yearly_data['Close'].pct_change()
    yearly_data['TotalReturn']=yearly_data['AdjClose'].pct_change()

    '''
    The CalReturn column is the yearly total return calculated from un-adjusted "Close" prices and yearly "dividend sum",
    which is expected to be equal to the total return that is calculated from "AdjClose" prices
    '''
    yearly_data['CalReturn'] = (yearly_data['Close'] + yearly_data['DivSum']) / yearly_data['Close'].shift(1) - 1
    # set the display format
    yearly_data[['DivRatio','Return','TotalReturn','CalReturn']] = yearly_data[['DivRatio','Return','TotalReturn','CalReturn']].mul(100).round(2)
    '''
    #yearly_data[['DivRatio','Return', 'TotalReturn', 'CalReturn']] = yearly_data[['DivRatio','Return', 'TotalReturn', 'CalReturn']].applymap("{:.2f}%".format)
    yearly_data[['DivRatio','Return', 'TotalReturn', 'CalReturn']]= \
     yearly_data[['DivRatio','Return', 'TotalReturn', 'CalReturn']].applymap(lambda x: f"{x:.2f}%" if not pd.isna(x) else "NaN")
    '''
    # Use .applymap() and lambda to format the values as percentage strings only if they are not NaN
    yearly_data[['DivRatio','Return', 'TotalReturn', 'CalReturn']]= \
         yearly_data[['DivRatio','Return', 'TotalReturn', 'CalReturn']].applymap(lambda x: f"{x:.2f}%" if not pd.isna(x) else x)

    # 'Date' column is no longer required
    yearly_data.drop('Date', axis=1, inplace=True)
    return yearly_data

#==============================================================================
# Part 7:
# print the year-end adjusted close prices of a list of stockes
def stock_prices_df(tickers_list, end_date_str):
    tickers_list_upper = [ticker.upper() for ticker in tickers_list]
    tickers_str = ", ".join(tickers_list_upper)
    # Download the historical data
    data = yf.download(tickers_str, period="max")
    data_adj_close = data['Adj Close']

    # Filter out rows with dates newer than calculation_end_date
    data_adj_close = data_adj_close[data_adj_close.index <= end_date_str]

    # Rearrange columns based on the order in tickers_list_upper
    if len(tickers_list)>1:
        data_adj_close = data_adj_close.reindex(columns=tickers_list_upper)

    data_adj_close = data_adj_close.resample('A').ffill().round(2)
    data_adj_close.index=data_adj_close.index.date
    last_date = data_adj_close.index[-1]
    data_adj_close = data_adj_close.rename(index={last_date: end_date_str})
    data_adj_close_df = pd.DataFrame(data_adj_close)
    data_adj_close_df.columns = map(str.lower, data_adj_close_df.columns)
    return data_adj_close_df

#==============================================================================
# Part 8: gradio handling - Input command handling and display in web page

help_info_str="Input Formats:\n  \
            1. ticker list....................Example:  spy vfv.to xiu.to xic.to xfn.to ry.to \n \
            2. One of default ticker list, a number between 1 and 7....Example:   0, or 1, ...,7 \n \
            3. CalculationEndDate as prefix.  Example:   2020-12-31 2 \n \
            .........................................2020-12-31 spy vfv.to xiu.to xic.to xfn.to ry.to \n \
            4. single ticker: Dividend/Close/AdjClose/Return/TotalReturn/CalReturn(by Close/Dividends).  @1 spy \n \
            note: daily adjusted close data are from Yahoo Finance. "

# Main Handling Process
def calculation_response(message):
    # if there is no input, display help information
    if message=="":
        return help_info_str

    tickers=message.split()

    # ******************************************************************************
    #  processing web input parameters
    #  set calculation_end_date_str, and tickers

    #---------------------------------------------------------
    # single stock ticker - detailed information
    if (tickers[0] == "@1"):
        tickers.pop(0) # remove the first string which is "@1"
        if len(tickers)==0:
            ticker = 'spy' # default ticker = spy
        else:
            ticker=tickers[0]
        output_string=f"\n {ticker}\n"
        output_dataframe0=get_yearly_single_stock_data(ticker)
        output_html=output_string + output_dataframe0.to_html()
        return output_html

    #----------------------------------------------------------
    # Get today's date, use .strftime("%Y-%m-%d") to convert to a string
    #calculation_end_date_str=datetime.now(pytz.timezone('America/New_York')).date().strftime("%Y-%m-%d")
    calculation_end_date_str = get_last_trading_day()
    # Check whether the first str is date for calculation end date
    if is_valid_date(tickers[0]):
        calculation_end_date_str = tickers[0] # reset calculation_end_date_str
        tickers.pop(0) # remove the first string which is the date

    #............ For display trailing and cumulative returns at month_boundary_date
    # Assuming calculation_end_date_str contains the date string '2024-01-03'
    calculation_end_date = datetime.strptime(calculation_end_date_str, '%Y-%m-%d')
    # Calculate the first day of the current month
    first_day_of_month = calculation_end_date.replace(day=1)
    # Calculate the last day of the month
    last_day_of_month = (calculation_end_date.replace(day=1) + timedelta(days=32)).replace(day=1) - timedelta(days=1)
    # Calculate the last day of the previous month
    last_day_of_previous_month = first_day_of_month - timedelta(days=1)
    # Check if the original date is the last day of the month
    if (calculation_end_date == last_day_of_month):
        calculation_end_date_month_boundary_date_str=calculation_end_date_str
    else:
        calculation_end_date_month_boundary_date_str=last_day_of_previous_month.strftime('%Y-%m-%d')
    # calculation_end_date_for_others are for trailing and cumulative returns
    calculation_end_date_for_others_str=calculation_end_date_month_boundary_date_str

    '''  Handling Feb 29 of leap years.
    For leap years, to simiplify the calculation,  Feb 28 will be used to replace Feb 29 for
    for calculating returns.
    Therefore, if calculation_end_date_for_others_str is Feb 29, then replace 29 to 28 of calculation_end_date_for_others_str
    '''
    leap_year=False
    if (
        calculation_end_date_for_others_str[-5:] == '02-29'
    ):
        calculation_end_date_for_others_str = calculation_end_date_for_others_str[:-2] + '28'
        leap_year=True
    #................End

    # Check whether numebr 0, 1, 2, .. is selected for using a default ticker list
    integer_value=str_to_integer(tickers[0])
    if (integer_value >= 0 and integer_value <len(tickers_lists)):
        tickers=tickers_lists[integer_value]

    # if no tickers were set, display help information
    if len(tickers)==0:
        return help_info_str

    #*********************************************************************************
    # Calculating year-end prices, Annual, Trailing, Cumulative, and CAGR returns & generating html for display
    #
    # list of year-end prices of stocks
    output_string0= f"\nAdj Close Prices ($)\n"
    output_dataframe= stock_prices_df(tickers, calculation_end_date_str)
    output_html0=output_string0 + output_dataframe.to_html()
    #  Annual Total Return
    output_string = f"\nAnnual Total Return (%) as {calculation_end_date_str}\n"
    output_dataframe = get_annual_returns_tickers_year_boundary_df(tickers, calculation_end_date_str)
    output_dataframe = output_dataframe.dropna(how='all')
    output_dataframe = output_dataframe.round(4)*100
    output_dataframe.index=output_dataframe.index.date
    # Assuming your DataFrame is named output_dataframe
    last_date = output_dataframe.index[-1]
    output_dataframe = output_dataframe.rename(index={last_date: calculation_end_date_str})
    # Convert the DataFrame to HTML, Combine the expected string outputs
    output_dataframe.columns = map(str.lower, output_dataframe.columns)
    output_html1 = output_string + output_dataframe.to_html()

    # annual_returns  - at any given day, for calculating trailing and cumulative returns, not to be displayed
    annual_returns_dataframe=get_annual_returns_tickers_df(tickers, calculation_end_date_for_others_str)

    # Trailing Return
    if (leap_year):
        output_string2 = f"\nTrailing Total Return (%) as {calculation_end_date_for_others_str} (leap year: Feb 29 replaced by Feb 28 for approximation)\n"
    else:
        output_string2 = f"\nTrailing Total Return (%) as {calculation_end_date_for_others_str}\n"
    output_dataframe2=get_trailing_return_all(tickers, annual_returns_dataframe)
    output_dataframe2 = output_dataframe2.dropna(how='all')
    # Insert an empty to align the ticker symbols with annual return display
    output_dataframe2.insert(0, "--------", "      ")
    output_dataframe2.index.name="years"
    output_html2=output_string2 + output_dataframe2.to_html()

    # Cumulative Return
    output_string3 = f"\nCumulative Return (%) as {calculation_end_date_for_others_str}\n"
    cumulative_return_all_dataframe=get_cumulative_return_all(tickers, annual_returns_dataframe)
    cumulative_return_all_dataframe = cumulative_return_all_dataframe.dropna(how='all')
    output_dataframe3=cumulative_return_all_dataframe.round(4)*100
    output_dataframe3.index.name="years"
    output_html3=output_string3 + output_dataframe3.to_html()

    # CAGR Return
    output_string4 = f"\nCompound Annual Growth Rate (CAGR) (%) as {calculation_end_date_for_others_str}\n"
    output_dataframe4=get_cagr_return_all (cumulative_return_all_dataframe)
    output_dataframe4=output_dataframe4.round(4)*100
    output_html4=output_string4 + output_dataframe4.to_html()

    #output_html = output_html1 + output_html2 + output_html3 + output_html4
    output_html = output_html0 + output_html1 + output_html2 + output_html3
    return  output_html

# Gradio Web interface
'''
def ChatInterface_response(message, history):
    output_html=calculation_response(message)
    return output_html
web_handling = gr.ChatInterface(ChatInterface_response)
web_handling.launch(debug=False, share=False)
'''
with gr.Blocks() as web_block:
    '''
    chatbot = gr.Chatbot()
    msg = gr.Textbox()
    clear = gr.ClearButton([msg, chatbot])
    '''
    chatbot = gr.Chatbot(height="500px")
    # Create a row element for the Textbox and Clear button
    with gr.Row():
        #msg = gr.Textbox(label="stock tickers input", scale=2, min_width=380)
        msg = gr.Textbox(show_label=False, scale=2, min_width=380)
        clear = gr.ClearButton([msg, chatbot], scale=0, min_width=50)

    def respond(message, chat_history):
        bot_message = calculation_response(message)
        chat_history.append((message, bot_message))
        return "", chat_history

    msg.submit(respond, # function
     [msg, chatbot],  # inputs of the function
     [msg, chatbot]   # outputs of the function
               )
web_block.launch()
#web_block.launch(debug=True)



Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://2c3cd8100964a19576.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


##  debug section

In [ ]:
# debug section
#calculation_response("0", [])
'''
tickers=["spy","vfv.to"]
calculation_end_date_str="2024-01-08"
calculation_end_date_month_boundary_date_str="2023-12-31"
output_dataframe = get_annual_returns_tickers_year_boundary_df(tickers, calculation_end_date_str)
annual_returns_dataframe=get_annual_returns_tickers_df(tickers, calculation_end_date_month_boundary_date_str)
#print(output_dataframe)
print(annual_returns_dataframe)


calculation_end_date_str= "2012-01-02"
print("get_last_trading_day", calculation_end_date_str)
tickers = ["spy", "vfv.to", "vgg.to", "goog", "msft", "meta", "tsla","AMZN", "AAPL", "shop.to"]
output_string = f"\nAnnual Total Return (%) as {calculation_end_date}\n"
annual_returns_dataframe=get_annual_returns_tickers_df(tickers, calculation_end_date_str)
output_dataframe = annual_returns_dataframe.round(4)*100
print(output_dataframe)

help_info_str = """Input Formats:
| {0:30} | {1:30} |
| {2:30} | {3:30} |
| {4:30} | {5:30} |""".format(
    "1. ticker list.", "Example: spy vfv.to xiu.to xic.to xfn.to ry.to",
    "2. One of default ticker list.", "Example: 0|1|...|5",
    "3. CalculationEndDate as prefix.", "Example: 2020-12-31 2 or 2020-12-31 spy vfv.to xiu.to xic.to xfn.to ry.to"
)

'''

## Backup Example 8

In [ ]:
'''
Example 8 for using yfinance

Calculate annual, trailing, cumumlative, and CAGR returns for multiple stocks.
* The start date can be an arbitrary date.  The default is the current date.
* annual return is displayed from the default current day, or an arbitrary given
  day (except for Feb 29 for leap year)
  For leap years, use Feb 28 to replace Feb 29 as simplification & approximation
* trailing, cumumlative returns are currently displayed from the month boundary (last day of Month)
  prior to the given date.
* However, trailing, cumumlative returns can be displayed
  from any date, which can be not at the month boundary (last day of Month),
  by minor change of setting calculation_end_date_for_others_str = calculation_end_date_str.
  prior to the given date in the function "calculation_response(message, history)"

Author: Gang Luo
'''
script_version = '(2024-03-01.1)'
import gradio as gr
import yfinance as yf
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import pytz
#==============================================================================

print_yearly_total_return = True
num_years_calculation=52   # total years for calculation

# Define a list of years to calculate the trailing returns, cumulative returns, and so on
# remove the row of current year row since it is not a full year.
years_list = [1, 2, 3, 5, 10, 15, 20, 25, 30, 40, 50]

# Set the stock tickers list
tickers_lists = [["qqq","hxq.to","spy", "vfv.to","xiu.to", "xbb.to","xcb.to","xhb.to"], #0  checking ETF
    ["qqq","spy", "vfv.to", "vgg.to", "zlu.to", "xiu.to", "vdy.to", "xfn.to", "ry.to", "td.to", "na.to",
      "slf.to", "gwo.to", "bce.to", "t.to", "rci-b.to", "enb.to", "trp.to", "zlb.to", "cp.to"], #1 main monitoring list
    ["qqq","spy","vfv.to", "xiu.to", "zeb.to", "xfn.to", "na.to","ry.to", "bmo.to","bns.to", "td.to", "cm.to", "cwb.to",
      "slf.to", "gwo.to", "bce.to", "t.to", "rci-b.to", "enb.to", "trp.to", "xdv.to","cdz.to","vdy.to", "xdiv.to"],  #2   financial  ETF & stocks
    ["^ndx","qqq","^GSPC","spy", "vfv.to", "vgg.to","zlu.to","msft","AAPL","goog","AMZN","NVDA","meta","tsla","BRK-A","shop.to","hxq.to"],   #3  US mega stocks + risky shopfy
    ["^DJI","dia","^GSPC","spy","voo","ivv", "tpu-u.to","vfv.to", "zsp.to","hxs.to","tpu.to","xus.to", "xsp.to",
      "^IXIC","^ndx", "qqq","hxq.to","^GSPTSE","xic.to","xiu.to", "HXT.TO", "TTP.TO","ZCN.TO", "xfn.to", "xit.to"], #4   indexes and index ETFs
    ["dia","^DJI","^GSPC","spy","vfv.to", "zsp.to","hxs.to","xus.to", "xsp.to",
      "^IXIC","qqq","hxq.to","^GSPTSE","xic.to","xiu.to", "HXT.TO", "xfn.to"], #5   indexes and typical index ETFs
    ["^IXIC","^ndx","ONEQ","CIBR","QQJG", "qqq", "tqqq", "spy", "vfv.to", "HXQ.to", "ZQQ.to", "XQQ.to", "QQC.to", "ZNQ.TO",
         "xiu.to", "xit.to"],  #6   Nasdaq ETF and TSX IT ETF
    ["qqq","tqqq","sqqq", "QLD", "spy", "spxu", "upro", "sso", "spxl","tecl"], #7 leveraged ETFs
    ["^IXIC","^DJI","^GSPC","^GSPTSE"] #8 testing
]

#==============================================================================
# Part 1: fetch retrieve yearly total returns by yfinance & display
# Function to fetch data from yfinance and extract yearly total returns#
# annual return calculation can start at any given day
def get_annual_returns_df(ticker, calculation_end_date_str):
    # Get the historical data for the given ticker
    stock = yf.Ticker(ticker)
    calculation_end_date=pd.to_datetime(calculation_end_date_str).tz_localize('America/New_York')
    try:
        '''
        'try' statement for handlingy the exception error of stock.history that a ticker is not yet at stock market,
        For example, "shop.to" is not there in 2012
        '''
        stock_history=stock.history(period="max")["Close"]
        '''
        Between the start and end days in stock_history variable, there are some missing days where there are no corresponding rows.
        Add rows of  missing  days such that the values of column "Close" are set to be the value of the closest earlier day's
        value, by using date_range to create full range without any missing date.
        '''
        # Create a DataFrame with a complete date range
        date_range = pd.date_range(start=stock_history.index.min(), end=stock_history.index.max(), freq='D')
        complete_stock_history = pd.DataFrame(index=date_range)
        # Merge the complete DataFrame with the original stock_history
        complete_stock_history = complete_stock_history.merge(stock_history, how='left', left_index=True, right_index=True)
        complete_stock_history['Close'] = complete_stock_history['Close'].ffill()  # fill the newy added rows with previous day value
        '''
        Filter out the rows that matches the month and date of calculation_end_date, which are the ends of
        annual periods from the calculation_end_date.
        '''
        # Filter out rows with dates newer than calculation_end_date
        filtered_stock_history = complete_stock_history[complete_stock_history.index <= calculation_end_date]
        #print(filtered_stock_history)
        target_month=filtered_stock_history.index.max().month
        target_day=filtered_stock_history.index.max().day
        #print("target_month", target_month, "target_day",target_day, "start_year", filtered_stock_history.index.max().year)
        annual_returns = filtered_stock_history[(filtered_stock_history.index.month == target_month)
           & (filtered_stock_history.index.day ==target_day)]
        annual_returns_percent = annual_returns.pct_change().dropna()
    except:
        return  pd.DataFrame()
    else:
        annual_returns_df = pd.DataFrame(annual_returns_percent, columns=['Close'])
        annual_returns_df.rename(columns={'Close': ticker}, inplace=True)
        return annual_returns_df

# Function to fetch data from yfinance and extract yearly total returns
# annual return calculation starts at only yaer end boundary, i.e, Dec 31,
# by resample('A')
def get_annual_returns_year_boundary_df(ticker, calculation_end_date_str):
    # Get the historical data for the given ticker
    stock = yf.Ticker(ticker)
    calculation_end_date = datetime.strptime(calculation_end_date_str, "%Y-%m-%d")
    calculation_start_date_str = (calculation_end_date
                - timedelta(days=num_years_calculation * 365)).strftime("%Y-%m-%d")

    try:
        '''
        1.  'try' statement for handlingy the exception error of stock.history that a ticker is not yet at stock market,
             For example, "shop.to" is not there in 2012
        2. The row with the latest day from .history(.., end='end_day_date') is the day prior to end_day_date.  Therefore,
           let end=the expected end day plus one day.
        '''
        calculation_end_date_plus_1day_str =  (calculation_end_date + timedelta(days=1)).strftime("%Y-%m-%d")
        annual_returns_history=stock.history(start=calculation_start_date_str,end=calculation_end_date_plus_1day_str)["Close"]

        #print("debug get_annual_returns_df ", ticker, annual_returns_history)
        # For 'A', 'Y', see https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html#offset-aliases
        ffilled_history=annual_returns = annual_returns_history.resample('A').ffill()
        #print(ffilled_history)
        annual_returns = ffilled_history.pct_change().dropna()
        #annual_returns = annual_returns_history.resample('A').ffill().pct_change().dropna()
        #print("debug get_annual_returns_df after resample()", ticker, calculation_end_date, "\n", annual_returns)
    except:
        return  pd.DataFrame()
    else:
        annual_returns_df = pd.DataFrame(annual_returns, columns=['Close'])
        annual_returns_df.rename(columns={'Close': ticker}, inplace=True)
        return annual_returns_df

#----------------------------------------------------------------------------------
# handling a list of tickers by calling the functions (either get_annual_returns_df
# get_annual_returns_year_boundary_df) that handle single tickers
def get_annual_returns_tickers_common_df(tickers, calculation_end_date_str, annual_returns_func_df):
    # Create an empty DataFrame to store all tickers' total returns
    all_tickers_returns_df = pd.DataFrame()

    # Loop through each ticker in the list
    for ticker in tickers:
        ticker_returns_df = annual_returns_func_df(ticker, calculation_end_date_str)
        if not ticker_returns_df.empty:
            if all_tickers_returns_df.empty:
                all_tickers_returns_df = ticker_returns_df
            else:
                '''
                When running in  huggingface, pd.concat changed the index order of ticker_returns_df
                when ticker_returns_df has more rows than all_tickers_returns_df. However, it is ok
                running in colab.  Therefore, use pd.merge to replace pd.concat.
                all_tickers_returns_df = pd.concat([all_tickers_returns_df, ticker_returns_df],axis=1,join='outer')  # Concatenate DataFrames
                all_tickers_returns_df.sort_index() # index may be changed when running in huggingface
                '''
                all_tickers_returns_df = pd.merge(all_tickers_returns_df, ticker_returns_df,
                        left_index=True, right_index=True, how='outer')
        else:
            # New column with NaN values
            new_column_name = ticker
            new_column_values = [None] * len(all_tickers_returns_df)
            new_column = pd.DataFrame({new_column_name: new_column_values}, index=all_tickers_returns_df.index)
            # Concatenate the new column to the original DataFrame
            all_tickers_returns_df = pd.concat([all_tickers_returns_df, new_column], axis=1)
    #return date_label_conversion_strip_time(all_tickers_returns_df, calculation_end_date_str)
    return all_tickers_returns_df

def get_annual_returns_tickers_df(tickers, calculation_end_date_str):
    return get_annual_returns_tickers_common_df(tickers, calculation_end_date_str,
                                                get_annual_returns_df)

def get_annual_returns_tickers_year_boundary_df(tickers, calculation_end_date_str):
    return get_annual_returns_tickers_common_df(tickers, calculation_end_date_str,
                                                get_annual_returns_year_boundary_df)

#==============================================================================
# Part 2: calculate the annualized trailing total return from the data generated in step 1 & display
# Define a function to calculate the annualized trailing total return for a given number of years
def get_trailing_return(ticker, data, years):
    # Get the total return values for the last n years
    trailing_data = data[ticker].tail(years)
    # Check if there are empty values within years
    if trailing_data.isna().any():
        return np.nan
    # Check if there are valid total return values for all years
    if len(trailing_data) == years:
        # Convert the percentage strings to numeric values
        trailing_data = trailing_data.astype(str).str.replace('%', '').astype(float)
        """ Calculate the annualized trailing total return using the formula from Investopedia[^1^][1]:
            Annualized Return = [(1 + r1) * (1 + r2) * ... * (1 + rn)]^(1/n) - 1
            Where r1, r2, ..., rn are the total return values for each year                    """
        annualized_trailing_return = (trailing_data + 1).prod() ** (1 / years) - 1

        # Format the result as a percentage with two decimal places
        annualized_trailing_return = annualized_trailing_return * 100
        annualized_trailing_return = annualized_trailing_return.round(2)
        return annualized_trailing_return
    else:
        return np.nan

# Define a function to Loop through the list and print the trailing returns for each num_years
def get_trailing_return_column(ticker, annual_returns_df):
    trailing_return_column = {}
    for num_years in years_list:
        # Check if the ticker data is available in all_tickers_returns_df
        if ticker in annual_returns_df.columns:
            # using data from step 1, avoiding get_annual_returns_df(ticker) for less traffic from yahoo server
            data = annual_returns_df[[ticker]]
            trailing_return = get_trailing_return(ticker, data, num_years)
            trailing_return_column[f"{num_years}-Year"] = trailing_return
        else:
            print(f"Data not available for {ticker}. Skipping.")
            trailing_return_column[f"{num_years}-Year"] = np.nan
    return trailing_return_column

# Create an empty DataFrame to store all tickers' trailing returns
def get_trailing_return_all(tickers, annual_returns_df):
    all_tickers_trailing_returns_df = pd.DataFrame(index=years_list)

    # Loop through each ticker in the list
    for ticker in tickers:
        trailing_returns = get_trailing_return_column(ticker, annual_returns_df)
        # Add the trailing returns to the DataFrame
        all_tickers_trailing_returns_df[ticker] = pd.Series(trailing_returns).values
    all_tickers_trailing_returns_df.columns = map(str.lower, all_tickers_trailing_returns_df.columns)
    return all_tickers_trailing_returns_df

#==============================================================================
# Part 3: calculate the cumulative return from the data (all_tickers_returns_df) generated in part 1 & display
#  Define a function to calculate the cumulative return for a given number of years from a ticker
def get_cumulative_return(ticker, data, years):
    # Calculate the cumulative return
    cumulative_return = (1 + data[ticker]).rolling(window=years).apply(lambda x: x.prod(), raw=True) - 1
    return cumulative_return

# Define a function to Loop through the list and return the cumulative returns for each num_years
def get_cumulative_return_column(ticker, annual_returns_df):
    cumulative_returns = {}
    for years in years_list:
        # Calculate the cumulative return for the given number of years
        cumulative_return = get_cumulative_return(ticker, annual_returns_df, years)
        # Get the last value, which is the cumulative return up to the current year
        cumulative_returns[years] = cumulative_return.iloc[-1]
    return cumulative_returns

def get_cumulative_return_all(tickers, annual_returns_df):
    # Create an empty DataFrame with years_list as the index for cumulative  returns
    all_tickers_cumulative_returns_df = pd.DataFrame(index=years_list)
    # Loop through each ticker in the list
    for ticker in tickers:
        cumulative_returns = get_cumulative_return_column(ticker, annual_returns_df)
        # Add the trailing returns to the DataFrame
        all_tickers_cumulative_returns_df[ticker] = pd.Series(cumulative_returns).values
    all_tickers_cumulative_returns_df.columns = map(str.lower, all_tickers_cumulative_returns_df.columns)
    return all_tickers_cumulative_returns_df

#==============================================================================
# Part 4: calculate the  CAGR (Compound Annual Growth Rate) from the data
# in all_tickers_cumulative_returns_df generated earlier & display
# Define a function to calculate the CAGR from the cumulative value and the years
def calculate_cagr(value, years):
    # Otherwise, calculate the CAGR using the formula
    cagr = (value + 1) ** (1 / np.array(years)) - 1
    #print("debug-cagr\n", cagr, "end")
    return cagr

# Define a function to format the Float64Index values into percentage strings
def format_to_percentage(value):
    # If any element in the value array is not null, format it as a percentage string with two decimal places
    if np.any(pd.notnull(value)):
        return f"{value:.2f}%"
    # Otherwise, return None
    return None

def get_cagr_return_all(all_tickers_cumulative_returns_df):
    # Apply the calculate_cagr function to each column of the DataFrame
    all_tickers_cagrs_df = all_tickers_cumulative_returns_df.apply(lambda x: calculate_cagr(x, x.index), axis=0)
    return all_tickers_cagrs_df

#==============================================================================
# Part 5: utility functions
# get the last trading day of S&P 500 in string format
def get_last_trading_day():
    # Get today's date, use .strftime("%Y-%m-%d") to convert to a string
    today_date_str=datetime.now(pytz.timezone('America/New_York')).date().strftime("%Y-%m-%d")
    stock = yf.Ticker("^GSPC") # S&P 500 (^GSPC) ticker
    #  search and see yfinance_BUG_1 NOTE in this file
    history_df=stock.history(period="max", end=today_date_str)["Close"]
    last_trading_day_str = history_df.index.max().date().strftime("%Y-%m-%d")
    return last_trading_day_str

def str_to_integer(integer_str):
    try:
        integer_number = int(integer_str)
        return integer_number
    except ValueError:
        return -1

# validate the date string
def is_valid_date(date_string):
    try:
        # Attempt to parse the date string
        datetime.strptime(date_string, "%Y-%m-%d")
        return True
    except ValueError:
        # Raised when the date string is not in the expected format
        return False

def date_label_conversion_strip_time(all_tickers_returns_df, calculation_end_date_str):
    all_tickers_returns_df.index=all_tickers_returns_df.index.date
    all_tickers_returns_df.index.name='date'
    # print("debug get_annual_returns_tickers_df", all_tickers_returns_df)
    # Convert calculation_end_date_str to a datetime object, replace the index's mon/day portion of date
    end_date_datetime_obj = datetime.strptime(calculation_end_date_str, "%Y-%m-%d")
    all_tickers_returns_df.index = all_tickers_returns_df.index.map(
      lambda x: x.replace(month=end_date_datetime_obj.month,
      day=end_date_datetime_obj.day))
    return all_tickers_returns_df

#==============================================================================
# Part 6:
# single ticker's Prices, Returns,Dividends, good for verifying whether "Adj Close" is correct.
'''
    Calculate and display: yearly dividendSum, 'Close' & 'Adj Close' prices,
    Return(by 'Close' price), total return(by 'Adj Close' price),
    CalReturn(total return by 'Close' price and "dividendSum).
    Note: CalReturn from  is expected to be nearly same as  total return,
           when the 'Adj Close' price is correct.
'''
def get_yearly_single_stock_data(ticker):
    stock = yf.Ticker(ticker)
    #-------- mainly for downloading 'Dividends'
    history = stock.history(period="max")
    dividend_history=history['Dividends']
    dividend_history.index=dividend_history.index.date

    #-------- mainly for downloading 'Close','Adj Close'
    dld_history=yf.download(ticker, period="max")
    dld_history=dld_history[['Close','Adj Close']]
    dld_history.rename(columns={'Adj Close': 'AdjClose'}, inplace=True)
    date_range = pd.date_range(start=dld_history.index.min(), end=dld_history.index.max(), freq='D')
    complete_history = pd.DataFrame(index=date_range)

    # Merge the complete DataFrame with the original stock_history
    complete_history = complete_history.merge(dld_history, how='left', left_index=True, right_index=True)
    complete_history[['Close','AdjClose']] = complete_history[['Close','AdjClose']].ffill().round(3)

    # Merge dividend into complete_history
    complete_history = complete_history.merge(dividend_history, how='left', left_index=True, right_index=True)
    # replace all NaN values in the 'Dividends' column with 0.0
    complete_history['Dividends'] = complete_history['Dividends'].fillna(0.0).round(3)

    complete_history['Year']=complete_history.index.year
    complete_history['Date']=complete_history.index
    yearly_data = complete_history.groupby('Year').agg({'Date': 'last', 'Close': 'last', 'AdjClose': 'last','Dividends': 'sum'})
    yearly_data.rename(columns={'Dividends': 'DivSum'}, inplace=True)

    # calculating 'Return' and 'TotalReturn'
    yearly_data['DivRatio']=yearly_data['DivSum'] / yearly_data['Close']
    yearly_data['Return']=yearly_data['Close'].pct_change()
    yearly_data['TotalReturn']=yearly_data['AdjClose'].pct_change()

    '''
    The CalReturn column is the yearly total return calculated from un-adjusted "Close" prices and yearly "dividend sum",
    which is expected to be equal to the total return that is calculated from "AdjClose" prices
    '''
    yearly_data['CalReturn'] = (yearly_data['Close'] + yearly_data['DivSum']) / yearly_data['Close'].shift(1) - 1
    # set the display format
    yearly_data[['DivRatio','Return','TotalReturn','CalReturn']] = yearly_data[['DivRatio','Return','TotalReturn','CalReturn']].mul(100).round(2)
    '''
    #yearly_data[['DivRatio','Return', 'TotalReturn', 'CalReturn']] = yearly_data[['DivRatio','Return', 'TotalReturn', 'CalReturn']].applymap("{:.2f}%".format)
    yearly_data[['DivRatio','Return', 'TotalReturn', 'CalReturn']]= \
     yearly_data[['DivRatio','Return', 'TotalReturn', 'CalReturn']].applymap(lambda x: f"{x:.2f}%" if not pd.isna(x) else "NaN")
    '''
    # Use .applymap() and lambda to format the values as percentage strings only if they are not NaN
    yearly_data[['DivRatio','Return', 'TotalReturn', 'CalReturn']]= \
         yearly_data[['DivRatio','Return', 'TotalReturn', 'CalReturn']].applymap(lambda x: f"{x:.2f}%" if not pd.isna(x) else x)

    # 'Date' column is no longer required
    yearly_data.drop('Date', axis=1, inplace=True)
    return yearly_data

#==============================================================================
# Part 7:
# print the year-end adjusted close prices of a list of stockes
def stock_prices_df(tickers_list, end_date_str):
    tickers_list_upper = [ticker.upper() for ticker in tickers_list]
    tickers_str = ", ".join(tickers_list_upper)
    # Download the historical data
    data = yf.download(tickers_str, period="max")
    data_adj_close = data['Adj Close']

    # Filter out rows with dates newer than calculation_end_date
    data_adj_close = data_adj_close[data_adj_close.index <= end_date_str]

    # Rearrange columns based on the order in tickers_list_upper
    if len(tickers_list)>1:
        data_adj_close = data_adj_close.reindex(columns=tickers_list_upper)

    data_adj_close = data_adj_close.resample('A').ffill().round(2)
    data_adj_close.index=data_adj_close.index.date
    last_date = data_adj_close.index[-1]
    data_adj_close = data_adj_close.rename(index={last_date: end_date_str})
    data_adj_close_df = pd.DataFrame(data_adj_close)
    data_adj_close_df.columns = map(str.lower, data_adj_close_df.columns)
    return data_adj_close_df

#==============================================================================
# Part 8: gradio handling - Input command handling and display in web page

help_info_str="Input Formats:\n  \
            1. ticker list....................Example:  spy vfv.to xiu.to xic.to xfn.to ry.to \n \
            2. One of default ticker list, a number between 1 and 7....Example:   0, or 1, ...,7 \n \
            3. CalculationEndDate as prefix.  Example:   2020-12-31 2 \n \
            .........................................2020-12-31 spy vfv.to xiu.to xic.to xfn.to ry.to \n \
            4. single ticker: Dividend/Close/AdjClose/Return/TotalReturn/CalReturn(by Close/Dividends).  @1 spy \n \
            note: daily adjusted close data are from Yahoo Finance. "

# Main Handling Process
def calculation_response(message):
    # if there is no input, display help information
    if message=="":
        return help_info_str

    tickers=message.split()

    # ******************************************************************************
    #  processing web input parameters
    #  set calculation_end_date_str, and tickers

    #---------------------------------------------------------
    # single stock ticker - detailed information
    if (tickers[0] == "@1"):
        tickers.pop(0) # remove the first string which is "@1"
        if len(tickers)==0:
            ticker = 'spy' # default ticker = spy
        else:
            ticker=tickers[0]
        output_string=f"\n {ticker}\n"
        output_dataframe0=get_yearly_single_stock_data(ticker)
        output_html=output_string + output_dataframe0.to_html()
        return output_html

    #----------------------------------------------------------
    # Get today's date, use .strftime("%Y-%m-%d") to convert to a string
    #calculation_end_date_str=datetime.now(pytz.timezone('America/New_York')).date().strftime("%Y-%m-%d")
    calculation_end_date_str = get_last_trading_day()
    # Check whether the first str is date for calculation end date
    if is_valid_date(tickers[0]):
        calculation_end_date_str = tickers[0] # reset calculation_end_date_str
        tickers.pop(0) # remove the first string which is the date

    #............ For display trailing and cumulative returns at month_boundary_date
    # Assuming calculation_end_date_str contains the date string '2024-01-03'
    calculation_end_date = datetime.strptime(calculation_end_date_str, '%Y-%m-%d')
    # Calculate the first day of the current month
    first_day_of_month = calculation_end_date.replace(day=1)
    # Calculate the last day of the month
    last_day_of_month = (calculation_end_date.replace(day=1) + timedelta(days=32)).replace(day=1) - timedelta(days=1)
    # Calculate the last day of the previous month
    last_day_of_previous_month = first_day_of_month - timedelta(days=1)
    # Check if the original date is the last day of the month
    if (calculation_end_date == last_day_of_month):
        calculation_end_date_month_boundary_date_str=calculation_end_date_str
    else:
        calculation_end_date_month_boundary_date_str=last_day_of_previous_month.strftime('%Y-%m-%d')
    # calculation_end_date_for_others are for trailing and cumulative returns
    calculation_end_date_for_others_str=calculation_end_date_month_boundary_date_str

    '''  Handling Feb 29 of leap years.
    For leap years, to simiplify the calculation,  Feb 28 will be used to replace Feb 29 for
    for calculating returns.
    Therefore, if calculation_end_date_for_others_str is Feb 29, then replace 29 to 28 of calculation_end_date_for_others_str
    '''
    leap_year=False
    if (
        calculation_end_date_for_others_str[-5:] == '02-29'
    ):
        calculation_end_date_for_others_str = calculation_end_date_for_others_str[:-2] + '28'
        leap_year=True
    #................End

    # Check whether numebr 0, 1, 2, .. is selected for using a default ticker list
    integer_value=str_to_integer(tickers[0])
    if (integer_value >= 0 and integer_value <len(tickers_lists)):
        tickers=tickers_lists[integer_value]

    # if no tickers were set, display help information
    if len(tickers)==0:
        return help_info_str

    #*********************************************************************************
    # Calculating year-end prices, Annual, Trailing, Cumulative, and CAGR returns & generating html for display
    #
    # list of year-end prices of stocks
    output_string0= f"\nAdj Close Prices ($)\n"
    output_dataframe= stock_prices_df(tickers, calculation_end_date_str)
    output_html0=output_string0 + output_dataframe.to_html()
    #  Annual Total Return
    output_string = f"\nAnnual Total Return (%) as {calculation_end_date_str}\n"
    output_dataframe = get_annual_returns_tickers_year_boundary_df(tickers, calculation_end_date_str)
    output_dataframe = output_dataframe.dropna(how='all')
    output_dataframe = output_dataframe.round(4)*100
    output_dataframe.index=output_dataframe.index.date
    # Assuming your DataFrame is named output_dataframe
    last_date = output_dataframe.index[-1]
    output_dataframe = output_dataframe.rename(index={last_date: calculation_end_date_str})
    # Convert the DataFrame to HTML, Combine the expected string outputs
    output_dataframe.columns = map(str.lower, output_dataframe.columns)
    output_html1 = output_string + output_dataframe.to_html()

    # annual_returns  - at any given day, for calculating trailing and cumulative returns, not to be displayed
    annual_returns_dataframe=get_annual_returns_tickers_df(tickers, calculation_end_date_for_others_str)

    # Trailing Return
    if (leap_year):
        output_string2 = f"\nTrailing Total Return (%) as {calculation_end_date_for_others_str} (leap year: Feb 29 replaced by Feb 28 for approximation)\n"
    else:
        output_string2 = f"\nTrailing Total Return (%) as {calculation_end_date_for_others_str}\n"
    output_dataframe2=get_trailing_return_all(tickers, annual_returns_dataframe)
    output_dataframe2 = output_dataframe2.dropna(how='all')
    # Insert an empty to align the ticker symbols with annual return display
    output_dataframe2.insert(0, "--------", "      ")
    output_dataframe2.index.name="years"
    output_html2=output_string2 + output_dataframe2.to_html()

    # Cumulative Return
    output_string3 = f"\nCumulative Return (%) as {calculation_end_date_for_others_str}\n"
    cumulative_return_all_dataframe=get_cumulative_return_all(tickers, annual_returns_dataframe)
    cumulative_return_all_dataframe = cumulative_return_all_dataframe.dropna(how='all')
    output_dataframe3=cumulative_return_all_dataframe.round(4)*100
    output_dataframe3.index.name="years"
    output_html3=output_string3 + output_dataframe3.to_html()

    # CAGR Return
    output_string4 = f"\nCompound Annual Growth Rate (CAGR) (%) as {calculation_end_date_for_others_str}\n"
    output_dataframe4=get_cagr_return_all (cumulative_return_all_dataframe)
    output_dataframe4=output_dataframe4.round(4)*100
    output_html4=output_string4 + output_dataframe4.to_html()

    #output_html = output_html1 + output_html2 + output_html3 + output_html4
    output_html = output_html0 + output_html1 + output_html2 + output_html3
    return  output_html

# Gradio Web interface
'''
def ChatInterface_response(message, history):
    output_html=calculation_response(message)
    return output_html
web_handling = gr.ChatInterface(ChatInterface_response)
web_handling.launch(debug=False, share=False)
'''
with gr.Blocks() as web_block:
    '''
    chatbot = gr.Chatbot()
    msg = gr.Textbox()
    clear = gr.ClearButton([msg, chatbot])
    '''
    chatbot = gr.Chatbot(height="500px")
    # Create a row element for the Textbox and Clear button
    with gr.Row():
        #msg = gr.Textbox(label="stock tickers input", scale=2, min_width=380)
        msg = gr.Textbox(show_label=False, scale=2, min_width=380)
        clear = gr.ClearButton([msg, chatbot], scale=0, min_width=50)

    def respond(message, chat_history):
        bot_message = calculation_response(message)
        chat_history.append((message, bot_message))
        return "", chat_history

    msg.submit(respond, # function
     [msg, chatbot],  # inputs of the function
     [msg, chatbot]   # outputs of the function
               )
web_block.launch()
#web_block.launch(debug=True)



Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://227ecebb7068aebc31.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


# Example 7 = gradio + Example 6- calender year boundary  (multiple stocks) annual,  trailing, cumumlative, and CAGR  returns


In [ ]:
'''
!pip install gradio
!pip install yfinance
!pip install datetime
!pip install pytz
!pip install pandas
!pip install numpy
'''
script_version = '(2024-01-06.1)'
import gradio as gr
import yfinance as yf
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import pytz
#--------------------------------------------------------------------

print_yearly_total_return = True
num_years_calculation=32   # total years for calculation

# Define a list of years to calculate the trailing returns, cumulative returns, and so on
# remove the row of current year row since it is not a full year.
years_list = [1, 2, 3, 5, 9, 10, 15, 20, 25, 30]

# Set the stock tickers list
tickers_lists = [["spy", "vfv.to","xiu.to"], #0
    ["spy", "vfv.to", "vgg.to", "zlu.to", "xiu.to", "vdy.to", "xfn.to", "ry.to", "td.to", "na.to",
      "slf.to", "gwo.to", "bce.to", "t.to", "rci-b.to", "enb.to", "trp.to", "zlb.to", "cp.to"], #1
    ["spy","vfv.to", "xiu.to", "fie.to", "xfn.to", "na.to","ry.to", "bmo.to", "cm.to", "bns.to", "cwb.to",
      "slf.to", "gwo.to", "bce.to", "t.to", "rci-b.to", "enb.to", "trp.to", "xdv.to","cdz.to","vdy.to"],  #2
    ["spy", "vfv.to", "vgg.to", "zlu.to","goog", "msft", "meta", "tsla","AMZN", "AAPL", "shop.to"],   #3
    ["^IXIC","^GSPC","spy", "vfv.to", "zsp.to", "xsp.to","^GSPTSE","xic.to","xiu.to","xfn.to","fie.to", "TX60.TS"], #4
    ["xic.to","xiu.to","xfn.to"]  #5
]

#-------------------------------------------------------------------
# step 1: fetch retrieve yearly total returns by yfinance & display
# Function to fetch data from yfinance and extract yearly total returns
def get_annual_returns_df(ticker, calculation_end_date):
    # Get the historical data for the given ticker
    stock = yf.Ticker(ticker)
    calculation_start_date = (datetime.strptime(calculation_end_date, "%Y-%m-%d")
                - timedelta(days=num_years_calculation * 365)).strftime("%Y-%m-%d")

    # Get today's date, use .strftime("%Y-%m-%d") to convert to a string
    today_date=datetime.now(pytz.timezone('America/New_York')).date().strftime("%Y-%m-%d")

    ''' yfinance_BUG_1 NOTE:
    There is bug for  when using end="2024-01-02" on 2024-01-02 which is the first trading of the year on 8:10pm
    for using annual_returns_history=stock.history(start=calculation_start_date,end=calculation_end_date)["Close"]
     (tried tickers: "^GSPC","spy" for stock = yf.Ticker(ticker)) that 2024-01-02 data is not returned.
     however, when end parameter, it is fine for returning  2024-01-02 data as below
     annual_returns_history=stock.history(start=calculation_start_date)["Close"]
    '''
    try:
        #'try' statement for handlingy the exception error of stock.history that a ticker is not yet at stock market, "shop.to" is not there in 2012
        # the following if statement for handling yfinance_BUG_1
        if (calculation_end_date != today_date):
            annual_returns_history=stock.history(start=calculation_start_date,end=calculation_end_date)["Close"]
        else:
            annual_returns_history=stock.history(start=calculation_start_date)["Close"]

        #print("debug get_annual_returns_df ", ticker, annual_returns_history)
        # Tried resample('A', origin=calculation_end_date),  "origin="" looks not working.
        # For 'A', 'Y', see https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html#offset-aliases
        annual_returns = annual_returns_history.resample('A').ffill().pct_change().dropna()
        #print("debug get_annual_returns_df after resample()", ticker, calculation_end_date, "\n", annual_returns)
    except:
        return  pd.DataFrame()
    else:
        annual_returns_df = pd.DataFrame(annual_returns, columns=['Close'])
        annual_returns_df.rename(columns={'Close': ticker}, inplace=True)
        return annual_returns_df

def get_annual_returns_tickers_df(tickers, calculation_end_date):
    # Create an empty DataFrame to store all tickers' total returns
    all_tickers_returns_df = pd.DataFrame()

    # Loop through each ticker in the list
    for ticker in tickers:
        ticker_returns_df = get_annual_returns_df(ticker, calculation_end_date)
        if not ticker_returns_df.empty:
            if all_tickers_returns_df.empty:
                all_tickers_returns_df = ticker_returns_df
            else:
                all_tickers_returns_df = pd.concat([all_tickers_returns_df, ticker_returns_df], axis=1)  # Concatenate DataFrames
        else:
            # New column with NaN values
            new_column_name = ticker
            new_column_values = [None] * len(all_tickers_returns_df)
            new_column = pd.DataFrame({new_column_name: new_column_values}, index=all_tickers_returns_df.index)
            # Concatenate the new column to the original DataFrame
            all_tickers_returns_df = pd.concat([all_tickers_returns_df, new_column], axis=1)
    #return date_label_conversion_strip_time(all_tickers_returns_df, calculation_end_date)
    return all_tickers_returns_df

#-------------------------------------------------------------------
# step 2: calculate the annualized trailing total return from the data generated in step 1 & display
# Define a function to calculate the annualized trailing total return for a given number of years
def get_trailing_return(ticker, data, years):
    # Get the total return values for the last n years
    trailing_data = data[ticker].tail(years)
    # Check if there are empty values within years
    if trailing_data.isna().any():
        return "N/A"
    # Check if there are valid total return values for all years
    if len(trailing_data) == years:
        # Convert the percentage strings to numeric values
        trailing_data = trailing_data.astype(str).str.replace('%', '').astype(float)
        """ Calculate the annualized trailing total return using the formula from Investopedia[^1^][1]:
            Annualized Return = [(1 + r1) * (1 + r2) * ... * (1 + rn)]^(1/n) - 1
            Where r1, r2, ..., rn are the total return values for each year                    """
        annualized_trailing_return = (trailing_data + 1).prod() ** (1 / years) - 1

        # Format the result as a percentage with two decimal places
        annualized_trailing_return = annualized_trailing_return * 100
        annualized_trailing_return = annualized_trailing_return.round(2)
        return annualized_trailing_return
    else:
        return "N/A"

# Define a function to Loop through the list and print the trailing returns for each num_years
def get_trailing_return_column(ticker, annual_returns_df):
    trailing_return_column = {}
    for num_years in years_list:
        # Check if the ticker data is available in all_tickers_returns_df
        if ticker in annual_returns_df.columns:
            # using data from step 1, avoiding get_annual_returns_df(ticker) for less traffic from yahoo server
            data = annual_returns_df[[ticker]]
            trailing_return = get_trailing_return(ticker, data, num_years)
            trailing_return_column[f"{num_years}-Year"] = trailing_return
        else:
            print(f"Data not available for {ticker}. Skipping.")
            trailing_return_column[f"{num_years}-Year"] = "N/A"
    return trailing_return_column

# Create an empty DataFrame to store all tickers' trailing returns
def get_trailing_return_all(tickers, annual_returns_df):
    all_tickers_trailing_returns_df = pd.DataFrame(index=years_list)

    # Loop through each ticker in the list
    for ticker in tickers:
        trailing_returns = get_trailing_return_column(ticker, annual_returns_df)
        # Add the trailing returns to the DataFrame
        all_tickers_trailing_returns_df[ticker] = pd.Series(trailing_returns).values
    return all_tickers_trailing_returns_df

#-------------------------------------------------------------------
# step 3: calculate the cumulative return from the data (all_tickers_returns_df) generated in step 1 & display
# 4.1 Define a function to calculate the cumulative return for a given number of years from a ticker
def get_cumulative_return(ticker, data, years):
    # Calculate the cumulative return
    cumulative_return = (1 + data[ticker]).rolling(window=years).apply(lambda x: x.prod(), raw=True) - 1
    return cumulative_return

# Define a function to Loop through the list and return the cumulative returns for each num_years
def get_cumulative_return_column(ticker, annual_returns_df):
    cumulative_returns = {}
    for years in years_list:
        # Calculate the cumulative return for the given number of years
        cumulative_return = get_cumulative_return(ticker, annual_returns_df, years)
        # Get the last value, which is the cumulative return up to the current year
        cumulative_returns[years] = cumulative_return.iloc[-1]
    return cumulative_returns

def get_cumulative_return_all(tickers, annual_returns_df):
    # Create an empty DataFrame with years_list as the index for cumulative  returns
    all_tickers_cumulative_returns_df = pd.DataFrame(index=years_list)
    # Loop through each ticker in the list
    for ticker in tickers:
        cumulative_returns = get_cumulative_return_column(ticker, annual_returns_df)
        # Add the trailing returns to the DataFrame
        all_tickers_cumulative_returns_df[ticker] = pd.Series(cumulative_returns).values
    return all_tickers_cumulative_returns_df

#-------------------------------------------------------------------
# step 4: calculate the  CAGR (Compound Annual Growth Rate) from the data
# in all_tickers_cumulative_returns_df generated earlier & display
# Define a function to calculate the CAGR from the cumulative value and the years
def calculate_cagr(value, years):
    # Otherwise, calculate the CAGR using the formula
    cagr = (value + 1) ** (1 / np.array(years)) - 1
    #print("debug-cagr\n", cagr, "end")
    return cagr

# Define a function to format the Float64Index values into percentage strings
def format_to_percentage(value):
    # If any element in the value array is not null, format it as a percentage string with two decimal places
    if np.any(pd.notnull(value)):
        return f"{value:.2f}%"
    # Otherwise, return None
    return None

def get_cagr_return_all(all_tickers_cumulative_returns_df):
    # Apply the calculate_cagr function to each column of the DataFrame
    all_tickers_cagrs_df = all_tickers_cumulative_returns_df.apply(lambda x: calculate_cagr(x, x.index), axis=0)
    return all_tickers_cagrs_df

#---------------------------------------------------------------
# utility functions
# get the last trading day of S&P 500 in string format
def get_last_trading_day():
    # Get today's date, use .strftime("%Y-%m-%d") to convert to a string
    #calculation_end_date=datetime.now(pytz.timezone('America/New_York')).date().strftime("%Y-%m-%d")
    today_date_str=datetime.now(pytz.timezone('America/New_York')).date().strftime("%Y-%m-%d")
    stock = yf.Ticker("^GSPC") # S&P 500 (^GSPC) ticker
    #  search and see yfinance_BUG_1 NOTE in this file
    history_df=stock.history(period="max", end=today_date_str)["Close"]
    calculation_end_date_str = history_df.index.max().date().strftime("%Y-%m-%d")
    return calculation_end_date_str

def str_to_integer(integer_str):
    try:
        integer_number = int(integer_str)
        return integer_number
    except ValueError:
        return -1

# validate the date string
def is_valid_date(date_string):
    try:
        # Attempt to parse the date string
        datetime.strptime(date_string, "%Y-%m-%d")
        return True
    except ValueError:
        # Raised when the date string is not in the expected format
        return False

def date_label_conversion_strip_time(all_tickers_returns_df, calculation_end_date):
    all_tickers_returns_df.index=all_tickers_returns_df.index.date
    all_tickers_returns_df.index.name='date'
    # print("debug get_annual_returns_tickers_df", all_tickers_returns_df)
    # Convert calculation_end_date to a datetime object, replace the index's mon/day portion of date
    calculation_end_date_datetime_obj = datetime.strptime(calculation_end_date, "%Y-%m-%d")
    all_tickers_returns_df.index = all_tickers_returns_df.index.map(
      lambda x: x.replace(month=calculation_end_date_datetime_obj.month,
      day=calculation_end_date_datetime_obj.day))
    return all_tickers_returns_df

help_info_str="Input Formats:\n  \
            1. ticker list....................Example:  spy vfv.to xiu.to xic.to xfn.to ry.to \n \
            2. One of default ticker list, a number between 1 and 5....Example:   0, or 1, ...,5 \n \
            3. CalculationEndDate as prefix.  Example:   2020-12-31 2 \n \
            .........................................2020-12-31 spy vfv.to xiu.to xic.to xfn.to ry.to \n \
            note: daily adjusted close data are from Yahoo Finance. "

#---------------------------------------------------------------
# step 5:  Display in web page using gradio
# Gradio Web interface
def calculation_response(message, history):
    # if there is no input, display help information
    if message=="":
        return help_info_str

    tickers=message.split()

    # -----------
    #  set calculation_end_date
    # Get today's date, use .strftime("%Y-%m-%d") to convert to a string
    #calculation_end_date=datetime.now(pytz.timezone('America/New_York')).date().strftime("%Y-%m-%d")
    calculation_end_date = get_last_trading_day()
    # Check whether the first str is date for calculation end date
    if is_valid_date(tickers[0]):
        calculation_end_date = tickers[0] # reset calculation_end_date
        tickers.pop(0) # remove the first string which is the date

    # Check whether numebr 0, 1, 2, .. is selected for using a default ticker list
    integer_value=str_to_integer(tickers[0])
    if (integer_value >= 0 and integer_value <len(tickers_lists)):
        tickers=tickers_lists[integer_value]

    # if no tickers were set, display help information
    if len(tickers)==0:
        return help_info_str

    #---------------
    # Calculating Annual, Trailing, Cumulative, and CAGR & generating html for display
    # annual_returns
    output_string = f"\nAnnual Total Return (%) as {calculation_end_date}\n"
    annual_returns_dataframe=get_annual_returns_tickers_df(tickers, calculation_end_date)
    output_dataframe = annual_returns_dataframe.round(4)*100
    output_dataframe.index=output_dataframe.index.year
    # Convert the DataFrame to HTML, Combine the expected string outputs
    output_html1 = output_string + output_dataframe.to_html()

    # Removing the current year's data
    today_date=datetime.now(pytz.timezone('America/New_York')).date()
    all_tickers_returns_all_years_df=annual_returns_dataframe
    if (annual_returns_dataframe.index.max().year == today_date.year):
        annual_returns_dataframe=annual_returns_dataframe[annual_returns_dataframe.index.year < today_date.year]
        month_date = "12-31"
        calculation_end_date = f"{annual_returns_dataframe.index.max().year}-{month_date}"

    # Trailing Return
    output_string2 = f"\nTrailing Total Return (%) as {calculation_end_date}\n"
    output_dataframe2=get_trailing_return_all(tickers, annual_returns_dataframe)
    output_dataframe2.index.name="years"
    output_html2=output_string2 + output_dataframe2.to_html()

    # Cumulative Return
    output_string3 = f"\nCumulative Return (%) as {calculation_end_date}\n"
    cumulative_return_all_dataframe=get_cumulative_return_all(tickers, annual_returns_dataframe)
    output_dataframe3=cumulative_return_all_dataframe.round(4)*100
    output_dataframe3.index.name="years"
    output_html3=output_string3 + output_dataframe3.to_html()

    # CAGR Return
    output_string4 = f"\nCompound Annual Growth Rate (CAGR) (%) as {calculation_end_date}\n"
    output_dataframe4=get_cagr_return_all (cumulative_return_all_dataframe)
    output_dataframe4=output_dataframe4.round(4)*100
    output_html4=output_string4 + output_dataframe4.to_html()

    #output_html = output_html1 + output_html2 + output_html3 + output_html4
    output_html = output_html1 + output_html2 + output_html3
    return  output_html

demo = gr.ChatInterface(calculation_response)
demo.launch(debug=False, share=False)



Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Note: opening Chrome Inspector may crash demo inside Colab notebooks.

To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>

In [ ]:
# debug section
#calculation_response("0", [])
'''
calculation_end_date= "2012-01-02"
print("get_last_trading_day", calculation_end_date)
tickers = ["spy", "vfv.to", "vgg.to", "goog", "msft", "meta", "tsla","AMZN", "AAPL", "shop.to"]
output_string = f"\nAnnual Total Return (%) as {calculation_end_date}\n"
annual_returns_dataframe=get_annual_returns_tickers_df(tickers, calculation_end_date)
output_dataframe = annual_returns_dataframe.round(4)*100
print(output_dataframe)

help_info_str = """Input Formats:
| {0:30} | {1:30} |
| {2:30} | {3:30} |
| {4:30} | {5:30} |""".format(
    "1. ticker list.", "Example: spy vfv.to xiu.to xic.to xfn.to ry.to",
    "2. One of default ticker list.", "Example: 0|1|...|5",
    "3. CalculationEndDate as prefix.", "Example: 2020-12-31 2 or 2020-12-31 spy vfv.to xiu.to xic.to xfn.to ry.to"
)

'''

## example 7 backup

In [ ]:
'''
!pip install gradio
!pip install yfinance
!pip install datetime
!pip install pytz
!pip install pandas
!pip install numpy
'''
script_version = '(2024-01-06.1)'
import gradio as gr
import yfinance as yf
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import pytz
#--------------------------------------------------------------------

print_yearly_total_return = True
num_years_calculation=32   # total years for calculation

# Define a list of years to calculate the trailing returns, cumulative returns, and so on
# remove the row of current year row since it is not a full year.
years_list = [1, 2, 3, 5, 9, 10, 15, 20, 25, 30]

# Set the stock tickers list
tickers_lists = [["spy", "vfv.to","xiu.to"], #0
    ["spy", "vfv.to", "vgg.to", "zlu.to", "xiu.to", "vdy.to", "xfn.to", "ry.to", "td.to", "na.to",
      "slf.to", "gwo.to", "bce.to", "t.to", "rci-b.to", "enb.to", "trp.to", "zlb.to", "cp.to"], #1
    ["spy","vfv.to", "xiu.to", "fie.to", "xfn.to", "na.to","ry.to", "bmo.to", "cm.to", "bns.to", "cwb.to",
      "slf.to", "gwo.to", "bce.to", "t.to", "rci-b.to", "enb.to", "trp.to", "xdv.to","cdz.to","vdy.to"],  #2
    ["spy", "vfv.to", "vgg.to", "zlu.to","goog", "msft", "meta", "tsla","AMZN", "AAPL", "shop.to"],   #3
    ["^IXIC","^GSPC","spy", "vfv.to", "zsp.to", "xsp.to","^GSPTSE","xic.to","xiu.to","xfn.to","fie.to", "TX60.TS"], #4
    ["xic.to","xiu.to","xfn.to"]  #5
]

#-------------------------------------------------------------------
# step 1: fetch retrieve yearly total returns by yfinance & display
# Function to fetch data from yfinance and extract yearly total returns
def get_annual_returns_df(ticker, calculation_end_date):
    # Get the historical data for the given ticker
    stock = yf.Ticker(ticker)
    calculation_start_date = (datetime.strptime(calculation_end_date, "%Y-%m-%d")
                - timedelta(days=num_years_calculation * 365)).strftime("%Y-%m-%d")

    # Get today's date, use .strftime("%Y-%m-%d") to convert to a string
    today_date=datetime.now(pytz.timezone('America/New_York')).date().strftime("%Y-%m-%d")

    ''' yfinance_BUG_1 NOTE:
    There is bug for  when using end="2024-01-02" on 2024-01-02 which is the first trading of the year on 8:10pm
    for using annual_returns_history=stock.history(start=calculation_start_date,end=calculation_end_date)["Close"]
     (tried tickers: "^GSPC","spy" for stock = yf.Ticker(ticker)) that 2024-01-02 data is not returned.
     however, when end parameter, it is fine for returning  2024-01-02 data as below
     annual_returns_history=stock.history(start=calculation_start_date)["Close"]
    '''
    try:
        #'try' statement for handlingy the exception error of stock.history that a ticker is not yet at stock market, "shop.to" is not there in 2012
        # the following if statement for handling yfinance_BUG_1
        if (calculation_end_date != today_date):
            annual_returns_history=stock.history(start=calculation_start_date,end=calculation_end_date)["Close"]
        else:
            annual_returns_history=stock.history(start=calculation_start_date)["Close"]

        #print("debug get_annual_returns_df ", ticker, annual_returns_history)
        # Tried resample('A', origin=calculation_end_date),  "origin="" looks not working.
        # For 'A', 'Y', see https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html#offset-aliases
        annual_returns = annual_returns_history.resample('A').ffill().pct_change().dropna()
        #print("debug get_annual_returns_df after resample()", ticker, calculation_end_date, "\n", annual_returns)
    except:
        return  pd.DataFrame()
    else:
        annual_returns_df = pd.DataFrame(annual_returns, columns=['Close'])
        annual_returns_df.rename(columns={'Close': ticker}, inplace=True)
        return annual_returns_df

def get_annual_returns_tickers_df(tickers, calculation_end_date):
    # Create an empty DataFrame to store all tickers' total returns
    all_tickers_returns_df = pd.DataFrame()

    # Loop through each ticker in the list
    for ticker in tickers:
        ticker_returns_df = get_annual_returns_df(ticker, calculation_end_date)
        if not ticker_returns_df.empty:
            if all_tickers_returns_df.empty:
                all_tickers_returns_df = ticker_returns_df
            else:
                all_tickers_returns_df = pd.concat([all_tickers_returns_df, ticker_returns_df], axis=1)  # Concatenate DataFrames
        else:
            # New column with NaN values
            new_column_name = ticker
            new_column_values = [None] * len(all_tickers_returns_df)
            new_column = pd.DataFrame({new_column_name: new_column_values}, index=all_tickers_returns_df.index)
            # Concatenate the new column to the original DataFrame
            all_tickers_returns_df = pd.concat([all_tickers_returns_df, new_column], axis=1)
    #return date_label_conversion_strip_time(all_tickers_returns_df, calculation_end_date)
    return all_tickers_returns_df

#-------------------------------------------------------------------
# step 2: calculate the annualized trailing total return from the data generated in step 1 & display
# Define a function to calculate the annualized trailing total return for a given number of years
def get_trailing_return(ticker, data, years):
    # Get the total return values for the last n years
    trailing_data = data[ticker].tail(years)
    # Check if there are empty values within years
    if trailing_data.isna().any():
        return "N/A"
    # Check if there are valid total return values for all years
    if len(trailing_data) == years:
        # Convert the percentage strings to numeric values
        trailing_data = trailing_data.astype(str).str.replace('%', '').astype(float)
        """ Calculate the annualized trailing total return using the formula from Investopedia[^1^][1]:
            Annualized Return = [(1 + r1) * (1 + r2) * ... * (1 + rn)]^(1/n) - 1
            Where r1, r2, ..., rn are the total return values for each year                    """
        annualized_trailing_return = (trailing_data + 1).prod() ** (1 / years) - 1

        # Format the result as a percentage with two decimal places
        annualized_trailing_return = annualized_trailing_return * 100
        annualized_trailing_return = annualized_trailing_return.round(2)
        return annualized_trailing_return
    else:
        return "N/A"

# Define a function to Loop through the list and print the trailing returns for each num_years
def get_trailing_return_column(ticker, annual_returns_df):
    trailing_return_column = {}
    for num_years in years_list:
        # Check if the ticker data is available in all_tickers_returns_df
        if ticker in annual_returns_df.columns:
            # using data from step 1, avoiding get_annual_returns_df(ticker) for less traffic from yahoo server
            data = annual_returns_df[[ticker]]
            trailing_return = get_trailing_return(ticker, data, num_years)
            trailing_return_column[f"{num_years}-Year"] = trailing_return
        else:
            print(f"Data not available for {ticker}. Skipping.")
            trailing_return_column[f"{num_years}-Year"] = "N/A"
    return trailing_return_column

# Create an empty DataFrame to store all tickers' trailing returns
def get_trailing_return_all(tickers, annual_returns_df):
    all_tickers_trailing_returns_df = pd.DataFrame(index=years_list)

    # Loop through each ticker in the list
    for ticker in tickers:
        trailing_returns = get_trailing_return_column(ticker, annual_returns_df)
        # Add the trailing returns to the DataFrame
        all_tickers_trailing_returns_df[ticker] = pd.Series(trailing_returns).values
    return all_tickers_trailing_returns_df

#-------------------------------------------------------------------
# step 3: calculate the cumulative return from the data (all_tickers_returns_df) generated in step 1 & display
# 4.1 Define a function to calculate the cumulative return for a given number of years from a ticker
def get_cumulative_return(ticker, data, years):
    # Calculate the cumulative return
    cumulative_return = (1 + data[ticker]).rolling(window=years).apply(lambda x: x.prod(), raw=True) - 1
    return cumulative_return

# Define a function to Loop through the list and return the cumulative returns for each num_years
def get_cumulative_return_column(ticker, annual_returns_df):
    cumulative_returns = {}
    for years in years_list:
        # Calculate the cumulative return for the given number of years
        cumulative_return = get_cumulative_return(ticker, annual_returns_df, years)
        # Get the last value, which is the cumulative return up to the current year
        cumulative_returns[years] = cumulative_return.iloc[-1]
    return cumulative_returns

def get_cumulative_return_all(tickers, annual_returns_df):
    # Create an empty DataFrame with years_list as the index for cumulative  returns
    all_tickers_cumulative_returns_df = pd.DataFrame(index=years_list)
    # Loop through each ticker in the list
    for ticker in tickers:
        cumulative_returns = get_cumulative_return_column(ticker, annual_returns_df)
        # Add the trailing returns to the DataFrame
        all_tickers_cumulative_returns_df[ticker] = pd.Series(cumulative_returns).values
    return all_tickers_cumulative_returns_df

#-------------------------------------------------------------------
# step 4: calculate the  CAGR (Compound Annual Growth Rate) from the data
# in all_tickers_cumulative_returns_df generated earlier & display
# Define a function to calculate the CAGR from the cumulative value and the years
def calculate_cagr(value, years):
    # Otherwise, calculate the CAGR using the formula
    cagr = (value + 1) ** (1 / np.array(years)) - 1
    #print("debug-cagr\n", cagr, "end")
    return cagr

# Define a function to format the Float64Index values into percentage strings
def format_to_percentage(value):
    # If any element in the value array is not null, format it as a percentage string with two decimal places
    if np.any(pd.notnull(value)):
        return f"{value:.2f}%"
    # Otherwise, return None
    return None

def get_cagr_return_all(all_tickers_cumulative_returns_df):
    # Apply the calculate_cagr function to each column of the DataFrame
    all_tickers_cagrs_df = all_tickers_cumulative_returns_df.apply(lambda x: calculate_cagr(x, x.index), axis=0)
    return all_tickers_cagrs_df

#---------------------------------------------------------------
# utility functions
# get the last trading day of S&P 500 in string format
def get_last_trading_day():
    # Get today's date, use .strftime("%Y-%m-%d") to convert to a string
    #calculation_end_date=datetime.now(pytz.timezone('America/New_York')).date().strftime("%Y-%m-%d")
    today_date_str=datetime.now(pytz.timezone('America/New_York')).date().strftime("%Y-%m-%d")
    stock = yf.Ticker("^GSPC") # S&P 500 (^GSPC) ticker
    #  search and see yfinance_BUG_1 NOTE in this file
    history_df=stock.history(period="max", end=today_date_str)["Close"]
    calculation_end_date_str = history_df.index.max().date().strftime("%Y-%m-%d")
    return calculation_end_date_str

def str_to_integer(integer_str):
    try:
        integer_number = int(integer_str)
        return integer_number
    except ValueError:
        return -1

# validate the date string
def is_valid_date(date_string):
    try:
        # Attempt to parse the date string
        datetime.strptime(date_string, "%Y-%m-%d")
        return True
    except ValueError:
        # Raised when the date string is not in the expected format
        return False

def date_label_conversion_strip_time(all_tickers_returns_df, calculation_end_date):
    all_tickers_returns_df.index=all_tickers_returns_df.index.date
    all_tickers_returns_df.index.name='date'
    # print("debug get_annual_returns_tickers_df", all_tickers_returns_df)
    # Convert calculation_end_date to a datetime object, replace the index's mon/day portion of date
    calculation_end_date_datetime_obj = datetime.strptime(calculation_end_date, "%Y-%m-%d")
    all_tickers_returns_df.index = all_tickers_returns_df.index.map(
      lambda x: x.replace(month=calculation_end_date_datetime_obj.month,
      day=calculation_end_date_datetime_obj.day))
    return all_tickers_returns_df

help_info_str="Input Formats:\n  \
            1. ticker list....................Example:  spy vfv.to xiu.to xic.to xfn.to ry.to \n \
            2. One of default ticker list, a number between 1 and 5....Example:   0, or 1, ...,5 \n \
            3. CalculationEndDate as prefix.  Example:   2020-12-31 2 \n \
            .........................................2020-12-31 spy vfv.to xiu.to xic.to xfn.to ry.to \n \
            note: daily adjusted close data are from Yahoo Finance. "

#---------------------------------------------------------------
# step 5:  Display in web page using gradio
# Gradio Web interface
def calculation_response(message, history):
    # if there is no input, display help information
    if message=="":
        return help_info_str

    tickers=message.split()

    # -----------
    #  set calculation_end_date
    # Get today's date, use .strftime("%Y-%m-%d") to convert to a string
    #calculation_end_date=datetime.now(pytz.timezone('America/New_York')).date().strftime("%Y-%m-%d")
    calculation_end_date = get_last_trading_day()
    # Check whether the first str is date for calculation end date
    if is_valid_date(tickers[0]):
        calculation_end_date = tickers[0] # reset calculation_end_date
        tickers.pop(0) # remove the first string which is the date

    # Check whether numebr 0, 1, 2, .. is selected for using a default ticker list
    integer_value=str_to_integer(tickers[0])
    if (integer_value >= 0 and integer_value <len(tickers_lists)):
        tickers=tickers_lists[integer_value]

    # if no tickers were set, display help information
    if len(tickers)==0:
        return help_info_str

    #---------------
    # Calculating Annual, Trailing, Cumulative, and CAGR & generating html for display
    # annual_returns
    output_string = f"\nAnnual Total Return (%) as {calculation_end_date}\n"
    annual_returns_dataframe=get_annual_returns_tickers_df(tickers, calculation_end_date)
    output_dataframe = annual_returns_dataframe.round(4)*100
    output_dataframe.index=output_dataframe.index.year
    # Convert the DataFrame to HTML, Combine the expected string outputs
    output_html1 = output_string + output_dataframe.to_html()

    # Removing the current year's data
    today_date=datetime.now(pytz.timezone('America/New_York')).date()
    all_tickers_returns_all_years_df=annual_returns_dataframe
    if (annual_returns_dataframe.index.max().year == today_date.year):
        annual_returns_dataframe=annual_returns_dataframe[annual_returns_dataframe.index.year < today_date.year]
        month_date = "12-31"
        calculation_end_date = f"{annual_returns_dataframe.index.max().year}-{month_date}"

    # Trailing Return
    output_string2 = f"\nTrailing Total Return (%) as {calculation_end_date}\n"
    output_dataframe2=get_trailing_return_all(tickers, annual_returns_dataframe)
    output_dataframe2.index.name="years"
    output_html2=output_string2 + output_dataframe2.to_html()

    # Cumulative Return
    output_string3 = f"\nCumulative Return (%) as {calculation_end_date}\n"
    cumulative_return_all_dataframe=get_cumulative_return_all(tickers, annual_returns_dataframe)
    output_dataframe3=cumulative_return_all_dataframe.round(4)*100
    output_dataframe3.index.name="years"
    output_html3=output_string3 + output_dataframe3.to_html()

    # CAGR Return
    output_string4 = f"\nCompound Annual Growth Rate (CAGR) (%) as {calculation_end_date}\n"
    output_dataframe4=get_cagr_return_all (cumulative_return_all_dataframe)
    output_dataframe4=output_dataframe4.round(4)*100
    output_html4=output_string4 + output_dataframe4.to_html()

    #output_html = output_html1 + output_html2 + output_html3 + output_html4
    output_html = output_html1 + output_html2 + output_html3
    return  output_html

demo = gr.ChatInterface(calculation_response)
demo.launch(debug=False, share=False)



Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Note: opening Chrome Inspector may crash demo inside Colab notebooks.

To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>

# Example 4: Calendar-year based - Yearly Total Return, Annualized Trailing return, Cumulative Returns & Compound Annual Growth Rate (GAGR)

In [ ]:
"""
### CODE start (for AI tool)
Retrieve multiple tickers for total returns and trailing total returns from yahoo
History:
Author: Gang Luo
"""
result_marker='###'
script_version = '(2024-01-04)'
print(f"The result of version-{script_version} script  start:\n {result_marker}")
#!pip install yfinance
import yfinance as yf
import pandas as pd

print_yearly_total_return = True

# Set the stock tickers list
tickers_list1 = ["spy", "vfv.to", "vgg.to", "zlu.to", "xiu.to", "xfn.to", "ry.to", "td.to", "na.to",
 "cm.to", "slf.to", "gwo.to", "bce.to", "t.to", "rci-b.to", "enb.to", "trp.to", "zlb.to"]
tickers_list2 = ["vfv.to", "xiu.to", "xic.to", "xfn.to", "ry.to", "td.to", "na.to",
                "bns.to", "bmo.to", "cm.to", "cwb.to", "slf.to", "gwo.to"]
tickers_list3 = ["spy", "vfv.to", "xfn.to", "ry.to"]
tickers_list = tickers_list1

#-------------------------------------------------------------------
# step 1: fetch retrieve yearly total returns by yfinance & display
# Function to fetch data from yfinance and extract yearly total returns
def get_annual_returns_df(ticker):
    # Get the historical data for the given ticker
    stock = yf.Ticker(ticker)
    ''' Get annual total return data.   Explanation the following statement
    1.  'Close'  in .
    Ticker(ticker).history is the same as 'Adj Close' in .download(ticker, ...)
        print(df.columns.tolist()) - Display all column names of DataFrame
        yf.Ticker(ticker).history: ['Open', 'High', 'Low', 'Close', 'Volume', 'Dividends', 'Stock Splits']
        yf.download(ticker, ...): ['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume']
    2.  ["Close"]: Selects the "Close" column from the historical stock data.
        .resample('Y'): Resamples the time series data annually ('Y'), based on the last trading day of each year.
        .resample('A'): Resamples the time series data annually ('A'), based on the any trading day of each year.
        .ffill(): Forward fills missing values, ensuring that each annual period has a value.
        .pct_change(): Calculates the percentage change between the current and previous values, representing the annual returns.
        .dropna(): Removes any rows with missing values (which may occur after calculating percentage changes).
    '''
    annual_returns = stock.history(period="max")["Close"].resample('Y').ffill().pct_change().dropna()
    annual_returns_df = pd.DataFrame(annual_returns, columns=['Close'])
    annual_returns_df.rename(columns={'Close': ticker}, inplace=True)
    return annual_returns_df
# Create an empty DataFrame to store all tickers' total returns
all_tickers_returns_df = pd.DataFrame()

# Loop through each ticker in the list
for ticker in tickers_list:
    ticker_returns_df = get_annual_returns_df(ticker)
    if not ticker_returns_df.empty:
        if all_tickers_returns_df.empty:
            all_tickers_returns_df = ticker_returns_df
        else:
            all_tickers_returns_df = pd.concat([all_tickers_returns_df, ticker_returns_df], axis=1)  # Concatenate DataFrames

# Display the results if the DataFrame is not empty

if all_tickers_returns_df.empty:
    print("No data available for the specified tickers.")

#print("\nAnnual Total Return (%) History:")
# keep the date portion of index only - stripping time portion
#all_tickers_returns_df.index=all_tickers_returns_df.index.date
#print(all_tickers_returns_df)

#-------------------------------------------------------------------
# step 2:
# Define a list of years to calculate the trailing returns, cumulative returns, and so on
# remove the row of current year row since it is not a full year.
years_list = [1, 2, 3, 5, 10, 15, 20, 25, 30]
current_year=all_tickers_returns_df.index.year.max()
all_tickers_returns_with_current_year_df=all_tickers_returns_df
all_tickers_returns_df=all_tickers_returns_df[all_tickers_returns_df.index.year < current_year]

#-------------------------------------------------------------------
# step 3: calculate the annualized trailing total return from the data generated in step 1 & display
# Define a function to calculate the annualized trailing total return for a given number of years
def get_trailing_return(ticker, data, years):
    # Get the total return values for the last n years
    trailing_data = data[ticker].tail(years)
    # Check if there are empty values within years
    if trailing_data.isna().any():
        return "N/A"
    # Check if there are valid total return values for all years
    if len(trailing_data) == years:
        # Convert the percentage strings to numeric values
        trailing_data = trailing_data.astype(str).str.replace('%', '').astype(float)
        """ Calculate the annualized trailing total return using the formula from Investopedia[^1^][1]:
            Annualized Return = [(1 + r1) * (1 + r2) * ... * (1 + rn)]^(1/n) - 1
            Where r1, r2, ..., rn are the total return values for each year                    """
        annualized_trailing_return = (trailing_data + 1).prod() ** (1 / years) - 1

        # Format the result as a percentage with two decimal places
        annualized_trailing_return = annualized_trailing_return * 100
        annualized_trailing_return = annualized_trailing_return.round(2)
        return annualized_trailing_return
    else:
        return "N/A"

# Define a function to Loop through the list and print the trailing returns for each num_years
def get_trailing_return_column(ticker):
    trailing_return_column = {}
    for num_years in years_list:
        # Check if the ticker data is available in all_tickers_returns_df
        if ticker in all_tickers_returns_df.columns:
            # using data from step 1, avoiding get_annual_returns_df(ticker) for less traffic from yahoo server
            data = all_tickers_returns_df[[ticker]]
            trailing_return = get_trailing_return(ticker, data, num_years)
            trailing_return_column[f"{num_years}-Year"] = trailing_return
        else:
            print(f"Data not available for {ticker}. Skipping.")
            trailing_return_column[f"{num_years}-Year"] = "N/A"
    return trailing_return_column

# Create an empty DataFrame to store all tickers' trailing returns
all_tickers_trailing_returns_df = pd.DataFrame(index=years_list)

# Loop through each ticker in the list
for ticker in tickers_list:
    trailing_returns = get_trailing_return_column(ticker)
    # Add the trailing returns to the DataFrame
    all_tickers_trailing_returns_df[ticker] = pd.Series(trailing_returns).values

# Display the trailing returns DataFrame
#print("\nAnnualized Trailing Returns as end of year", current_year-1, ":")
#print("years")
#print(all_tickers_trailing_returns_df)

#-------------------------------------------------------------------
# step 4: calculate the cumulative return from the data (all_tickers_returns_df) generated in step 1 & display
# 4.1 Define a function to calculate the cumulative return for a given number of years from a ticker
def get_cumulative_return(ticker, data, years):
    # Calculate the cumulative return
    cumulative_return = (1 + data[ticker]).rolling(window=years).apply(lambda x: x.prod(), raw=True) - 1
    return cumulative_return

# Create an empty DataFrame with years_list as the index for cumulative  returns
all_tickers_cumulative_returns_df = pd.DataFrame(index=years_list)

# Define a function to Loop through the list and return the cumulative returns for each num_years
def get_cumulative_return_column(ticker):
    cumulative_returns = {}
    for years in years_list:
        # Calculate the cumulative return for the given number of years
        cumulative_return = get_cumulative_return(ticker, all_tickers_returns_df, years)
        # Get the last value, which is the cumulative return up to the current year
        cumulative_returns[years] = cumulative_return.iloc[-1]
    return cumulative_returns

# Loop through each ticker in the list
for ticker in tickers_list:
    cumulative_returns = get_cumulative_return_column(ticker)
    # Add the trailing returns to the DataFrame
    all_tickers_cumulative_returns_df[ticker] = pd.Series(cumulative_returns).values

# Display the cumulative returns DataFrame
#print("\nCumulative Returns as end of year", current_year-1, ":")
#print("years")
#print(all_tickers_cumulative_returns_df.round(4) * 100)

#-------------------------------------------------------------------
# step 5: calculate the  CAGR from the data (all_tickers_cumulative_returns_df) generated earlier & display
# Define a function to calculate the CAGR from the cumulative value and the years
import numpy as np
def calculate_cagr(value, years):
    # Otherwise, calculate the CAGR using the formula
    cagr = (value + 1) ** (1 / np.array(years)) - 1
    #print("debug-cagr\n", cagr, "end")
    return cagr

# Define a function to format the Float64Index values into percentage strings
def format_to_percentage(value):
    # If any element in the value array is not null, format it as a percentage string with two decimal places
    if np.any(pd.notnull(value)):
        return f"{value:.2f}%"
    # Otherwise, return None
    return None

# Apply the calculate_cagr function to each column of the DataFrame
all_tickers_cagrs_df = all_tickers_cumulative_returns_df.apply(lambda x: calculate_cagr(x, x.index), axis=0)


#----------------------------------------------------
# Display the formatted DataFrame  all_tickers_returns_df
print("\nAnnual Total Return (%) as year", current_year)
if (print_yearly_total_return):
    all_tickers_returns_with_current_year_df.index=all_tickers_returns_with_current_year_df.index.year
    all_tickers_returns_with_current_year_df.index.name='year'
    print(all_tickers_returns_with_current_year_df.round(4) * 100)

# Display the trailing returns DataFrame
print("\nAnnualized Trailing Returns as end of year", current_year-1, ":")
all_tickers_trailing_returns_df.index.name = 'Years'
print(all_tickers_trailing_returns_df)

# Display the cumulative returns DataFrame
print("\nCumulative Returns as end of year", current_year-1, ":")
all_tickers_cumulative_returns_df.index.name = 'Years'
print(all_tickers_cumulative_returns_df.round(4) * 100)


# Display the formatted DataFrame  all_tickers_cagrs_df
print("\nCompound Annual Growth Rates (GAGRs) (%) as end of year", current_year-1, ":")
all_tickers_cagrs_df.index.name = 'Years'
print(all_tickers_cagrs_df.round(4) * 100)


# print an indicator to mark the end of execution result
print(f"The result of version-{script_version} script  ends here.\n {result_marker}")
### CODE end (for AI tool)


The result of version-(2024-01-04) script  start:
 ###

Annual Total Return (%) as year 2024
        spy  vfv.to  vgg.to  zlu.to  xiu.to  xfn.to  ry.to  td.to   na.to  \
year                                                                        
1994   0.40     NaN     NaN     NaN     NaN     NaN    NaN    NaN     NaN   
1995  38.05     NaN     NaN     NaN     NaN     NaN    NaN    NaN     NaN   
1996  22.50     NaN     NaN     NaN     NaN     NaN  59.20  52.30   29.73   
1997  33.48     NaN     NaN     NaN     NaN     NaN  61.23  56.87   75.97   
1998  28.69     NaN     NaN     NaN     NaN     NaN   3.58   2.50    7.92   
1999  20.39     NaN     NaN     NaN     NaN     NaN -14.82  46.56  -22.65   
2000  -9.74     NaN     NaN     NaN    8.02     NaN  65.34  14.84   43.78   
2001 -11.76     NaN     NaN     NaN  -14.24     NaN   4.90  -2.81   11.65   
2002 -21.58     NaN     NaN     NaN  -14.02   -3.06  14.85 -14.56    8.75   
2003  28.18     NaN     NaN     NaN   24.82   26.14   9.13  

##  backup - Example 4: Calendar-year based - Yearly Total Return, Annualized Trailing return, Cumulative Returns & Compound Annual Growth Rate (GAGR)

In [ ]:
"""
### CODE start (for AI tool)
Retrieve multiple tickers for total returns and trailing total returns from yahoo
History:
Author: Gang Luo
"""
result_marker='###'
script_version = '(2024-01-04)'
print(f"The result of version-{script_version} script  start:\n {result_marker}")
#!pip install yfinance
import yfinance as yf
import pandas as pd

print_yearly_total_return = True

# Set the stock tickers list
tickers_list1 = ["spy", "vfv.to", "vgg.to", "zlu.to", "xiu.to", "xfn.to", "ry.to", "td.to", "na.to",
 "cm.to", "slf.to", "gwo.to", "bce.to", "t.to", "rci-b.to", "enb.to", "trp.to", "zlb.to"]
tickers_list2 = ["vfv.to", "xiu.to", "xic.to", "xfn.to", "ry.to", "td.to", "na.to",
                "bns.to", "bmo.to", "cm.to", "cwb.to", "slf.to", "gwo.to"]
tickers_list3 = ["spy", "vfv.to", "xfn.to", "ry.to"]
tickers_list = tickers_list1

#-------------------------------------------------------------------
# step 1: fetch retrieve yearly total returns by yfinance & display
# Function to fetch data from yfinance and extract yearly total returns
def get_annual_returns_df(ticker):
    # Get the historical data for the given ticker
    stock = yf.Ticker(ticker)
    ''' Get annual total return data.   Explanation the following statement
    1.  'Close'  in .
    Ticker(ticker).history is the same as 'Adj Close' in .download(ticker, ...)
        print(df.columns.tolist()) - Display all column names of DataFrame
        yf.Ticker(ticker).history: ['Open', 'High', 'Low', 'Close', 'Volume', 'Dividends', 'Stock Splits']
        yf.download(ticker, ...): ['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume']
    2.  ["Close"]: Selects the "Close" column from the historical stock data.
        .resample('Y'): Resamples the time series data annually ('Y'), based on the last trading day of each year.
        .resample('A'): Resamples the time series data annually ('A'), based on the any trading day of each year.
        .ffill(): Forward fills missing values, ensuring that each annual period has a value.
        .pct_change(): Calculates the percentage change between the current and previous values, representing the annual returns.
        .dropna(): Removes any rows with missing values (which may occur after calculating percentage changes).
    '''
    annual_returns = stock.history(period="max")["Close"].resample('Y').ffill().pct_change().dropna()
    annual_returns_df = pd.DataFrame(annual_returns, columns=['Close'])
    annual_returns_df.rename(columns={'Close': ticker}, inplace=True)
    return annual_returns_df
# Create an empty DataFrame to store all tickers' total returns
all_tickers_returns_df = pd.DataFrame()

# Loop through each ticker in the list
for ticker in tickers_list:
    ticker_returns_df = get_annual_returns_df(ticker)
    if not ticker_returns_df.empty:
        if all_tickers_returns_df.empty:
            all_tickers_returns_df = ticker_returns_df
        else:
            all_tickers_returns_df = pd.concat([all_tickers_returns_df, ticker_returns_df], axis=1)  # Concatenate DataFrames

# Display the results if the DataFrame is not empty

if all_tickers_returns_df.empty:
    print("No data available for the specified tickers.")

#print("\nAnnual Total Return (%) History:")
# keep the date portion of index only - stripping time portion
#all_tickers_returns_df.index=all_tickers_returns_df.index.date
#print(all_tickers_returns_df)

#-------------------------------------------------------------------
# step 2:
# Define a list of years to calculate the trailing returns, cumulative returns, and so on
# remove the row of current year row since it is not a full year.
years_list = [1, 2, 3, 5, 10, 15, 20, 25, 30]
current_year=all_tickers_returns_df.index.year.max()
all_tickers_returns_with_current_year_df=all_tickers_returns_df
all_tickers_returns_df=all_tickers_returns_df[all_tickers_returns_df.index.year < current_year]

#-------------------------------------------------------------------
# step 3: calculate the annualized trailing total return from the data generated in step 1 & display
# Define a function to calculate the annualized trailing total return for a given number of years
def get_trailing_return(ticker, data, years):
    # Get the total return values for the last n years
    trailing_data = data[ticker].tail(years)
    # Check if there are empty values within years
    if trailing_data.isna().any():
        return "N/A"
    # Check if there are valid total return values for all years
    if len(trailing_data) == years:
        # Convert the percentage strings to numeric values
        trailing_data = trailing_data.astype(str).str.replace('%', '').astype(float)
        """ Calculate the annualized trailing total return using the formula from Investopedia[^1^][1]:
            Annualized Return = [(1 + r1) * (1 + r2) * ... * (1 + rn)]^(1/n) - 1
            Where r1, r2, ..., rn are the total return values for each year                    """
        annualized_trailing_return = (trailing_data + 1).prod() ** (1 / years) - 1

        # Format the result as a percentage with two decimal places
        annualized_trailing_return = annualized_trailing_return * 100
        annualized_trailing_return = annualized_trailing_return.round(2)
        return annualized_trailing_return
    else:
        return "N/A"

# Define a function to Loop through the list and print the trailing returns for each num_years
def get_trailing_return_column(ticker):
    trailing_return_column = {}
    for num_years in years_list:
        # Check if the ticker data is available in all_tickers_returns_df
        if ticker in all_tickers_returns_df.columns:
            # using data from step 1, avoiding get_annual_returns_df(ticker) for less traffic from yahoo server
            data = all_tickers_returns_df[[ticker]]
            trailing_return = get_trailing_return(ticker, data, num_years)
            trailing_return_column[f"{num_years}-Year"] = trailing_return
        else:
            print(f"Data not available for {ticker}. Skipping.")
            trailing_return_column[f"{num_years}-Year"] = "N/A"
    return trailing_return_column

# Create an empty DataFrame to store all tickers' trailing returns
all_tickers_trailing_returns_df = pd.DataFrame(index=years_list)

# Loop through each ticker in the list
for ticker in tickers_list:
    trailing_returns = get_trailing_return_column(ticker)
    # Add the trailing returns to the DataFrame
    all_tickers_trailing_returns_df[ticker] = pd.Series(trailing_returns).values

# Display the trailing returns DataFrame
#print("\nAnnualized Trailing Returns as end of year", current_year-1, ":")
#print("years")
#print(all_tickers_trailing_returns_df)

#-------------------------------------------------------------------
# step 4: calculate the cumulative return from the data (all_tickers_returns_df) generated in step 1 & display
# 4.1 Define a function to calculate the cumulative return for a given number of years from a ticker
def get_cumulative_return(ticker, data, years):
    # Calculate the cumulative return
    cumulative_return = (1 + data[ticker]).rolling(window=years).apply(lambda x: x.prod(), raw=True) - 1
    return cumulative_return

# Create an empty DataFrame with years_list as the index for cumulative  returns
all_tickers_cumulative_returns_df = pd.DataFrame(index=years_list)

# Define a function to Loop through the list and return the cumulative returns for each num_years
def get_cumulative_return_column(ticker):
    cumulative_returns = {}
    for years in years_list:
        # Calculate the cumulative return for the given number of years
        cumulative_return = get_cumulative_return(ticker, all_tickers_returns_df, years)
        # Get the last value, which is the cumulative return up to the current year
        cumulative_returns[years] = cumulative_return.iloc[-1]
    return cumulative_returns

# Loop through each ticker in the list
for ticker in tickers_list:
    cumulative_returns = get_cumulative_return_column(ticker)
    # Add the trailing returns to the DataFrame
    all_tickers_cumulative_returns_df[ticker] = pd.Series(cumulative_returns).values

# Display the cumulative returns DataFrame
#print("\nCumulative Returns as end of year", current_year-1, ":")
#print("years")
#print(all_tickers_cumulative_returns_df.round(4) * 100)

#-------------------------------------------------------------------
# step 5: calculate the  CAGR from the data (all_tickers_cumulative_returns_df) generated earlier & display
# Define a function to calculate the CAGR from the cumulative value and the years
import numpy as np
def calculate_cagr(value, years):
    # Otherwise, calculate the CAGR using the formula
    cagr = (value + 1) ** (1 / np.array(years)) - 1
    #print("debug-cagr\n", cagr, "end")
    return cagr

# Define a function to format the Float64Index values into percentage strings
def format_to_percentage(value):
    # If any element in the value array is not null, format it as a percentage string with two decimal places
    if np.any(pd.notnull(value)):
        return f"{value:.2f}%"
    # Otherwise, return None
    return None

# Apply the calculate_cagr function to each column of the DataFrame
all_tickers_cagrs_df = all_tickers_cumulative_returns_df.apply(lambda x: calculate_cagr(x, x.index), axis=0)


#----------------------------------------------------
# Display the formatted DataFrame  all_tickers_returns_df
print("\nAnnual Total Return (%) as year", current_year)
if (print_yearly_total_return):
    all_tickers_returns_with_current_year_df.index=all_tickers_returns_with_current_year_df.index.year
    all_tickers_returns_with_current_year_df.index.name='year'
    print(all_tickers_returns_with_current_year_df.round(4) * 100)

# Display the trailing returns DataFrame
print("\nAnnualized Trailing Returns as end of year", current_year-1, ":")
all_tickers_trailing_returns_df.index.name = 'Years'
print(all_tickers_trailing_returns_df)

# Display the cumulative returns DataFrame
print("\nCumulative Returns as end of year", current_year-1, ":")
all_tickers_cumulative_returns_df.index.name = 'Years'
print(all_tickers_cumulative_returns_df.round(4) * 100)


# Display the formatted DataFrame  all_tickers_cagrs_df
print("\nCompound Annual Growth Rates (GAGRs) (%) as end of year", current_year-1, ":")
all_tickers_cagrs_df.index.name = 'Years'
print(all_tickers_cagrs_df.round(4) * 100)


# print an indicator to mark the end of execution result
print(f"The result of version-{script_version} script  ends here.\n {result_marker}")
### CODE end (for AI tool)


The result of version-(2024-01-04) script  start:
 ###

Annual Total Return (%) as year 2024
        spy  vfv.to  vgg.to  zlu.to  xiu.to  xfn.to  ry.to  td.to   na.to  \
year                                                                        
1994   0.40     NaN     NaN     NaN     NaN     NaN    NaN    NaN     NaN   
1995  38.05     NaN     NaN     NaN     NaN     NaN    NaN    NaN     NaN   
1996  22.50     NaN     NaN     NaN     NaN     NaN  59.20  52.30   29.73   
1997  33.48     NaN     NaN     NaN     NaN     NaN  61.23  56.87   75.97   
1998  28.69     NaN     NaN     NaN     NaN     NaN   3.58   2.50    7.92   
1999  20.39     NaN     NaN     NaN     NaN     NaN -14.82  46.56  -22.65   
2000  -9.74     NaN     NaN     NaN    8.02     NaN  65.34  14.84   43.78   
2001 -11.76     NaN     NaN     NaN  -14.24     NaN   4.90  -2.81   11.65   
2002 -21.58     NaN     NaN     NaN  -14.02   -3.06  14.85 -14.56    8.75   
2003  28.18     NaN     NaN     NaN   24.82   26.14   9.13  

In [ ]:
"""
### CODE start (for AI tool)
Retrieve multiple tickers for total returns and trailing total returns from yahoo
History:
Dec 21, 2023, version 12

Author: Gang Luo
"""
result_marker='###'
script_version = 12
print(f"The result of version-{script_version} script  start:\n {result_marker}")
#!pip install yfinance
import yfinance as yf
import pandas as pd

# Set the stock tickers list
#tickers_list = ["spy", "vfv.to", "xiu.to", "xic.to", "xfn.to", "ry.to", "enb.to"]
tickers_list = ["xiu.to", "xfn.to", "ry.to", "td.to", "na.to", "cwb.to", "slf.to", "gwo.to"]
#-------------------------------------------------------------------
# step 1: fetch retrieve yearly total returns by yfinance & display

# Function to fetch data from yfinance and extract yearly total returns
def get_annual_returns_df(ticker):
    # Get the historical data for the given ticker
    stock = yf.Ticker(ticker)
    ''' Get annual total return data.   Explanation the following statement
    1.  'Close'  in .Ticker(ticker).history is the same as 'Adj Close' in .download(ticker, ...)
        print(df.columns.tolist()) - Display all column names of DataFrame
        yf.Ticker(ticker).history: ['Open', 'High', 'Low', 'Close', 'Volume', 'Dividends', 'Stock Splits']
        yf.download(ticker, ...): ['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume']
    2.  ["Close"]: Selects the "Close" column from the historical stock data.
        .resample('Y'): Resamples the time series data annually ('Y').
        .ffill(): Forward fills missing values, ensuring that each annual period has a value.
        .pct_change(): Calculates the percentage change between the current and previous values, representing the annual returns.
        .dropna(): Removes any rows with missing values (which may occur after calculating percentage changes).
    '''
    annual_returns = stock.history(period="max")["Close"].resample('Y').ffill().pct_change().dropna()
    annual_returns_df = pd.DataFrame(annual_returns, columns=['Close'])
    annual_returns_df.rename(columns={'Close': ticker}, inplace=True)
    return annual_returns_df
# Create an empty DataFrame to store all tickers' total returns
all_tickers_returns_df = pd.DataFrame()

# Loop through each ticker in the list
for ticker in tickers_list:
    ticker_returns_df = get_annual_returns_df(ticker)
    if not ticker_returns_df.empty:
        if all_tickers_returns_df.empty:
            all_tickers_returns_df = ticker_returns_df
        else:
            all_tickers_returns_df = pd.concat([all_tickers_returns_df, ticker_returns_df], axis=1)  # Concatenate DataFrames

# Display the results if the DataFrame is not empty
print("\nAnnual Total Return (%) History:")
if not all_tickers_returns_df.empty:
    # keep the date portion of index only - stripping time portion
    #all_tickers_returns_df.index=all_tickers_returns_df.index.date

    # Format the DataFrame to display percentage
    all_tickers_returns_df_copy = all_tickers_returns_df * 100
    all_tickers_returns_df_copy=all_tickers_returns_df_copy.round(2)
    all_tickers_returns_df_copy.index=all_tickers_returns_df_copy.index.date
    print(all_tickers_returns_df_copy)
else:
    print("No data available for the specified tickers.")

#-------------------------------------------------------------------
# step 2: Remove the row of the current year for the later steps
# Define a list of years to calculate the trailing returns, cumulative returns, and so on
years_list = [1, 2, 3, 5, 10, 15, 20, 25, 30]
current_year=all_tickers_returns_df.index.year.max()
all_tickers_returns_df['Year']=all_tickers_returns_df.index.year
all_tickers_returns_df=all_tickers_returns_df[all_tickers_returns_df['Year'] < current_year]


#-------------------------------------------------------------------
# step 3: calculate the annualized trailing total return from the data generated in step 1 & display
# Define a function to calculate the annualized trailing total return for a given number of years
def get_trailing_return(ticker, data, years):
    # Get the total return values for the last n years
    trailing_data = data[ticker].tail(years)

    # Check if there are valid total return values for all years
    if len(trailing_data) == years:
        # Convert the percentage strings to numeric values
        trailing_data = trailing_data.astype(str).str.replace('%', '').astype(float)

        """ Calculate the annualized trailing total return using the formula from Investopedia[^1^][1]:
            Annualized Return = [(1 + r1) * (1 + r2) * ... * (1 + rn)]^(1/n) - 1
            Where r1, r2, ..., rn are the total return values for each year                    """
        annualized_trailing_return = (trailing_data + 1).prod() ** (1 / years) - 1

        # Format the result as a percentage with two decimal places
        annualized_trailing_return = annualized_trailing_return * 100
        annualized_trailing_return = annualized_trailing_return.round(2)
        return annualized_trailing_return
    else:
        return "N/A"

# Create an empty DataFrame with years_list as the index for trailing total returns
all_tickers_trailing_returns_df = pd.DataFrame(index=years_list)

# Define a function to Loop through the list and print the trailing returns for each num_years
def get_trailing_return_column(ticker):
    trailing_return_column = {}
    for num_years in years_list:
        # Check if the ticker data is available in all_tickers_returns_df
        if ticker in all_tickers_returns_df.columns:
            # using data from step 1, avoiding get_annual_returns_df(ticker) for less traffic from yahoo server
            data = all_tickers_returns_df[[ticker]]
            trailing_return = get_trailing_return(ticker, data, num_years)
            trailing_return_column[f"{num_years}-Year"] = trailing_return
        else:
            print(f"Data not available for {ticker}. Skipping.")
            trailing_return_column[f"{num_years}-Year"] = "N/A"
    return trailing_return_column

# Create an empty DataFrame to store all tickers' trailing returns
all_tickers_trailing_returns_df = pd.DataFrame(index=years_list)

# Loop through each ticker in the list
for ticker in tickers_list:
    trailing_returns = get_trailing_return_column(ticker)
    # Add the trailing returns to the DataFrame
    all_tickers_trailing_returns_df[ticker] = pd.Series(trailing_returns).values

# Display the trailing returns DataFrame
print("\nAnnualized Trailing Returns as end of year", current_year-1, ":")
print("years")
print(all_tickers_trailing_returns_df)

# print an indicator to mark the end of execution result
print(f"The result of version-{script_version} script  ends here.\n {result_marker}")
### CODE end (for AI tool)


The result of version-12 script  start:
 ###

Annual Total Return (%) History:
            xiu.to  xfn.to  ry.to  td.to   na.to  cwb.to  slf.to  gwo.to
1996-12-31     NaN     NaN  59.20  52.30   29.73   38.08     NaN     NaN
1997-12-31     NaN     NaN  61.23  56.87   75.97   62.90     NaN     NaN
1998-12-31     NaN     NaN   3.58   2.50    7.92   10.75     NaN     NaN
1999-12-31     NaN     NaN -14.82  46.56  -22.65  -21.16     NaN     NaN
2000-12-31    8.02     NaN  65.34  14.84   43.78   42.46     NaN     NaN
2001-12-31  -14.24     NaN   4.90  -2.81   11.65    9.86  -13.63     NaN
2002-12-31  -14.02   -3.06  14.85 -14.56    8.75   -5.13  -19.87     NaN
2003-12-31   24.82   26.14   9.13  31.37   35.63   50.03   23.78   25.64
2004-12-31   13.28   18.57   7.38  18.83   18.76   37.62   27.27   20.62
2005-12-31   25.47   22.90  45.87  26.19   25.57   36.52   19.14   18.24
2006-12-31   19.12   17.79  26.02  16.55   12.71   49.19    8.15   13.56
2007-12-31   10.83   -1.44  -5.57   2.66  -17

In [ ]:
"""
### CODE start (for AI tool)
Retrieve multiple tickers for total returns and trailing total returns from yahoo
History:
Dec 21, 2023, version 12

Author: Gang Luo
"""
result_marker='###'
script_version = 12
print(f"The result of version-{script_version} script  start:\n {result_marker}")
#!pip install yfinance
import yfinance as yf
import pandas as pd

# Set the stock tickers list
#tickers_list = ["spy", "vfv.to", "xiu.to", "xic.to", "xfn.to", "ry.to", "enb.to"]
tickers_list = ["xiu.to", "xfn.to", "ry.to", "td.to", "na.to", "cwb.to", "slf.to", "gwo.to"]
#-------------------------------------------------------------------
# step 1: fetch retrieve yearly total returns by yfinance & display

# Function to fetch data from yfinance and extract yearly total returns
def get_annual_returns_df(ticker):
    # Get the historical data for the given ticker
    stock = yf.Ticker(ticker)
    ''' Get annual total return data.   Explanation the following statement
    1.  'Close'  in .Ticker(ticker).history is the same as 'Adj Close' in .download(ticker, ...)
        print(df.columns.tolist()) - Display all column names of DataFrame
        yf.Ticker(ticker).history: ['Open', 'High', 'Low', 'Close', 'Volume', 'Dividends', 'Stock Splits']
        yf.download(ticker, ...): ['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume']
    2.  ["Close"]: Selects the "Close" column from the historical stock data.
        .resample('Y'): Resamples the time series data annually ('Y').
        .ffill(): Forward fills missing values, ensuring that each annual period has a value.
        .pct_change(): Calculates the percentage change between the current and previous values, representing the annual returns.
        .dropna(): Removes any rows with missing values (which may occur after calculating percentage changes).
    '''
    annual_returns = stock.history(period="max")["Close"].resample('Y').ffill().pct_change().dropna()
    annual_returns_df = pd.DataFrame(annual_returns, columns=['Close'])
    annual_returns_df.rename(columns={'Close': ticker}, inplace=True)
    return annual_returns_df
# Create an empty DataFrame to store all tickers' total returns
all_tickers_returns_df = pd.DataFrame()

# Loop through each ticker in the list
for ticker in tickers_list:
    ticker_returns_df = get_annual_returns_df(ticker)
    if not ticker_returns_df.empty:
        if all_tickers_returns_df.empty:
            all_tickers_returns_df = ticker_returns_df
        else:
            all_tickers_returns_df = pd.concat([all_tickers_returns_df, ticker_returns_df], axis=1)  # Concatenate DataFrames

# Display the results if the DataFrame is not empty
print("\nAnnual Total Return (%) History:")
if not all_tickers_returns_df.empty:
    # keep the date portion of index only - stripping time portion
    #all_tickers_returns_df.index=all_tickers_returns_df.index.date

    # Format the DataFrame to display percentage
    all_tickers_returns_df_copy = all_tickers_returns_df * 100
    all_tickers_returns_df_copy=all_tickers_returns_df_copy.round(2)
    all_tickers_returns_df_copy.index=all_tickers_returns_df_copy.index.date
    print(all_tickers_returns_df_copy)
else:
    print("No data available for the specified tickers.")

#-------------------------------------------------------------------
# step 2: Remove the row of the current year for the later steps
# Define a list of years to calculate the trailing returns, cumulative returns, and so on
years_list = [1, 2, 3, 5, 10, 15, 20, 25, 30]
current_year=all_tickers_returns_df.index.year.max()
all_tickers_returns_df['Year']=all_tickers_returns_df.index.year
all_tickers_returns_df=all_tickers_returns_df[all_tickers_returns_df['Year'] < current_year]


#-------------------------------------------------------------------
# step 3: calculate the annualized trailing total return from the data generated in step 1 & display
# Define a function to calculate the annualized trailing total return for a given number of years
def get_trailing_return(ticker, data, years):
    # Get the total return values for the last n years
    trailing_data = data[ticker].tail(years)

    # Check if there are valid total return values for all years
    if len(trailing_data) == years:
        # Convert the percentage strings to numeric values
        trailing_data = trailing_data.astype(str).str.replace('%', '').astype(float)

        """ Calculate the annualized trailing total return using the formula from Investopedia[^1^][1]:
            Annualized Return = [(1 + r1) * (1 + r2) * ... * (1 + rn)]^(1/n) - 1
            Where r1, r2, ..., rn are the total return values for each year                    """
        annualized_trailing_return = (trailing_data + 1).prod() ** (1 / years) - 1

        # Format the result as a percentage with two decimal places
        annualized_trailing_return = annualized_trailing_return * 100
        annualized_trailing_return = annualized_trailing_return.round(2)
        return annualized_trailing_return
    else:
        return "N/A"

# Create an empty DataFrame with years_list as the index for trailing total returns
all_tickers_trailing_returns_df = pd.DataFrame(index=years_list)

# Define a function to Loop through the list and print the trailing returns for each num_years
def get_trailing_return_column(ticker):
    trailing_return_column = {}
    for num_years in years_list:
        # Check if the ticker data is available in all_tickers_returns_df
        if ticker in all_tickers_returns_df.columns:
            # using data from step 1, avoiding get_annual_returns_df(ticker) for less traffic from yahoo server
            data = all_tickers_returns_df[[ticker]]
            trailing_return = get_trailing_return(ticker, data, num_years)
            trailing_return_column[f"{num_years}-Year"] = trailing_return
        else:
            print(f"Data not available for {ticker}. Skipping.")
            trailing_return_column[f"{num_years}-Year"] = "N/A"
    return trailing_return_column

# Create an empty DataFrame to store all tickers' trailing returns
all_tickers_trailing_returns_df = pd.DataFrame(index=years_list)

# Loop through each ticker in the list
for ticker in tickers_list:
    trailing_returns = get_trailing_return_column(ticker)
    # Add the trailing returns to the DataFrame
    all_tickers_trailing_returns_df[ticker] = pd.Series(trailing_returns).values

# Display the trailing returns DataFrame
print("\nAnnualized Trailing Returns as end of year", current_year-1, ":")
print("years")
print(all_tickers_trailing_returns_df)

# print an indicator to mark the end of execution result
print(f"The result of version-{script_version} script  ends here.\n {result_marker}")
### CODE end (for AI tool)


The result of version-12 script  start:
 ###

Annual Total Return (%) History:
            xiu.to  xfn.to  ry.to  td.to   na.to  cwb.to  slf.to  gwo.to
1996-12-31     NaN     NaN  59.20  52.30   29.73   38.08     NaN     NaN
1997-12-31     NaN     NaN  61.23  56.87   75.97   62.90     NaN     NaN
1998-12-31     NaN     NaN   3.58   2.50    7.92   10.75     NaN     NaN
1999-12-31     NaN     NaN -14.82  46.56  -22.65  -21.16     NaN     NaN
2000-12-31    8.02     NaN  65.34  14.84   43.78   42.47     NaN     NaN
2001-12-31  -14.24     NaN   4.90  -2.81   11.65    9.86  -13.63     NaN
2002-12-31  -14.02   -3.06  14.85 -14.56    8.75   -5.13  -19.87     NaN
2003-12-31   24.82   26.14   9.13  31.37   35.63   50.03   23.78   25.64
2004-12-31   13.28   18.57   7.38  18.83   18.76   37.62   27.27   20.62
2005-12-31   25.47   22.90  45.87  26.19   25.57   36.52   19.14   18.24
2006-12-31   19.12   17.79  26.02  16.55   12.71   49.19    8.15   13.56
2007-12-31   10.83   -1.44  -5.57   2.66  -17

# Example 2 EndOfYear- AdjPrice, dividendSum, the yearly total return

In [ ]:
import yfinance as yf

def get_yearly_data(ticker):
    # Get the historical data for the given ticker
    stock = yf.Ticker(ticker)

    history = stock.history(period="max").reset_index()
    # Extract the year from the Date column
    history['Year'] = history['Date'].dt.year

    # Group the data by year and get the last trading date, Adj Close, and sum of dividends for each year
    yearly_data = history.groupby('Year').agg({'Date': 'last', 'Close': 'last', 'Dividends': 'sum'})
    yearly_data.rename(columns={'Date': 'LastTradingDate', 'Close': 'Adj Close', 'Dividends': 'DividendSum'}, inplace=True)

    # Calculate the total return for each year
    yearly_data['TotalReturn'] = yearly_data['Adj Close'] / yearly_data['Adj Close'].shift(1) - 1
    yearly_data['TotalReturn'] = yearly_data['TotalReturn'] * 100
    yearly_data['TotalReturn'] = yearly_data['TotalReturn'].map("{:.2f}%".format)

    # Format the "Last Trading Date" column to display only the date portion
    yearly_data['LastTradingDate'] = yearly_data['LastTradingDate'].dt.date
    return yearly_data

# Main program
ticker = 'BNS.TO'
data = get_yearly_data(ticker)

print(data)

     LastTradingDate  Adj Close  DividendSum TotalReturn
Year                                                    
1995      1995-12-29   0.375843       0.3100        nan%
1996      1996-12-31   0.649846       0.2475      72.90%
1997      1997-12-31   1.057006       0.3700      62.65%
1998      1998-12-31   1.172340       0.5050      10.91%
1999      1999-12-31   1.176476       0.4500       0.35%
2000      2000-12-29   1.810161       0.5200      53.86%
2001      2001-12-31   2.272631       0.6500      25.55%
2002      2002-12-31   2.648307       0.5550      16.53%
2003      2003-12-31   3.658473       0.8400      38.14%
2004      2004-12-31   4.948505       1.4200      35.26%
2005      2005-12-30   6.224385       1.3600      25.78%
2006      2006-12-29   7.792041       1.5600      25.19%
2007      2007-12-31   8.365834       1.7900       7.36%
2008      2008-12-31   6.086138       1.4500     -27.25%
2009      2009-12-31  10.791874       2.4500      77.32%
2010      2010-12-31  13.926376

In [ ]:
import yfinance as yf

def get_yearly_data(ticker):
    # Get the historical data for the given ticker
    stock = yf.Ticker(ticker)

    history = stock.history(period="max").reset_index()
    # Extract the year from the Date column
    history['Year'] = history['Date'].dt.year

    # Group the data by year and get the last trading date, Adj Close, and sum of dividends for each year
    yearly_data = history.groupby('Year').agg({'Date': 'last', 'Close': 'last', 'Dividends': 'sum'})
    yearly_data.rename(columns={'Date': 'LastTradingDate', 'Close': 'Adj Close', 'Dividends': 'DividendSum'}, inplace=True)

    # Calculate the total return for each year
    yearly_data['TotalReturn'] = yearly_data['Adj Close'] / yearly_data['Adj Close'].shift(1) - 1
    yearly_data['TotalReturn'] = yearly_data['TotalReturn'] * 100
    yearly_data['TotalReturn'] = yearly_data['TotalReturn'].map("{:.2f}%".format)

    # Format the "Last Trading Date" column to display only the date portion
    yearly_data['LastTradingDate'] = yearly_data['LastTradingDate'].dt.date
    return yearly_data

# Main program
ticker = 'XFN.TO'
data = get_yearly_data(ticker)

print(data)



     LastTradingDate  Adj Close  DividendSum TotalReturn
Year                                                    
2001      2001-12-31   7.318615       0.1875        nan%
2002      2002-12-31   7.094312       0.2825      -3.06%
2003      2003-12-31   8.948480       0.3455      26.14%
2004      2004-12-31  10.610480       0.3805      18.57%
2005      2005-12-30  13.040280       0.4635      22.90%
2006      2006-12-29  15.360328       0.5420      17.79%
2007      2007-12-31  15.138770       0.6590      -1.44%
2008      2008-12-31   9.691955       0.8115     -35.98%
2009      2009-12-31  14.012818       0.8150      44.58%
2010      2010-12-31  15.120352       0.7440       7.90%
2011      2011-12-30  14.453518       0.7970      -4.41%
2012      2012-12-31  16.829203       0.8220      16.44%
2013      2013-12-31  21.204639       0.8320      26.00%
2014      2014-12-31  23.722937       0.8830      11.88%
2015      2015-12-31  22.843807       0.9390      -3.71%
2016      2016-12-30  28.184969

## backup Example 2 EndOfYear- AdjPrice, dividendSum, the yearly total return

In [ ]:
import yfinance as yf

def get_yearly_data(ticker):
    # Get the historical data for the given ticker
    stock = yf.Ticker(ticker)

    history = stock.history(period="max").reset_index()
    # Extract the year from the Date column
    history['Year'] = history['Date'].dt.year

    # Group the data by year and get the last trading date, Adj Close, and sum of dividends for each year
    yearly_data = history.groupby('Year').agg({'Date': 'last', 'Close': 'last', 'Dividends': 'sum'})
    yearly_data.rename(columns={'Date': 'LastTradingDate', 'Close': 'Adj Close', 'Dividends': 'DividendSum'}, inplace=True)

    # Calculate the total return for each year
    yearly_data['TotalReturn'] = yearly_data['Adj Close'] / yearly_data['Adj Close'].shift(1) - 1
    yearly_data['TotalReturn'] = yearly_data['TotalReturn'] * 100
    yearly_data['TotalReturn'] = yearly_data['TotalReturn'].map("{:.2f}%".format)

    # Format the "Last Trading Date" column to display only the date portion
    yearly_data['LastTradingDate'] = yearly_data['LastTradingDate'].dt.date
    return yearly_data

# Main program
ticker = 'XFN.TO'
data = get_yearly_data(ticker)

print(data)

     LastTradingDate  Adj Close  DividendSum TotalReturn
Year                                                    
2001      2001-12-31   7.318619       0.1875        nan%
2002      2002-12-31   7.094314       0.2825      -3.06%
2003      2003-12-31   8.948478       0.3455      26.14%
2004      2004-12-31  10.610483       0.3805      18.57%
2005      2005-12-30  13.040282       0.4635      22.90%
2006      2006-12-29  15.360314       0.5420      17.79%
2007      2007-12-31  15.138775       0.6590      -1.44%
2008      2008-12-31   9.691957       0.8115     -35.98%
2009      2009-12-31  14.012822       0.8150      44.58%
2010      2010-12-31  15.120358       0.7440       7.90%
2011      2011-12-30  14.453508       0.7970      -4.41%
2012      2012-12-31  16.829201       0.8220      16.44%
2013      2013-12-31  21.204636       0.8320      26.00%
2014      2014-12-31  23.722931       0.8830      11.88%
2015      2015-12-31  22.843813       0.9390      -3.71%
2016      2016-12-30  28.184965

# Example 1   Close, Adj Close, dividendSum, yearly total return, yearly total return

In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
script_version = '(2024-01-10.1)'

def get_yearly_data(ticker):
    stock = yf.Ticker(ticker)
    #-------- mainly for downloading 'Dividends'
    history = stock.history(period="max")
    dividend_history=history['Dividends']
    dividend_history.index=dividend_history.index.date

    #-------- mainly for downloading 'Close','Adj Close'
    dld_history=yf.download(ticker, period="max")
    dld_history=dld_history[['Close','Adj Close']]
    date_range = pd.date_range(start=dld_history.index.min(), end=dld_history.index.max(), freq='D')
    complete_history = pd.DataFrame(index=date_range)

    # Merge the complete DataFrame with the original stock_history
    complete_history = complete_history.merge(dld_history, how='left', left_index=True, right_index=True)
    complete_history[['Close','Adj Close']] = complete_history[['Close','Adj Close']].ffill()

    # Merge dividend into complete_history
    complete_history = complete_history.merge(dividend_history, how='left', left_index=True, right_index=True)
    # replace all NaN values in the 'Dividends' column with 0.0
    complete_history['Dividends'] = complete_history['Dividends'].fillna(0.0)

    complete_history['Year']=complete_history.index.year
    complete_history['Date']=complete_history.index
    yearly_data = complete_history.groupby('Year').agg({'Date': 'last', 'Close': 'last', 'Adj Close': 'last','Dividends': 'sum'})
    yearly_data.rename(columns={'Dividends': 'DivSum'}, inplace=True)

    # calculating 'Return' and 'TotalReturn'
    yearly_data['DivRatio']=yearly_data['DivSum'] / yearly_data['Close']
    yearly_data['Return']=yearly_data['Close'].pct_change()
    yearly_data['TotalReturn']=yearly_data['Adj Close'].pct_change()

    '''
    The CalReturn column is the yearly total return calculated from un-adjusted "Close" prices and yearly "dividend sum",
    which is expected to be equal to the total return that is calculated from "Adj Close" prices
    '''
    yearly_data['CalReturn'] = (yearly_data['Close'] + yearly_data['DivSum']) / yearly_data['Close'].shift(1) - 1
    # set the display format
    yearly_data[['DivRatio','Return','TotalReturn','CalReturn']] = yearly_data[['DivRatio','Return','TotalReturn','CalReturn']].mul(100).round(2)
    yearly_data[['DivRatio','Return', 'TotalReturn', 'CalReturn']] = yearly_data[['DivRatio','Return', 'TotalReturn', 'CalReturn']].applymap("{:.2f}%".format)
    # 'Date' column is no longer required
    yearly_data.drop('Date', axis=1, inplace=True)
    return yearly_data

# Main program
ticker = 'spy'
data = get_yearly_data(ticker)
print(data)



[*********************100%%**********************]  1 of 1 completed
           Close   Adj Close  DivSum DivRatio   Return TotalReturn CalReturn
Year                                                                        
1993   46.593750   27.004082   1.134    2.43%     nan%        nan%      nan%
1994   45.562500   27.111387   1.227    2.69%   -2.21%       0.40%     0.42%
1995   61.484375   37.426991   1.278    2.08%   34.95%      38.05%    37.75%
1996   73.843750   45.847099   1.355    1.83%   20.10%      22.50%    22.31%
1997   97.062500   61.194511   1.377    1.42%   31.44%      33.48%    33.31%
1998  123.312500   78.752670   1.416    1.15%   27.04%      28.69%    28.50%
1999  146.875000   94.809868   1.445    0.98%   19.11%      20.39%    20.28%
2000  131.187500   85.573944   1.505    1.15%  -10.68%      -9.74%    -9.66%
2001  114.300003   75.511703   1.424    1.25%  -12.87%     -11.76%   -11.79%
2002   88.230003   59.212761   1.498    1.70%  -22.81%     -21.58%   -21.50%
2003  1

## Backup Example 1

In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
script_version = '(2024-01-10.1)'

def get_yearly_data(ticker):
    stock = yf.Ticker(ticker)
    #-------- mainly for downloading 'Dividends'
    history = stock.history(period="max")
    dividend_history=history['Dividends']
    dividend_history.index=dividend_history.index.date

    #-------- mainly for downloading 'Close','Adj Close'
    dld_history=yf.download(ticker, period="max")
    dld_history=dld_history[['Close','Adj Close']]
    date_range = pd.date_range(start=dld_history.index.min(), end=dld_history.index.max(), freq='D')
    complete_history = pd.DataFrame(index=date_range)

    # Merge the complete DataFrame with the original stock_history
    complete_history = complete_history.merge(dld_history, how='left', left_index=True, right_index=True)
    complete_history[['Close','Adj Close']] = complete_history[['Close','Adj Close']].ffill()

    # Merge dividend into complete_history
    complete_history = complete_history.merge(dividend_history, how='left', left_index=True, right_index=True)
    # replace all NaN values in the 'Dividends' column with 0.0
    complete_history['Dividends'] = complete_history['Dividends'].fillna(0.0)

    complete_history['Year']=complete_history.index.year
    complete_history['Date']=complete_history.index
    yearly_data = complete_history.groupby('Year').agg({'Date': 'last', 'Close': 'last', 'Adj Close': 'last','Dividends': 'sum'})
    yearly_data.rename(columns={'Dividends': 'DivSum'}, inplace=True)

    # calculating 'Return' and 'TotalReturn'
    yearly_data['DivRatio']=yearly_data['DivSum'] / yearly_data['Close']
    yearly_data['Return']=yearly_data['Close'].pct_change()
    yearly_data['TotalReturn']=yearly_data['Adj Close'].pct_change()

    '''
    The CalReturn column is the yearly total return calculated from un-adjusted "Close" prices and yearly "dividend sum",
    which is expected to be equal to the total return that is calculated from "Adj Close" prices
    '''
    yearly_data['CalReturn'] = (yearly_data['Close'] + yearly_data['DivSum']) / yearly_data['Close'].shift(1) - 1
    # set the display format
    yearly_data[['DivRatio','Return','TotalReturn','CalReturn']] = yearly_data[['DivRatio','Return','TotalReturn','CalReturn']].mul(100).round(2)
    yearly_data[['DivRatio','Return', 'TotalReturn', 'CalReturn']] = yearly_data[['DivRatio','Return', 'TotalReturn', 'CalReturn']].applymap("{:.2f}%".format)
    # 'Date' column is no longer required
    yearly_data.drop('Date', axis=1, inplace=True)
    print(yearly_data.index)
    return yearly_data

# Main program
ticker = 'BNS.TO'
data = get_yearly_data(ticker)
print(data)



[*********************100%%**********************]  1 of 1 completed
Int64Index([1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005,
            2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016,
            2017, 2018, 2019, 2020, 2021, 2022, 2023, 2024],
           dtype='int64', name='Year')
          Close  Adj Close  DivSum DivRatio   Return TotalReturn CalReturn
Year                                                                      
1995   2.521655   0.375843  0.3100   12.29%     nan%        nan%      nan%
1996   4.001173   0.649847  0.2475    6.19%   58.67%      72.90%    68.49%
1997   6.034898   1.057006  0.3700    6.13%   50.83%      62.65%    60.08%
1998   6.240640   1.172339  0.5050    8.09%    3.41%      10.91%    11.78%
1999   5.906535   1.176476  0.4500    7.62%   -5.35%       0.35%     1.86%
2000   8.449734   1.810161  0.5200    6.15%   43.06%      53.86%    51.86%
2001   9.857067   2.272631  0.6500    6.59%   16.66%      25.55%    24.35%


# Example 3: retrieve  current last stock price from yahoo finance


In [ ]:
import yfinance as yf
import pandas as pd
from datetime import datetime

def get_current_prices(tickers_list):
    # Create an empty dictionary to store data
    data = {}
    # Get current prices for each ticker
    for ticker in tickers_list:
        # Create a ticker object
        stock = yf.Ticker(ticker)
        # Get the current market last price
        last_price = stock.basic_info['last_price']
        # Store the price in the dictionary
        data[ticker] = [last_price]
    # Create a DataFrame from the dictionary
    df = pd.DataFrame(data)
    return df

# Define the list of tickers
tickers_list = ["spy", "vfv.to", "xiu.to", "xic.to", "xfn.to", "ry.to", "enb.to"]
# Call the function to get the DataFrame
current_prices_df = get_current_prices(tickers_list)

current_datetime = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
print(f"Market last price from Yahoo Finance (Current time: {current_datetime})")
# Print the returned DataFrame with only 2 digit after dot, without index
print(current_prices_df.round(2).to_string(index=False))



Market last price from Yahoo Finance (Current time: 2023-12-21 02:41:31)
   spy  vfv.to  xiu.to  xic.to  xfn.to  ry.to  enb.to
468.26  111.66   31.42   32.93   47.15 132.33   47.28


# a simple example

In [ ]:
#!pip install yfinance
import yfinance as yf
ticker = "XFN.TO"
# Create a ticker object
stock = yf.Ticker(ticker)
# Get the current market price
price = stock.basic_info['last_price']
print(f"The last_price price of {ticker} is ${price:.4f}")


The last_price price of XFN.TO is $48.3000


# trailing total return from hard-coded data input


Certainly! To convert the provided data into a 2-dimensional array in Python, you can use a list of lists. Here's how you can represent the data:

python
Copy code
data_array = [
    ["Year", "Total Return %"],
    [2013, 26.00],
    [2014, 11.89],
    [2015, -3.71],
    [2016, 23.36],
    [2017, 12.54],
    [2018, -9.76],
    [2019, 20.66],
    [2020, 0.98],
    [2021, 35.56],
    [2022, -9.93],
    ["YTD", None]  # Assuming YTD is a placeholder, set to None or provide the actual value
]

In [ ]:
import yfinance as yf

ticker = 'XFN.TO'
# data from Ishare web site
# xfn_yearly_total_return as input
xfn_yearly_total_return = [
    ['Year', 'Total Return'],
    [2013, 26.0],
    [2014, 11.89],
    [2015, -3.71],
    [2016, 23.36],
    [2017, 12.54],
    [2018, -9.76],
    [2019, 20.66],
    [2020, 0.98],
    [2021, 35.56],
    [2022, -9.93],
]
# expect output according Ishare web
xfn_trailing_return_At_2022_END = [
    ["years", "1y", "3y", "5y", "10y"],
    ["Total Return %", -9.90, 7.24, 6.07, 9.73]
]

# Define a function to calculate the annualized trailing total return for a given number of years
def get_trailing_return(data, years):
    # Get the total return values for the last n years
    trailing_data = data['Total Return'].tail(years)

    # Convert the percentage strings to numeric values
    #trailing_data = trailing_data.str.replace('%', '').astype(float)
    trailing_data = trailing_data.astype(str).str.replace('%', '').astype(float)

    # Calculate the annualized trailing total return using the formula from Investopedia[^1^][1]:
    # Annualized Return = [(1 + r1) * (1 + r2) * ... * (1 + rn)]^(1/n) - 1
    # Where r1, r2, ..., rn are the total return values for each year
    annualized_return = (trailing_data / 100 + 1).prod() ** (1 / years) - 1

    # Format the result as a percentage with two decimal places
    annualized_return = "{:.2f}%".format(annualized_return * 100)
    return annualized_return

TRAIL_RETURN_AS_YEAR=2022

# Define a list of years to calculate the trailing returns
years_list = [1, 3, 5, 10]

# please add a code section to convert variable 'xfn_yearly_total_return'
# to variable 'data' for get_trailing_return(data, years)
# Convert the list of lists to a pandas dataframe
import pandas as pd
data = pd.DataFrame(xfn_yearly_total_return[1:], columns=xfn_yearly_total_return[0])
# Set the index to the year column
data.set_index('Year', inplace=True)
#print(data.transpose())
print(data.T)

# Loop through the list and print the trailing returns for each year
for years in years_list:
    trailing_return = get_trailing_return(data, years)
    print(f"{years}-Year {trailing_return}")

Year          2013   2014  2015   2016   2017  2018   2019  2020   2021  2022
Total Return  26.0  11.89 -3.71  23.36  12.54 -9.76  20.66  0.98  35.56 -9.93
1-Year -9.93%
3-Year 7.23%
5-Year 6.07%
10-Year 9.73%


# Example 5: non-calender year/single stock - calculate  the trailing total returns (used in my article in google site)

In [ ]:

# Import libraries
import yfinance as yf
import pandas as pd

# Define the stock ticker and the time periods
ticker = "XFN.to"
stock = yf.Ticker(ticker)
# Define the stock ticker and the time periods
periods = [1, 3, 5, 10, 15, 20, 25, 30] # in years

calculation_start_day="1990-01-01"
calculation_end_day="2023-12-15"
#calculation_end_day="2022-12-31"

#--------------------------------------------------------------------------------------------
# Step 1: Fetch the historical data, Calculate annual returns based on the any last trading day of each year
# Check if the variable 'last_trading_day' is defined in the local scope
# "Close" price in history is actually Adj Close in download, taking account of dividends and splits
# Get the historical data for the given ticker
stock = yf.Ticker(ticker)
''' Get annual total return data.
   .resample('Y'): Resamples the time series data annually ('Y'), based on the last trading day of each year.
   .resample('A'): Resamples the time series data annually ('A'), based on the any trading day of each year.
'''
annual_returns = stock.history(period="max", start=calculation_start_day,
  end=calculation_end_day)["Close"].resample('A').ffill().pct_change().dropna()
annual_returns_df = pd.DataFrame(annual_returns, columns=['Close'])
annual_returns_df.rename(columns={'Close': ticker}, inplace=True)


#--------------------------------------------------------------------------------------------
# Step 2: Calculate the annualized trailing return for each period
# Create a DataFrame to store the annualized trailing return information
trailing_return_data = []
for period in periods:
    data=annual_returns_df[ticker].tail(period)
    if (len(data) != period):
       trailing_return_data.append("NaN")
       continue;
    # Calculate the annualized trailing return
    """ Calculate the annualized trailing total return using the formula from Investopedia[^1^][1]:
        Annualized Return = [(1 + r1) * (1 + r2) * ... * (1 + rn)]^(1/n) - 1
        Where r1, r2, ..., rn are the total return values for each year                    """
    atr = (1 + data).prod() ** (1 / period) - 1
    trailing_return_data.append(atr)

# Create an  DataFrame with years_list as the index for trailing total returns
trailing_return_column = pd.DataFrame(trailing_return_data, index=periods, columns=[ticker])
# Print or display the DataFrame
print(f"The annualized trailing return for {ticker} \n as", calculation_end_day)
trailing_return_column_percent = trailing_return_column.astype(float) * 100
trailing_return_column_formatted = trailing_return_column_percent.applymap(lambda x: f"{x:.2f}%" if not pd.isna(x) else "N/A")
trailing_return_column_formatted.index.name="years"
print(trailing_return_column_formatted)


The annualized trailing return for XFN.to 
 as 2023-12-15
       XFN.to
years        
1      10.65%
3      10.56%
5      10.49%
10      8.31%
15     11.12%
20      8.66%
25        N/A
30        N/A


## backup Example 5

In [ ]:

# Import libraries
import yfinance as yf
import pandas as pd

# Define the stock ticker and the time periods
ticker = "XFN.to"
stock = yf.Ticker(ticker)
# Define the stock ticker and the time periods
periods = [1, 3, 5, 10, 15, 20, 25, 30] # in years

calculation_start_day="1990-01-01"
calculation_end_day="2023-12-15"
#calculation_end_day="2022-12-31"

#--------------------------------------------------------------------------------------------
# Step 1: Fetch the historical data, Calculate annual returns based on the any last trading day of each year
# Check if the variable 'last_trading_day' is defined in the local scope
# "Close" price in history is actually Adj Close in download, taking account of dividends and splits
# Get the historical data for the given ticker
stock = yf.Ticker(ticker)
''' Get annual total return data.
   .resample('Y'): Resamples the time series data annually ('Y'), based on the last trading day of each year.
   .resample('A'): Resamples the time series data annually ('A'), based on the any trading day of each year.
'''
annual_returns = stock.history(period="max", start=calculation_start_day,
  end=calculation_end_day)["Close"].resample('A').ffill().pct_change().dropna()
annual_returns_df = pd.DataFrame(annual_returns, columns=['Close'])
annual_returns_df.rename(columns={'Close': ticker}, inplace=True)


#--------------------------------------------------------------------------------------------
# Step 2: Calculate the annualized trailing return for each period
# Create a DataFrame to store the annualized trailing return information
trailing_return_data = []
for period in periods:
    data=annual_returns_df[ticker].tail(period)
    if (len(data) != period):
       trailing_return_data.append("NaN")
       continue;
    # Calculate the annualized trailing return
    """ Calculate the annualized trailing total return using the formula from Investopedia[^1^][1]:
        Annualized Return = [(1 + r1) * (1 + r2) * ... * (1 + rn)]^(1/n) - 1
        Where r1, r2, ..., rn are the total return values for each year                    """
    atr = (1 + data).prod() ** (1 / period) - 1
    trailing_return_data.append(atr)

# Create an  DataFrame with years_list as the index for trailing total returns
trailing_return_column = pd.DataFrame(trailing_return_data, index=periods, columns=[ticker])
# Print or display the DataFrame
print(f"The annualized trailing return for {ticker} \n as", calculation_end_day)
trailing_return_column_percent = trailing_return_column.astype(float) * 100
trailing_return_column_formatted = trailing_return_column_percent.applymap(lambda x: f"{x:.2f}%" if not pd.isna(x) else "N/A")
print(trailing_return_column_formatted)



The annualized trailing return for XFN.to 
 as 2023-12-15
    XFN.to
1   10.65%
3   10.56%
5   10.49%
10   8.31%
15  11.12%
20   8.66%
25     N/A
30     N/A


# ! Example 6 non-calender year (multiple stocks)  the trailing total returns

In [ ]:
"""
### CODE start (for AI tool)
"""
result_marker='###'
script_version = '(2023.1224.1)'
print(f"The result of version-{script_version} script  start:\n {result_marker}")
# Import libraries
import yfinance as yf
import pandas as pd
from datetime import datetime
#!pip install pytz
import pytz
print_yearly_total_return = True

# Define the stock ticker and the time periods
periods = [1, 3, 5, 10, 15, 20, 25, 30] # in years
tickers_list1=["spy", "vfv.to","vgg.to", "zlu.to", "xiu.to", "xfn.to", "ry.to", "td.to", "na.to","bmo.to",
"bns.to", "slf.to", "gwo.to", "bce.to", "t.to", "rci-b.to", "enb.to", "trp.to", "zlb.to"]
tickers_list2=["xfn.to", "xiu.to", "ry.to", "vfv.to", "spy"]
tickers_list3=["spy", "vfv.to", "goog", "msft", "meta", "tsla", "shop.to", "amzn", "abnb", "ibm"]
tickers_list=tickers_list1

calculation_start_day="1993-01-01"
#calculation_end_day="2022-12-31"
# Get today's date, use .strftime("%Y-%m-%d") to convert to a string
calculation_end_day=datetime.now(pytz.timezone('America/New_York')).date().strftime("%Y-%m-%d") # Get today's date
#-------------------------------------------------------------------
# step 1: fetch retrieve yearly total returns by yfinance & display
# Function to fetch data from yfinance and extract yearly total returns
def get_annual_returns_df(ticker):
    # Get the historical data for the given ticker
    stock = yf.Ticker(ticker)
    ''' Get annual total return data.
        .resample('Y'): Resamples the time series data annually ('Y'), based on the last trading day of each year.
        .resample('A'): Resamples the time series data annually ('A'), based on the any trading day of each year.
    '''
    annual_returns = stock.history(period="max", start=calculation_start_day,
      end=calculation_end_day)["Close"].resample('A').ffill().pct_change().dropna()
    annual_returns_df = pd.DataFrame(annual_returns, columns=['Close'])
    annual_returns_df.rename(columns={'Close': ticker}, inplace=True)
    return annual_returns_df
# Create an empty DataFrame to store all tickers' total returns
all_tickers_returns_df = pd.DataFrame()

# Loop through each ticker in the list
for ticker in tickers_list:
    ticker_returns_df = get_annual_returns_df(ticker)
    if not ticker_returns_df.empty:
        if all_tickers_returns_df.empty:
            all_tickers_returns_df = ticker_returns_df
        else:
            # Concatenate DataFrames
            all_tickers_returns_df = pd.concat([all_tickers_returns_df, ticker_returns_df], axis=1)

#-------------------------------------------------------------------
# step 2:
# Define a list of years to calculate the trailing returns, cumulative returns, and so on
# remove the row of current year row since it is not a full year.
years_list = [1, 2, 3, 5, 10, 15, 20, 25, 30]


#-------------------------------------------------------------------
# step 3: calculate the annualized trailing total return from the data generated in step 1 & display
# Define a function to calculate the annualized trailing total return for a given number of years
def get_trailing_return(ticker, data, years):
    # Get the total return values for the last n years
    trailing_data = data[ticker].tail(years)
    # Check if there are empty values within years
    if trailing_data.isna().any():
        return "N/A"
    # Check if there are valid total return values for all years
    if len(trailing_data) == years:
        # Convert the percentage strings to numeric values
        trailing_data = trailing_data.astype(str).str.replace('%', '').astype(float)
        """ Calculate the annualized trailing total return using the formula from Investopedia[^1^][1]:
            Annualized Return = [(1 + r1) * (1 + r2) * ... * (1 + rn)]^(1/n) - 1
            Where r1, r2, ..., rn are the total return values for each year                    """
        annualized_trailing_return = (trailing_data + 1).prod() ** (1 / years) - 1

        # Format the result as a percentage with two decimal places
        annualized_trailing_return = annualized_trailing_return * 100
        annualized_trailing_return = annualized_trailing_return.round(2)
        return annualized_trailing_return
    else:
        return "N/A"

# Define a function to Loop through the list and print the trailing returns for each num_years
def get_trailing_return_column(ticker):
    trailing_return_column = {}
    for num_years in years_list:
        # Check if the ticker data is available in all_tickers_returns_df
        if ticker in all_tickers_returns_df.columns:
            # using data from step 1, avoiding get_annual_returns_df(ticker) for less traffic from yahoo server
            data = all_tickers_returns_df[[ticker]]
            trailing_return = get_trailing_return(ticker, data, num_years)
            trailing_return_column[f"{num_years}-Year"] = trailing_return
        else:
            print(f"Data not available for {ticker}. Skipping.")
            trailing_return_column[f"{num_years}-Year"] = "N/A"
    return trailing_return_column

# Create an empty DataFrame to store all tickers' trailing returns
all_tickers_trailing_returns_df = pd.DataFrame(index=years_list)

# Loop through each ticker in the list
for ticker in tickers_list:
    trailing_returns = get_trailing_return_column(ticker)
    # Add the trailing returns to the DataFrame
    all_tickers_trailing_returns_df[ticker] = pd.Series(trailing_returns).values

#-------------------------------------------------------------------
# step 4: calculate the cumulative return from the data (all_tickers_returns_df) generated in step 1 & display
# 4.1 Define a function to calculate the cumulative return for a given number of years from a ticker
def get_cumulative_return(ticker, data, years):
    # Calculate the cumulative return
    cumulative_return = (1 + data[ticker]).rolling(window=years).apply(lambda x: x.prod(), raw=True) - 1
    return cumulative_return

# Create an empty DataFrame with years_list as the index for cumulative  returns
all_tickers_cumulative_returns_df = pd.DataFrame(index=years_list)

# Define a function to Loop through the list and return the cumulative returns for each num_years
def get_cumulative_return_column(ticker):
    cumulative_returns = {}
    for years in years_list:
        # Calculate the cumulative return for the given number of years
        cumulative_return = get_cumulative_return(ticker, all_tickers_returns_df, years)
        # Get the last value, which is the cumulative return up to the current year
        cumulative_returns[years] = cumulative_return.iloc[-1]
    return cumulative_returns

# Loop through each ticker in the list
for ticker in tickers_list:
    cumulative_returns = get_cumulative_return_column(ticker)
    # Add the trailing returns to the DataFrame
    all_tickers_cumulative_returns_df[ticker] = pd.Series(cumulative_returns).values

#-------------------------------------------------------------------
# step 5: calculate the  CAGR from the data (all_tickers_cumulative_returns_df) generated earlier & display
# Define a function to calculate the CAGR from the cumulative value and the years
import numpy as np
def calculate_cagr(value, years):
    # Otherwise, calculate the CAGR using the formula
    cagr = (value + 1) ** (1 / np.array(years)) - 1
    #print("debug-cagr\n", cagr, "end")
    return cagr

# Define a function to format the Float64Index values into percentage strings
def format_to_percentage(value):
    # If any element in the value array is not null, format it as a percentage string with two decimal places
    if np.any(pd.notnull(value)):
        return f"{value:.2f}%"
    # Otherwise, return None
    return None

# Apply the calculate_cagr function to each column of the DataFrame
all_tickers_cagrs_df = all_tickers_cumulative_returns_df.apply(lambda x: calculate_cagr(x, x.index), axis=0)

#----------------------------------------------------
# Display the formatted DataFrame  all_tickers_returns_df
if (print_yearly_total_return):
    all_tickers_returns_df.index=all_tickers_returns_df.index.date
    all_tickers_returns_df.index.name='date'
    # Convert calculation_end_day to a datetime object, replace the index's mon/day portion of date
    calculation_end_day_datetime_obj = datetime.strptime(calculation_end_day, "%Y-%m-%d")
    all_tickers_returns_df.index = all_tickers_returns_df.index.map(\
      lambda x: x.replace(month=calculation_end_day_datetime_obj.month,\
      day=calculation_end_day_datetime_obj.day))
    print("\nAnnual Total Return (%) as ", calculation_end_day)
    print(all_tickers_returns_df.round(4) * 100)

# Display the trailing returns DataFrame
print("\nAnnualized Trailing Returns as ", calculation_end_day, ":")
all_tickers_trailing_returns_df.index.name = 'Years'
print(all_tickers_trailing_returns_df)

# Display the cumulative returns DataFrame
print("\nCumulative Returns as end of year", calculation_end_day, ":")
all_tickers_cumulative_returns_df.index.name = 'Years'
print(all_tickers_cumulative_returns_df.round(4) * 100)

# Display the formatted DataFrame  all_tickers_cagrs_df
print("\nCompound Annual Growth Rates (GAGRs) (%) as end of year", calculation_end_day, ":")
all_tickers_cagrs_df.index.name = 'Years'
print(all_tickers_cagrs_df.round(4) * 100)

print(f"The result of version-{script_version} script  ends here.\n {result_marker}")
### CODE end (for AI tool)

The result of version-(2023.1224.1) script  start:
 ###

Annual Total Return (%) as  2023-12-28
              spy  vfv.to  vgg.to  zlu.to  xiu.to  xfn.to  ry.to  td.to  \
date                                                                      
1994-12-28   0.40     NaN     NaN     NaN     NaN     NaN    NaN    NaN   
1995-12-28  38.05     NaN     NaN     NaN     NaN     NaN    NaN    NaN   
1996-12-28  22.50     NaN     NaN     NaN     NaN     NaN  59.20  52.30   
1997-12-28  33.48     NaN     NaN     NaN     NaN     NaN  61.23  56.87   
1998-12-28  28.69     NaN     NaN     NaN     NaN     NaN   3.58   2.50   
1999-12-28  20.39     NaN     NaN     NaN     NaN     NaN -14.82  46.56   
2000-12-28  -9.74     NaN     NaN     NaN    8.02     NaN  65.34  14.84   
2001-12-28 -11.76     NaN     NaN     NaN  -14.24     NaN   4.90  -2.81   
2002-12-28 -21.58     NaN     NaN     NaN  -14.02   -3.06  14.85 -14.56   
2003-12-28  28.18     NaN     NaN     NaN   24.82   26.14   9.13  31.37   
2004

## Backup Example 6

In [ ]:
"""
### CODE start (for AI tool)
"""
result_marker='###'
script_version = '(2023.1224.1)'
print(f"The result of version-{script_version} script  start:\n {result_marker}")
# Import libraries
import yfinance as yf
import pandas as pd
from datetime import datetime
print_yearly_total_return = True

# Define the stock ticker and the time periods
periods = [1, 3, 5, 10, 15, 20, 25, 30] # in years
tickers_list1=["spy", "vfv.to","vgg.to", "zlu.to", "xiu.to", "xfn.to", "ry.to", "td.to", "na.to",
    "slf.to", "gwo.to", "bce.to", "t.to", "rci-b.to", "enb.to", "trp.to", "zlb.to"]
tickers_list2=["xfn.to", "xiu.to", "ry.to", "vfv.to", "spy"]
tickers_list3=["spy", "vfv.to", "goog", "msft", "meta", "tsla", "shop.to", "amzn", "abnb", "ibm"]
tickers_list=tickers_list3

calculation_start_day="1993-01-01"
#calculation_end_day="2022-12-31"
# Get today's date, use .strftime("%Y-%m-%d") to convert to a string
calculation_end_day=datetime.now().date().strftime("%Y-%m-%d") # Get today's date
#-------------------------------------------------------------------
# step 1: fetch retrieve yearly total returns by yfinance & display
# Function to fetch data from yfinance and extract yearly total returns
def get_annual_returns_df(ticker):
    # Get the historical data for the given ticker
    stock = yf.Ticker(ticker)
    ''' Get annual total return data.
        .resample('Y'): Resamples the time series data annually ('Y'), based on the last trading day of each year.
        .resample('A'): Resamples the time series data annually ('A'), based on the any trading day of each year.
    '''
    annual_returns = stock.history(period="max", start=calculation_start_day,
      end=calculation_end_day)["Close"].resample('A').ffill().pct_change().dropna()
    annual_returns_df = pd.DataFrame(annual_returns, columns=['Close'])
    annual_returns_df.rename(columns={'Close': ticker}, inplace=True)
    return annual_returns_df
# Create an empty DataFrame to store all tickers' total returns
all_tickers_returns_df = pd.DataFrame()

# Loop through each ticker in the list
for ticker in tickers_list:
    ticker_returns_df = get_annual_returns_df(ticker)
    if not ticker_returns_df.empty:
        if all_tickers_returns_df.empty:
            all_tickers_returns_df = ticker_returns_df
        else:
            # Concatenate DataFrames
            all_tickers_returns_df = pd.concat([all_tickers_returns_df, ticker_returns_df], axis=1)

#-------------------------------------------------------------------
# step 2:
# Define a list of years to calculate the trailing returns, cumulative returns, and so on
# remove the row of current year row since it is not a full year.
years_list = [1, 2, 3, 5, 10, 15, 20, 25, 30]


#-------------------------------------------------------------------
# step 3: calculate the annualized trailing total return from the data generated in step 1 & display
# Define a function to calculate the annualized trailing total return for a given number of years
def get_trailing_return(ticker, data, years):
    # Get the total return values for the last n years
    trailing_data = data[ticker].tail(years)
    # Check if there are empty values within years
    if trailing_data.isna().any():
        return "N/A"
    # Check if there are valid total return values for all years
    if len(trailing_data) == years:
        # Convert the percentage strings to numeric values
        trailing_data = trailing_data.astype(str).str.replace('%', '').astype(float)
        """ Calculate the annualized trailing total return using the formula from Investopedia[^1^][1]:
            Annualized Return = [(1 + r1) * (1 + r2) * ... * (1 + rn)]^(1/n) - 1
            Where r1, r2, ..., rn are the total return values for each year                    """
        annualized_trailing_return = (trailing_data + 1).prod() ** (1 / years) - 1

        # Format the result as a percentage with two decimal places
        annualized_trailing_return = annualized_trailing_return * 100
        annualized_trailing_return = annualized_trailing_return.round(2)
        return annualized_trailing_return
    else:
        return "N/A"

# Define a function to Loop through the list and print the trailing returns for each num_years
def get_trailing_return_column(ticker):
    trailing_return_column = {}
    for num_years in years_list:
        # Check if the ticker data is available in all_tickers_returns_df
        if ticker in all_tickers_returns_df.columns:
            # using data from step 1, avoiding get_annual_returns_df(ticker) for less traffic from yahoo server
            data = all_tickers_returns_df[[ticker]]
            trailing_return = get_trailing_return(ticker, data, num_years)
            trailing_return_column[f"{num_years}-Year"] = trailing_return
        else:
            print(f"Data not available for {ticker}. Skipping.")
            trailing_return_column[f"{num_years}-Year"] = "N/A"
    return trailing_return_column

# Create an empty DataFrame to store all tickers' trailing returns
all_tickers_trailing_returns_df = pd.DataFrame(index=years_list)

# Loop through each ticker in the list
for ticker in tickers_list:
    trailing_returns = get_trailing_return_column(ticker)
    # Add the trailing returns to the DataFrame
    all_tickers_trailing_returns_df[ticker] = pd.Series(trailing_returns).values

#-------------------------------------------------------------------
# step 4: calculate the cumulative return from the data (all_tickers_returns_df) generated in step 1 & display
# 4.1 Define a function to calculate the cumulative return for a given number of years from a ticker
def get_cumulative_return(ticker, data, years):
    # Calculate the cumulative return
    cumulative_return = (1 + data[ticker]).rolling(window=years).apply(lambda x: x.prod(), raw=True) - 1
    return cumulative_return

# Create an empty DataFrame with years_list as the index for cumulative  returns
all_tickers_cumulative_returns_df = pd.DataFrame(index=years_list)

# Define a function to Loop through the list and return the cumulative returns for each num_years
def get_cumulative_return_column(ticker):
    cumulative_returns = {}
    for years in years_list:
        # Calculate the cumulative return for the given number of years
        cumulative_return = get_cumulative_return(ticker, all_tickers_returns_df, years)
        # Get the last value, which is the cumulative return up to the current year
        cumulative_returns[years] = cumulative_return.iloc[-1]
    return cumulative_returns

# Loop through each ticker in the list
for ticker in tickers_list:
    cumulative_returns = get_cumulative_return_column(ticker)
    # Add the trailing returns to the DataFrame
    all_tickers_cumulative_returns_df[ticker] = pd.Series(cumulative_returns).values

#-------------------------------------------------------------------
# step 5: calculate the  CAGR from the data (all_tickers_cumulative_returns_df) generated earlier & display
# Define a function to calculate the CAGR from the cumulative value and the years
import numpy as np
def calculate_cagr(value, years):
    # Otherwise, calculate the CAGR using the formula
    cagr = (value + 1) ** (1 / np.array(years)) - 1
    #print("debug-cagr\n", cagr, "end")
    return cagr

# Define a function to format the Float64Index values into percentage strings
def format_to_percentage(value):
    # If any element in the value array is not null, format it as a percentage string with two decimal places
    if np.any(pd.notnull(value)):
        return f"{value:.2f}%"
    # Otherwise, return None
    return None

# Apply the calculate_cagr function to each column of the DataFrame
all_tickers_cagrs_df = all_tickers_cumulative_returns_df.apply(lambda x: calculate_cagr(x, x.index), axis=0)

#----------------------------------------------------
# Display the formatted DataFrame  all_tickers_returns_df
if (print_yearly_total_return):
    all_tickers_returns_df.index=all_tickers_returns_df.index.date
    all_tickers_returns_df.index.name='date'
    # Convert calculation_end_day to a datetime object, replace the index's mon/day portion of date
    calculation_end_day_datetime_obj = datetime.strptime(calculation_end_day, "%Y-%m-%d")
    all_tickers_returns_df.index = all_tickers_returns_df.index.map(\
      lambda x: x.replace(month=calculation_end_day_datetime_obj.month,\
      day=calculation_end_day_datetime_obj.day))
    print("\nAnnual Total Return (%) as ", calculation_end_day)
    print(all_tickers_returns_df.round(4) * 100)

# Display the trailing returns DataFrame
print("\nAnnualized Trailing Returns as ", calculation_end_day, ":")
all_tickers_trailing_returns_df.index.name = 'Years'
print(all_tickers_trailing_returns_df)

# Display the cumulative returns DataFrame
print("\nCumulative Returns as end of year", calculation_end_day, ":")
all_tickers_cumulative_returns_df.index.name = 'Years'
print(all_tickers_cumulative_returns_df.round(4) * 100)

# Display the formatted DataFrame  all_tickers_cagrs_df
print("\nCompound Annual Growth Rates (GAGRs) (%) as end of year", calculation_end_day, ":")
all_tickers_cagrs_df.index.name = 'Years'
print(all_tickers_cagrs_df.round(4) * 100)

print(f"The result of version-{script_version} script  ends here.\n {result_marker}")
### CODE end (for AI tool)


The result of version-(2023.1224.1) script  start:
 ###

Annual Total Return (%) as  2023-12-25
              spy  vfv.to    goog    msft    meta    tsla  shop.to    amzn  \
date                                                                         
1994-12-25   0.40     NaN     NaN   51.63     NaN     NaN      NaN     NaN   
1995-12-25  38.05     NaN     NaN   43.56     NaN     NaN      NaN     NaN   
1996-12-25  22.50     NaN     NaN   88.32     NaN     NaN      NaN     NaN   
1997-12-25  33.48     NaN     NaN   56.43     NaN     NaN      NaN     NaN   
1998-12-25  28.69     NaN     NaN  114.60     NaN     NaN      NaN  966.39   
1999-12-25  20.39     NaN     NaN   68.36     NaN     NaN      NaN   42.18   
2000-12-25  -9.74     NaN     NaN  -62.85     NaN     NaN      NaN  -79.56   
2001-12-25 -11.76     NaN     NaN   52.74     NaN     NaN      NaN  -30.47   
2002-12-25 -21.58     NaN     NaN  -21.96     NaN     NaN      NaN   74.58   
2003-12-25  28.18     NaN     NaN    6.82     

# library test & misc

In [ ]:
from datetime import datetime

# Get today's date
calculation_end_day_str = datetime.now().date()
calculation_end_day_str = calculation_end_day.strftime("%Y-%m-%d")
calculation_end_day_str = datetime.now().date().strftime("%Y-%m-%d")
# Print or use the date as needed
print("Today's Date:", calculation_end_day_str)


Today's Date: 2023-12-23


## Pandas  test

### resample   BAS, BAS-JAN,  ... BAS-DEC

pandas DataFrame resampling annualy with anchored-offsets, unexpected behavior?

Pandas provides annual frequency options with anchored offsets, allowing users to resample time series data according to specific annual patterns. As stated in the user guide (https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html#anchored-offsets), the (B)A(S)-DEC frequency represents an annual frequency anchored to the end of December. The guide indicates that (BAS-DEC) is expected to be equivalent to ‘A’.

However, users may observe unexpected behavior with (BAS-DEC) not being equivalent to ‘A’.

This discrepancy could potentially be an issue in the pandas library?

On the other hand, (BA-DEC) appears to match the behavior of ‘A’.

In [ ]:
'''
pandas DataFrame resampling annualy with anchored-offsets, unexpected behavior?

According to user_guide (https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html#anchored-offsets):
the (B)A(S)-DEC  annual frequency, anchored end of December. Same as ‘A’
1.  However, 'BAS-DEC' looks not to be the same as ‘A’.  Is this an error in pandas library?
2. 'BA-DEC' looks  to be the same as ‘A’.

'''

import yfinance as yf
import pandas as pd
import numpy as np

stock = yf.Ticker("spy")
stock_history=stock.history(period="max", start='2015-06-01')["Close"]
#annual_returns = stock_history.resample('A').ffill().pct_change().dropna() #1 ok
#annual_returns = stock_history.resample('BAS-DEC').ffill().pct_change().dropna() #2 not correct
#annual_returns = stock_history.resample('BA-DEC').ffill().pct_change().dropna() #3 same as 'A' - correct
#annual_returns = stock_history.resample('AS-DEC').ffill().pct_change().dropna() #4 not correct
#annual_returns = stock_history.resample('BY-DEC').ffill().pct_change().dropna() #5  execution error
#annual_returns = stock_history.resample('BA').ffill().pct_change().dropna()
annual_returns = stock_history.resample('BA', origin="start_day").ffill().pct_change().dropna()

print(annual_returns)
print("pandas version:", pd.__version__)

Date
2016-12-30 00:00:00-05:00    0.119979
2017-12-29 00:00:00-05:00    0.217054
2018-12-31 00:00:00-05:00   -0.045690
2019-12-31 00:00:00-05:00    0.312239
2020-12-31 00:00:00-05:00    0.183316
2021-12-31 00:00:00-05:00    0.287287
2022-12-30 00:00:00-05:00   -0.181754
2023-12-29 00:00:00-05:00    0.261758
2024-12-31 00:00:00-05:00   -0.015548
Freq: BA-DEC, Name: Close, dtype: float64
pandas version: 1.5.3


In [ ]:
'''
pandas version: 1.5.3

.resample('A') result(correct):
Date
2016-12-31 00:00:00-05:00    0.119979
2017-12-31 00:00:00-05:00    0.217054
2018-12-31 00:00:00-05:00   -0.045690
2019-12-31 00:00:00-05:00    0.312239
2020-12-31 00:00:00-05:00    0.183316
2021-12-31 00:00:00-05:00    0.287288
2022-12-31 00:00:00-05:00   -0.181754
2023-12-31 00:00:00-05:00    0.261758
2024-12-31 00:00:00-05:00   -0.016894
Freq: A-DEC, Name: Close, dtype: float64


resample('BAS-DEC') result(incorrect, which is not identical to .resample('A') ):
Date
2016-12-01 00:00:00-05:00    0.064638
2017-12-01 00:00:00-05:00    0.228817
2018-12-03 00:00:00-05:00    0.075396
2019-12-02 00:00:00-05:00    0.137962
2020-12-01 00:00:00-05:00    0.197208
2021-12-01 00:00:00-05:00    0.248145
2022-12-01 00:00:00-05:00   -0.082032
2023-12-01 00:00:00-05:00    0.144698
Freq: BAS-DEC, Name: Close, dtype: float64

Date
2016-12-30 00:00:00-05:00    0.119979
2017-12-29 00:00:00-05:00    0.217054
2018-12-31 00:00:00-05:00   -0.045690
2019-12-31 00:00:00-05:00    0.312239
2020-12-31 00:00:00-05:00    0.183316
2021-12-31 00:00:00-05:00    0.287288
2022-12-30 00:00:00-05:00   -0.181754
2023-12-29 00:00:00-05:00    0.261758
2024-12-31 00:00:00-05:00   -0.016894
Freq: BA-DEC, Name: Close, dtype: float64

Date
2016-12-01 00:00:00-05:00    0.064638
2017-12-01 00:00:00-05:00    0.228817
2018-12-01 00:00:00-05:00    0.061342
2019-12-01 00:00:00-05:00    0.162909
2020-12-01 00:00:00-05:00    0.187039
2021-12-01 00:00:00-05:00    0.248145
2022-12-01 00:00:00-05:00   -0.082032
2023-12-01 00:00:00-05:00    0.144699
Freq: AS-DEC, Name: Close, dtype: float64

'''

#### by chatGpt failed  cases

In [ ]:
'''
 If you want to resample annually with a specific starting date in a year,
 you can use the 'BAS' frequency string (Business Year Start frequency).

In this example, 'BAS' stands for Business Year Start frequency, and you can append the specific starting date to
it using the format BAS{month}-{day}. This will resample the data annually with a business year starting on March 15th.
You can adjust the start_date variable to your desired date.

'''
import pandas as pd
import numpy as np

# Create a sample DataFrame with daily data
date_rng = pd.date_range(start='2022-01-01', end='2023-12-31', freq='D')
df = pd.DataFrame(index=date_rng, data={'value': np.random.rand(len(date_rng))})

# Specify the custom starting date
start_date = '03-15'  # March 15th

# Resample to business year start frequency
df_year_start_custom = df.resample('BAS').sum()

# Shift the resampled data to align with the custom starting date
shift_amount = pd.DateOffset(days=pd.to_datetime(start_date).dayofyear - 1)
df_year_start_custom_shifted = df_year_start_custom.shift(freq=shift_amount)

print("Business Year Start Frequency with Custom Starting Date:")
print(df_year_start_custom_shifted.head())


OutOfBoundsDatetime: ignored

In [ ]:
import pandas as pd
import numpy as np

# Create a sample DataFrame with daily data
date_rng = pd.date_range(start='2022-01-01', end='2023-12-31', freq='D')
df = pd.DataFrame(index=date_rng, data={'value': np.random.rand(len(date_rng))})

# Specify the custom starting date
start_date = '03-15'  # March 15th

# Resample to business year start frequency
df_year_start_custom = df.resample('BAS').sum()

# Shift the resampled data to align with the custom starting date
shift_amount = pd.DateOffset(days=(pd.to_datetime(start_date).dayofyear - 1) % 365)
df_year_start_custom_shifted = df_year_start_custom.shift(freq=shift_amount)

print("Business Year Start Frequency with Custom Starting Date:")
print(df_year_start_custom_shifted.head())


OutOfBoundsDatetime: ignored

In [ ]:
import pandas as pd
import numpy as np

# Create a sample DataFrame with daily data
date_rng = pd.date_range(start='2022-01-01', end='2023-12-31', freq='D')
df = pd.DataFrame(index=date_rng, data={'value': np.random.rand(len(date_rng))})

# Specify the custom starting date
start_date = '03-15'  # March 15th

# Resample to business year start frequency
df_year_start_custom = df.resample('BAS').sum()

# Shift the resampled data to align with the custom starting date
shift_amount = np.timedelta64((pd.to_datetime(start_date).dayofyear - 1) * 24 * 3600 * 1e9, 'ns')
df_year_start_custom_shifted = df_year_start_custom.shift(freq=shift_amount)

print("Business Year Start Frequency with Custom Starting Date:")
print(df_year_start_custom_shifted.head())


OutOfBoundsDatetime: ignored

In [ ]:
import pandas as pd
import numpy as np

# Create a sample DataFrame with daily data
date_rng = pd.date_range(start='2022-01-01', end='2023-12-31', freq='D')
df = pd.DataFrame(index=date_rng, data={'value': np.random.rand(len(date_rng))})

# Specify the custom starting date
start_date = '03-15'  # March 15th

# Resample to business year start frequency
df_year_start_custom = df.resample('BAS').sum()

# Manually shift the resampled data to align with the custom starting date
start_date_index = pd.to_datetime(start_date)
shift_amount = (start_date_index - df_year_start_custom.index[0]).days
df_year_start_custom_shifted = df_year_start_custom.shift(freq=f'{shift_amount}D')

print("Business Year Start Frequency with Custom Starting Date:")
print(df_year_start_custom_shifted.head())


OutOfBoundsDatetime: ignored

In [ ]:
import pandas as pd
import numpy as np

# Create a sample DataFrame with daily data
date_rng = pd.date_range(start='2022-01-01', end='2023-12-31', freq='D')
df = pd.DataFrame(index=date_rng, data={'value': np.random.rand(len(date_rng))})

# Specify the custom starting date
start_date = '03-15'  # March 15th

# Resample to business year start frequency
df_year_start_custom = df.resample('BAS').sum()

# Calculate the custom offset
start_date_offset = pd.DateOffset(months=pd.to_datetime(start_date).month - 1, day=1)

# Manually shift the resampled data to align with the custom starting date
df_year_start_custom_shifted = df_year_start_custom.shift(freq=start_date_offset)

print("Business Year Start Frequency with Custom Starting Date:")
print(df_year_start_custom_shifted.head())


OutOfBoundsDatetime: ignored

In [ ]:
import pandas as pd
import numpy as np

# Create a sample DataFrame with daily data
date_rng = pd.date_range(start='2022-01-01', end='2023-12-31', freq='D')
df = pd.DataFrame(index=date_rng, data={'value': np.random.rand(len(date_rng))})

# Specify the custom starting date
start_date = '03-15'  # March 15th

# Resample to business year start frequency
df_year_start_custom = df.resample('BAS').sum()

# Calculate the custom offset
start_date_offset = pd.DateOffset(months=(pd.to_datetime(start_date).month - 1), day=1)
start_date_offset_custom = pd.offsets.DateOffset(n=0, normalize=True) + start_date_offset

# Manually shift the resampled data to align with the custom starting date
df_year_start_custom_shifted = df_year_start_custom.shift(freq=start_date_offset_custom)

print("Business Year Start Frequency with Custom Starting Date:")
print(df_year_start_custom_shifted.head())


OutOfBoundsDatetime: ignored

### resample A, AS, AS-JAN

In [ ]:
# Resample Year end frequency, Year start frequency, Year start frequency with January as the starting month
'''
In pandas, when using the resample method with a frequency string for yearly frequency,
you can use various format codes to specify the starting month of the year. Some common options include:

'A': Year end frequency
'AS': Year start frequency
'AS-JAN', 'AS-FEB', ..., 'AS-DEC': Year start frequency with the specified starting month
Here's an example of how you can use these:
'''
import pandas as pd
import numpy as np

# Create a sample DataFrame with daily data
date_rng = pd.date_range(start='2020-01-01', end='2023-12-31', freq='D')
df = pd.DataFrame(index=date_rng, data={'value': np.random.rand(len(date_rng))})

# Resample to different yearly frequencies
df_year_end = df.resample('A').sum()  # Year end frequency
df_year_start = df.resample('AS').sum()  # Year start frequency
df_year_start_jan = df.resample('AS-JAN').sum()  # Year start frequency with January as the starting month

print("Year End Frequency:")
print(df_year_end.head())

print("\nYear Start Frequency:")
print(df_year_start.head())

print("\nYear Start Frequency with January as Starting Month:")
print(df_year_start_jan.head())


Year End Frequency:
                 value
2020-12-31  187.290611
2021-12-31  186.791401
2022-12-31  183.520457
2023-12-31  180.929612

Year Start Frequency:
                 value
2020-01-01  187.290611
2021-01-01  186.791401
2022-01-01  183.520457
2023-01-01  180.929612

Year Start Frequency with January as Starting Month:
                 value
2020-01-01  187.290611
2021-01-01  186.791401
2022-01-01  183.520457
2023-01-01  180.929612


In [ ]:
'''
# Display all column names of DataFrame
print(history.columns)
print(history.columns.tolist())
print(history.head)
'''

In [ ]:
import yfinance as yf
import pandas as pd
ticker="SPY"
stock=yf.Ticker("SPY")
history=stock.history(period="max")

periods = [1, 3, 5, 10, 15, 20] # in years, length = 8
trailing_return_data = [0.10647016778880825, 0.10556152327764257, 0.10488446528717588,\
                        0.08314514086640723, 0.11120038296103063, 0.08661932862864385]
trailing_return_column = pd.DataFrame(trailing_return_data, index=periods, columns=["XFN.to"])


print(trailing_return_column)


      XFN.to
1   0.106470
3   0.105562
5   0.104884
10  0.083145
15  0.111200
20  0.086619


## yfinace test

In [ ]:
import yfinance as yf
import pandas as pd


#-------------------------------------------------------------------
# step 1: fetch retrieve yearly total returns by yfinance & display
# Function to fetch data from yfinance and extract yearly total returns
def get_annual_returns_df(ticker, calculation_end_date):
    # Get the historical data for the given ticker
    stock = yf.Ticker(ticker)
    ''' Get annual total return data.
        .resample('Y'): Resamples the time series data annually ('Y'), based on the last trading day of each year.
        .resample('A'): Resamples the time series data annually ('A'), based on the any trading day of each year.
    '''
    calculation_start_date = (datetime.strptime(calculation_end_date, "%Y-%m-%d")
                - timedelta(days=num_years_calculation * 365)).strftime("%Y-%m-%d")

    ''' yfinance_BUG_1 NOTE:
    There is bug for  when using end="2024-01-02" on 2024-01-02 which is the first trading of the year on 8:10pm
     (tried tickers: "^GSPC","spy" for stock = yf.Ticker(ticker)) that 2024-01-02 data is not returned.
     however, when end parameter, it is fine for returning  2024-01-02 data as below
     annual_returns_history=stock.history(start=calculation_start_date)["Close"]
    annual_returns_history=stock.history(start=calculation_start_date,end=calculation_end_date)["Close"]
    '''
    try:
        annual_returns_history=stock.history(start=calculation_start_date,end=calculation_end_date)["Close"]
        # need to handle the exception error that a ticker is not yet at stock market, "shop.to" is not there in 2012
        annual_returns = annual_returns_history.resample('A').ffill().pct_change().dropna()
    except Exception as e:
        annual_returns_df=pd.DataFrame({ticker: [0] }, index=pd.to_datetime([calculation_end_date]))
    else:
        #print("debug get_annual_returns_df ", ticker, annual_returns_history)
        print("debug get_annual_returns_df after resample('A').ffill().pct_change().dropna()", ticker, "\n", annual_returns)
        annual_returns_df = pd.DataFrame(annual_returns, columns=['Close'])
        annual_returns_df.rename(columns={'Close': ticker}, inplace=True)
    return annual_returns_df

t_annual_returns_df=get_annual_returns_df("shop.to", "2012-12-31")
print(t_annual_returns_df)

# need to handle the exception error that a ticker is not yet at stock market, "shop.to" is not there in 2012
#annual_returns_history=stock.history(start=calculation_start_date,end=calculation_end_date)["Close"]


ERROR:yfinance:SHOP.TO: Data doesn't exist for startDate = 32418000, endDate = 1356930000


            shop.to
2012-12-31        0


### New Section

In [ ]:
import yfinance as yf
from yfinance import shared
import pandas as pd
from datetime import datetime, timedelta

# Function to fetch data from yfinance and extract yearly total returns
def get_annual_returns_df(ticker, calculation_end_date, num_years_calculation=5):
    # Get the historical data for the given ticker
    stock = yf.Ticker(ticker)
    calculation_start_date = (datetime.strptime(calculation_end_date, "%Y-%m-%d")
                - timedelta(days=num_years_calculation * 365)).strftime("%Y-%m-%d")

    try:
        annual_returns_history = stock.history(start=calculation_start_date, end=calculation_end_date)["Close"]

        if error_message:
            print(f"An error occurred while fetching data for {ticker}: {error_message}")
        else:
            print(f"Data for {ticker} was successfully fetched")

        if len(annual_returns_history) == 0:
            raise ValueError("No data available for the specified period")

        # Continue with the normal processing
        #annual_returns = annual_returns_history.resample('A').ffill().pct_change().dropna()
    except Exception as e:
        print(f"An error occurred while fetching data for {ticker}: {e}")
        annual_returns_df = pd.DataFrame({ticker: [0]}, index=pd.to_datetime([calculation_end_date]))
    else:
        print("debug get_annual_returns_df after resample('A').ffill().pct_change().dropna()", ticker, "\n", annual_returns)
        annual_returns_df = pd.DataFrame(annual_returns, columns=['Close'])
        annual_returns_df.rename(columns={'Close': ticker}, inplace=True)
    return annual_returns_df

t_annual_returns_df = get_annual_returns_df("shop.to", "2012-12-31")
print("t_annual_returns_df", t_annual_returns_df)


ERROR:yfinance:SHOP.TO: Data doesn't exist for startDate = 1199250000, endDate = 1356930000


An error occurred while fetching data for shop.to: name 'error_message' is not defined
t_annual_returns_df             shop.to
2012-12-31        0


In [ ]:
import yfinance as yf
import pandas as pd
stock=yf.Ticker("SPY")

# .index.year,  index.year.max()
#history=stock.history(period="1mo")
#print(history.index.year)
#print(history.index.year.max())

# multiple tickers
tickers = yf.Tickers('msft aapl goog')

# access each ticker using (example)
#tickers.tickers['MSFT'].info
print("###\n")
tickers_history=tickers.tickers['AAPL'].history(period="max")
print("###\n",tickers_history)
#tickers.tickers['GOOG'].actions

###

###
                                  Open        High         Low       Close  \
Date                                                                        
1980-12-12 00:00:00-05:00    0.099319    0.099750    0.099319    0.099319   
1980-12-15 00:00:00-05:00    0.094569    0.094569    0.094137    0.094137   
1980-12-16 00:00:00-05:00    0.087659    0.087659    0.087228    0.087228   
1980-12-17 00:00:00-05:00    0.089387    0.089818    0.089387    0.089387   
1980-12-18 00:00:00-05:00    0.091978    0.092410    0.091978    0.091978   
...                               ...         ...         ...         ...   
2023-12-18 00:00:00-05:00  196.089996  196.630005  194.389999  195.889999   
2023-12-19 00:00:00-05:00  196.160004  196.949997  195.889999  196.940002   
2023-12-20 00:00:00-05:00  196.899994  197.679993  194.830002  194.830002   
2023-12-21 00:00:00-05:00  196.100006  197.080002  193.500000  194.679993   
2023-12-22 00:00:00-05:00  195.179993  195.410004  192.970001  193

##  Template of markers for marking CODE start and end Marker for AI  debug


In [ ]:
"""
### CODE start (for AI tool)
.....
History:
Versions:
 * 2023.1222.1:
Author: Gang Luo
"""
result_marker='###'
script_version = '(2023.1222.1)'
print(f"The result of version-{script_version} script  start:\n {result_marker}")
import yfinance as yf
import pandas as pd

print_yearly_total_return = True

# Set the stock tickers list
tickers_list1 = ["spy", "vfv.to", "zlu.to", "xiu.to", "xfn.to", "ry.to", "td.to", "na.to",\
                "slf.to", "gwo.to", "bce.to", "t.to", "rci-b.to", "enb.to", "trp.to", "zlb.to"]
tickers_list2 = ["vfv.to", "xiu.to", "xic.to", "xfn.to", "ry.to", "td.to", "na.to",\
                "bns.to", "bmo.to", "cm.to", "cwb.to", "slf.to", "gwo.to"]
tickers_list3 = ["spy", "vfv.to", "xfn.to", "ry.to"]
tickers_list = tickers_list1


# print an indicator to mark the end of execution result
print(f"The result of version-{script_version} script  ends here.\n {result_marker}")
### CODE end (for AI tool)

The result of version-(2023.1222.1) script  start:
 ###
The result of version-(2023.1222.1) script  ends here.
 ###


# Graphical line diagram of stock cumulative returns

In [ ]:
'''
graphical line diagram of stock cumulative returns

Note: Raw Data is from Yahoo Finance using python yfinance
'''
script_version = '(2024-01-28.1)'
# Import the libraries
import gradio as gr
import yfinance as yf
import pandas as pd

def stock_return_df(tickers, start, end):
  # Download the historical data
  data = yf.download(tickers, start=start, end=end)
  # Calculate the daily and cumulative percentage change
  data_adj_close = data['Adj Close']
  data_adj_close_pct = data_adj_close.pct_change()
  data_adj_close_pct_cum = (data_adj_close_pct + 1).cumprod(axis=0) - 1
  return data_adj_close_pct_cum

def plot_graph(tickers, start, end):
  data_adj_close_pct_cum=stock_return_df(tickers, start, end)
  # Plot a line chart of the salary by name
  #ax = data_adj_close_pct_cum.plot.line(x="Name", y="Salary", rot=0, legend=False, style="-o", markersize=10, color="green")
  ax = data_adj_close_pct_cum.plot()
  ax.set_ylabel("Percentage")
  ax.set_title("Cumulative Returns")
  # Return the plot as a matplotlib figure
  return ax.get_figure()

# Create a gradio.Interface object with the DataFrame component as input and the Matplotlib component as output
interface = gr.Interface(
    fn=plot_graph, # The function to call
    inputs=[gr.Textbox(label="Tickers", value="qqq,spy,vfv.to,xiu.to,xfn.to,ry.to", scale=0,  min_width=150),
        gr.Textbox(label="Start Date", value="2014-01-01", scale=0, min_width=150),
        gr.Textbox(label="End Date", value="2023-12-30", scale=0, min_width=150)],
    outputs=gr.Plot(label="Output Plot", scale=2, min_width=250), # The Matplotlib component as output
    title="Stock cumulative returns - DataFrame Line Plot by Gradio" # The title of the interface
)

# Launch the interface
interface.launch(debug=False)


Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://5c94a02b22d28bc483.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


## backup - Graphical line diagram of stock cumulative returns

In [ ]:
'''
graphical line diagram of stock cumulative returns

Note: Raw Data is from Yahoo Finance using python yfinance
'''
script_version = '(2024-01-28.1)'
# Import the libraries
import gradio as gr
import yfinance as yf
import pandas as pd

def stock_return_df(tickers, start, end):
  # Download the historical data
  data = yf.download(tickers, start=start, end=end)
  # Calculate the daily and cumulative percentage change
  data_adj_close = data['Adj Close']
  data_adj_close_pct = data_adj_close.pct_change()
  data_adj_close_pct_cum = (data_adj_close_pct + 1).cumprod(axis=0) - 1
  return data_adj_close_pct_cum

def plot_graph(tickers, start, end):
  data_adj_close_pct_cum=stock_return_df(tickers, start, end)
  # Plot a line chart of the salary by name
  #ax = data_adj_close_pct_cum.plot.line(x="Name", y="Salary", rot=0, legend=False, style="-o", markersize=10, color="green")
  ax = data_adj_close_pct_cum.plot()
  ax.set_ylabel("Percentage")
  ax.set_title("Cumulative Returns")
  # Return the plot as a matplotlib figure
  return ax.get_figure()

# Create a gradio.Interface object with the DataFrame component as input and the Matplotlib component as output
interface = gr.Interface(
    fn=plot_graph, # The function to call
    inputs=[gr.Textbox(label="Tickers", value="qqq,spy,vfv.to,xiu.to,xfn.to,ry.to"),
        gr.Textbox(label="Start Date", value="2014-01-01"),
        gr.Textbox(label="End Date", value="2023-12-30")],
    outputs=gr.Plot(label="Output Plot"), # The Matplotlib component as output
    title="Stock cumulative returns - DataFrame Line Plot by Gradio" # The title of the interface
)

# Launch the interface
interface.launch(debug=False)


Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://008eab34382b4350da.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


# plot by using gr.Blocks()

In [ ]:
# Import the libraries
import gradio as gr
import yfinance as yf
import pandas as pd

def stock_return_df(tickers, start, end):
  # Download the historical data
  data = yf.download(tickers, start=start, end=end)
  # Calculate the daily and cumulative percentage change
  data_adj_close = data['Adj Close']
  data_adj_close_pct = data_adj_close.pct_change()
  data_adj_close_pct_cum = (data_adj_close_pct + 1).cumprod(axis=0) - 1
  return data_adj_close_pct_cum

def plot_graph(tickers, start, end):
  data_adj_close_pct_cum=stock_return_df(tickers, start, end)
  # Plot a line chart of the salary by name
  #ax = data_adj_close_pct_cum.plot.line(x="Name", y="Salary", rot=0, legend=False, style="-o", markersize=10, color="green")
  ax = data_adj_close_pct_cum.plot()
  ax.set_ylabel("Percentage")
  ax.set_title("Cumulative Returns")
  # Return the plot as a matplotlib figure
  return ax.get_figure()

# Create a gr.Blocks() object and use it as a context
with gr.Blocks() as demo:
    # Define the input and output components within the gr.Blocks() context
    tickers = gr.Textbox(label="Tickers", value="qqq,spy,vfv.to,xiu.to,xfn.to,ry.to")
    start_date = gr.Textbox(label="Start Date", value="2014-01-01")
    end_date = gr.Textbox(label="End Date", value="2023-12-30")
    output_plot = gr.Plot(label="Output Plot")
    # Define an event listener that calls the plot_graph function when the input components are changed
    tickers.change(plot_graph, # The function to call
                   inputs=[tickers, start_date, end_date], # The inputs of the function
                   outputs=output_plot, # The output of the function
                   api_name="plot_graph" # The API name of the function
                   )
    start_date.change(plot_graph, # The function to call
                      inputs=[tickers, start_date, end_date], # The inputs of the function
                      outputs=output_plot, # The output of the function
                      api_name="plot_graph" # The API name of the function
                      )
    end_date.change(plot_graph, # The function to call
                    inputs=[tickers, start_date, end_date], # The inputs of the function
                    outputs=output_plot, # The output of the function
                    api_name="plot_graph" # The API name of the function
                    )
    # Set the title of the demo
    demo.title = "Stock cumulative returns - DataFrame Line Plot by Gradio"
# Launch the demo
demo.launch(debug=False)


/usr/local/lib/python3.10/dist-packages/yfinance/base.py:48: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  _empty_series = pd.Series()


Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://7327a226d13620528a.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


## New Section

In [ ]:
# Import the libraries
import gradio as gr
import yfinance as yf
import pandas as pd

def stock_return_df(tickers, start, end):
  # Download the historical data
  data = yf.download(tickers, start=start, end=end)
  # Calculate the daily and cumulative percentage change
  data_adj_close = data['Adj Close']
  data_adj_close_pct = data_adj_close.pct_change()
  data_adj_close_pct_cum = (data_adj_close_pct + 1).cumprod(axis=0) - 1
  return data_adj_close_pct_cum

def plot_graph(tickers, start, end):
  data_adj_close_pct_cum=stock_return_df(tickers, start, end)
  # Plot a line chart of the salary by name
  #ax = data_adj_close_pct_cum.plot.line(x="Name", y="Salary", rot=0, legend=False, style="-o", markersize=10, color="green")
  ax = data_adj_close_pct_cum.plot()
  ax.set_ylabel("Percentage")
  ax.set_title("Cumulative Returns")
  # Return the plot as a matplotlib figure
  return ax.get_figure()

# Create a gr.Blocks() object and use it as a context
with gr.Blocks() as demo:
    # Define the input and output components within the gr.Blocks() context
    tickers = gr.Textbox(label="Tickers", value="qqq,spy,vfv.to,xiu.to,xfn.to,ry.to")
    start_date = gr.Textbox(label="Start Date", value="2014-01-01")
    end_date = gr.Textbox(label="End Date", value="2023-12-30")
    output_plot = gr.Plot(label="Output Plot")
    # Create a submit button
    submit_btn = gr.Button(label="Submit")
    # Define an event listener that calls the plot_graph function when the submit button is clicked
    submit_btn.click(plot_graph, # The function to call
                     inputs=[tickers, start_date, end_date], # The inputs of the function
                     outputs=output_plot, # The output of the function
                     api_name="plot_graph" # The API name of the function
                     )
    # Set the title of the demo
    demo.title = "Stock cumulative returns - DataFrame Line Plot by Gradio"
# Launch the demo
demo.launch(debug=False)

TypeError: Button.__init__() got an unexpected keyword argument 'label'

# a list of end-of year stock prices

In [ ]:
# Import the libraries
import gradio as gr
import yfinance as yf
import pandas as pd

tickers_list2=["qqq","spy","vfv.to","xiu.to","xfn.to","ry.to"]
tickers_list1=["^IXIC","^DJI","^GSPC","^GSPTSE"]
tickers_list0=["qqq","tqqq","QLD", "spy","upro", "sso", "spxl"]
tickers_list3=["iwy","nvda"]
tickers_list=["^IXIC", "qqq","tqqq"]
start_date="2014-01-01"
calculation_end_date_str="2024-02-09"

def stock_close_prices_df(tickers_list, end_date_str):
  tickers_list_upper = [ticker.upper() for ticker in tickers_list]
  # Download the historical data
  tickers_str = ", ".join(tickers_list_upper)
  #tickers_str = ", ".join(f'"{ticker}"' for ticker in tickers_list) # maintain the same order of tickers
  data = yf.download(tickers_str, period="max")
  data_adj_close = data['Adj Close']

  # Rearrange columns based on the order in tickers_list
  if len(tickers_list)>1:
      data_adj_close = data_adj_close.reindex(columns=tickers_list_upper)
  #data_adj_close = data_adj_close.reindex(index=tickers_list_upper)
  #data_adj_close.columns = tickers_list_upper

  data_adj_close = data_adj_close.resample('A').ffill().round(2)
  data_adj_close.index=data_adj_close.index.date
  last_date = data_adj_close.index[-1]
  data_adj_close = data_adj_close.rename(index={last_date: end_date_str})
  return data_adj_close

output_dataframe= stock_close_prices_df(tickers_list, calculation_end_date_str)
#output_html0=output_dataframe.to_html()
output_dataframe.columns = map(str.lower, output_dataframe.columns)
print(" ")
print( output_dataframe)

[*********************100%%**********************]  3 of 3 completed

 
               ^ixic     qqq   tqqq
1971-12-31    114.12     NaN    NaN
1972-12-31    133.73     NaN    NaN
1973-12-31     92.19     NaN    NaN
1974-12-31     59.82     NaN    NaN
1975-12-31     77.62     NaN    NaN
1976-12-31     97.88     NaN    NaN
1977-12-31    105.05     NaN    NaN
1978-12-31    117.98     NaN    NaN
1979-12-31    151.14     NaN    NaN
1980-12-31    202.34     NaN    NaN
1981-12-31    195.84     NaN    NaN
1982-12-31    232.41     NaN    NaN
1983-12-31    278.60     NaN    NaN
1984-12-31    247.10     NaN    NaN
1985-12-31    324.90     NaN    NaN
1986-12-31    348.80     NaN    NaN
1987-12-31    330.50     NaN    NaN
1988-12-31    381.40     NaN    NaN
1989-12-31    454.80     NaN    NaN
1990-12-31    373.80     NaN    NaN
1991-12-31    586.34     NaN    NaN
1992-12-31    676.95     NaN    NaN
1993-12-31    776.80     NaN    NaN
1994-12-31    751.96     NaN    NaN
1995-12-31   1052.13     NaN    NaN
1996-12-31   1291.03     NaN    NaN
1997-12-31   1570.35     N

# Maximum price drop & a list of tickers' daily adj close :prices

In [ ]:
# Import the libraries
import gradio as gr
import yfinance as yf
import pandas as pd
import numpy as np

tickers_list2=["qqq","spy","vfv.to","xiu.to","xfn.to","ry.to"]
tickers_list1=["^IXIC","^DJI","^GSPC","^GSPTSE"]
tickers_list=["qqq","tqqq","QLD", "spy","upro", "sso", "spxl"]
tickers_list3=["tqqq"]
#start_date="2014-01-01"
calculation_end_date_str="2024-02-10"

def stock_close_prices_df(tickers_list, end_date_str):
  tickers_list_upper = [ticker.upper() for ticker in tickers_list]
  # Download the historical data
  tickers_str = ", ".join(tickers_list_upper)
  #tickers_str = ", ".join(f'"{ticker}"' for ticker in tickers_list) # maintain the same order of tickers
  data = yf.download(tickers_str, period="max", end=end_date_str)
  data_adj_close = data['Adj Close']

  # Rearrange columns based on the order in tickers_list
  if len(tickers_list_upper)>1:
      data_adj_close = data_adj_close.reindex(columns=tickers_list_upper)
  #data_adj_close.columns = tickers_list_upper

  data_adj_close = data_adj_close.round(2)
  data_adj_close.index=data_adj_close.index.date

  return data_adj_close

output_dataframe= stock_close_prices_df(tickers_list, calculation_end_date_str)
#output_html0=output_dataframe.to_html()
print(" ")
print( output_dataframe)
# convert to datafranme sicne output_dataframe could become series when having a single ticker
output_dataframe=pd.DataFrame(output_dataframe)

# find the highest price and corresponding date for each column
high_price = output_dataframe.max() # get the maximum value for each column
high_price_date = output_dataframe.idxmax() # get the index of the maximum value for each column

# create a DataFrame to store the results
high_price_df = pd.DataFrame(
    {
        "Ticker": output_dataframe.columns,
        "HighPrice": high_price,
        "HighPriceDate": high_price_date,
    }
)

# display the DataFrame
#print(high_price_df)

# find the lowest price and corresponding date after the high price date for each column
# create an empty list to store the results
low_price_list = []

# loop through each column
for ticker in output_dataframe.columns:
    # get the high price date for the current column
    high_date = high_price_date[ticker]
    # get the subset of the output_dataframe after the high price date
    sub_df = output_dataframe.loc[high_date:, ticker]
    # get the minimum value and the index of the minimum value for the subset
    low_price = sub_df.min()
    low_price_date = sub_df.idxmin()
    # append the results to the list
    low_price_list.append([ticker, low_price, low_price_date])

# create a DataFrame from the list
low_price_df = pd.DataFrame(
    low_price_list, columns=["Ticker", "LowPrice", "LowPriceDate"]
)

# display the DataFrame
#print(low_price_df)

# set the index of high_price_df and low_price_df to be the Ticker column
high_price_df = high_price_df.set_index('Ticker')
low_price_df = low_price_df.set_index('Ticker')

# create a single DataFrame by concatenating the two DataFrames along the columns
combined_df = pd.concat([high_price_df, low_price_df], axis=1)

# define a function to calculate the percentage of price drop
def price_drop(row):
    # get the high price and low price from the row
    high_price = row['HighPrice']
    low_price = row['LowPrice']
    # calculate the percentage of price drop
    drop = (high_price - low_price) / high_price * 100
    # return the result
    return drop

# apply the function to each row of the combined_df and store the result in a new column
combined_df['PriceDrop%'] = combined_df.apply(price_drop, axis=1)


'''
# This version let NaN be the price of inception
#-------------------------------------------------
# find the earliest date and the price of that day for each column
inceptionDate = output_dataframe.first_valid_index() # get the first valid index for the DataFrame
inceptPrice = output_dataframe.loc[inceptionDate] # get the values at the first valid index

# find the latest date and the price of that day for each column
currentDate = output_dataframe.last_valid_index() # get the last valid index for the DataFrame
curPrice = output_dataframe.loc[currentDate] # get the values at the last valid index

# calculate the price change percentage from the earliest date to the latest date
priceChange = (curPrice - inceptPrice) / inceptPrice * 100 # calculate the percentage change

# add these three new columns to the combined_df
combined_df = pd.concat([high_price_df, low_price_df], axis=1) # create the combined_df from the previous task
combined_df['Inception Date'] = inceptionDate # add the inceptionDate column
combined_df['Inception Price'] = inceptPrice # add the inceptPrice column
combined_df['Current Date'] = currentDate # add the currentDate column
combined_df['Current Price'] = curPrice # add the curPrice column
combined_df['Price Change (%)'] = priceChange # add the priceChange column
'''

'''
#-----------------------------------------------
# This version drop the whole row that has NaN be the price of inception
# drop any rows with missing values
output_dataframe = output_dataframe.dropna()

# find the earliest date and the price of that day for each column
inceptionDate = output_dataframe.first_valid_index() # get the first valid index for the DataFrame
inceptPrice = output_dataframe.loc[inceptionDate] # get the values at the first valid index

# find the latest date and the price of that day for each column
currentDate = output_dataframe.last_valid_index() # get the last valid index for the DataFrame
curPrice = output_dataframe.loc[currentDate] # get the values at the last valid index

# calculate the price change percentage from the earliest date to the latest date
priceChange = (curPrice - inceptPrice) / inceptPrice * 100 # calculate the percentage change

# add these five new columns to the combined_df
combined_df = pd.concat([high_price_df, low_price_df], axis=1) # create the combined_df from the previous task
combined_df['Inception Date'] = inceptionDate # add the inceptionDate column
combined_df['Inception Price'] = inceptPrice # add the inceptPrice column
combined_df['Current Date'] = currentDate # add the currentDate column
combined_df['Current Price'] = curPrice # add the curPrice column
combined_df['Price Change (%)'] = priceChange # add the priceChange column
'''

# define a function to return the first valid value and index for a Series
def first_valid(series):
    # get the first valid value
    value = series.dropna().iloc[0]
    # get the index of the first valid value
    index = series.first_valid_index()
    # return a tuple of value and index
    return (value, index)

# apply the function to each column and get the inception price and date
inceptPrice, inceptionDate = output_dataframe.apply(first_valid).apply(pd.Series).values

# convert the arrays to pandas Series
inceptPrice = pd.Series(inceptPrice, index=output_dataframe.columns)
inceptionDate = pd.Series(inceptionDate, index=output_dataframe.columns)

# find the latest date and the price of that day for each column
currentDate = output_dataframe.last_valid_index() # get the last valid index for the DataFrame
curPrice = output_dataframe.loc[currentDate] # get the values at the last valid index

# calculate the price change percentage from the earliest date to the latest date
priceChange = (curPrice - inceptPrice) / inceptPrice * 100 # calculate the percentage change

# add these  new columns to the combined_df
combined_df['InceptDate'] = inceptionDate # add the inceptionDate column
combined_df['InceptPrice'] = inceptPrice # add the inceptPrice column
combined_df['CurDate'] = currentDate # add the currentDate column
combined_df['CurPrice'] = curPrice # add the curPrice column
combined_df['PriceChange%'] = priceChange # add the priceChange column

# round the PriceChange% column to two decimal places
combined_df['PriceChange%'] = combined_df['PriceChange%'].astype(float).round(2)
combined_df['PriceChange%'] = combined_df['PriceChange%'].apply(lambda x: round(x, 2))



# display the DataFrame
print(combined_df)

#combined_tranpose_df=combined_df.transpose()
#print(combined_tranpose_df)

[*********************100%%**********************]  7 of 7 completed


 
Ticker         QQQ   TQQQ    QLD     SPY   UPRO    SSO    SPXL
1993-01-29     NaN    NaN    NaN   24.84    NaN    NaN     NaN
1993-02-01     NaN    NaN    NaN   25.02    NaN    NaN     NaN
1993-02-02     NaN    NaN    NaN   25.07    NaN    NaN     NaN
1993-02-03     NaN    NaN    NaN   25.34    NaN    NaN     NaN
1993-02-04     NaN    NaN    NaN   25.44    NaN    NaN     NaN
...            ...    ...    ...     ...    ...    ...     ...
2024-02-05  428.45  56.95  82.44  492.55  59.91  69.29  113.82
2024-02-06  427.59  56.62  82.14  493.98  60.40  69.66  114.74
2024-02-07  431.99  58.31  83.79  498.10  61.88  70.80  117.57
2024-02-08  432.79  58.55  84.06  498.32  61.95  70.86  117.71
2024-02-09  437.05  60.27  85.68  501.20  62.97  71.67  119.62

[7814 rows x 7 columns]
        HighPrice HighPriceDate  LowPrice LowPriceDate  PriceDrop%  \
Ticker                                                               
QQQ        437.05    2024-02-09    437.05   2024-02-09    0.000000   
TQQQ   

## prompt for generating script

In [ ]:
output_dataframe is a Pandas DataFrame in python, where the index is dates.  The names of Columns are stock tickers.
Each column contains historic daily close prices for a ticker.
print( output_dataframe) has the following result:

Ticker         QQQ   TQQQ    QLD     SPY   UPRO    SSO    SPXL
1993-01-29     NaN    NaN    NaN   24.84    NaN    NaN     NaN
1993-02-01     NaN    NaN    NaN   25.02    NaN    NaN     NaN
1993-02-02     NaN    NaN    NaN   25.07    NaN    NaN     NaN
1993-02-03     NaN    NaN    NaN   25.34    NaN    NaN     NaN
1993-02-04     NaN    NaN    NaN   25.44    NaN    NaN     NaN
...            ...    ...    ...     ...    ...    ...     ...
2024-02-05  428.45  56.95  82.44  492.55  59.91  69.29  113.82
2024-02-06  427.59  56.62  82.14  493.98  60.40  69.66  114.74
2024-02-07  431.99  58.31  83.79  498.10  61.88  70.80  117.57
2024-02-08  432.79  58.55  84.06  498.32  61.95  70.86  117.71
2024-02-09  437.05  60.27  85.68  501.20  62.97  71.67  119.62

May you help me to provide a script:
1. find the highest price and corresponding date (called high_price_date) for each column
2. find the lowest price and corresponding date (called maximum_drop_date) after the date (high_price_date) found
   in the previous step for for each column

Here is my modified instruction. May you help me to provide a script:
1. find the  earlist date (inceptionDate) such that the price of that day is not NaN (should be valid price) for  each column
2. find the price of inceptionDate () for  each column
3. find the latest date (currentDate) for  each column
4. find the price of that day (called CurPRice)  for each column
5. calculate the price change percentage from the earlist date to the te latest date.
6. add these five new columns to the combined_df


The above droped the whole row that has a NaN such that droped the ticker that has earlier inception date.
May you to find the  earlist date (inceptionDate) such that the price of that day is not NaN (should be valid price)
for  each column without droping the whole row. Instead, seach each ticker idividually without changing output_dataframe congtent
for keeping data of the other tickers?




SyntaxError: leading zeros in decimal integer literals are not permitted; use an 0o prefix for octal integers (<ipython-input-5-1ac6a6f9bc9e>, line 4)

# New Section

In [ ]:
# import pandas library
import pandas as pd

# create a DataFrame from the given data
output_dataframe = pd.DataFrame(
    {
        "QQQ": [None, None, None, None, None, 428.45, 427.59, 431.99, 432.79, 437.05],
        "TQQQ": [None, None, None, None, None, 56.95, 56.62, 58.31, 58.55, 60.27],
        "QLD": [None, None, None, None, None, 82.44, 82.14, 83.79, 84.06, 85.68],
        "SPY": [24.84, 25.02, 25.07, 25.34, 25.44, 492.55, 493.98, 498.10, 498.32, 501.20],
        "UPRO": [None, None, None, None, None, 59.91, 60.40, 61.88, 61.95, 62.97],
        "SSO": [None, None, None, None, None, 69.29, 69.66, 70.80, 70.86, 71.67],
        "SPXL": [None, None, None, None, None, 113.82, 114.74, 117.57, 117.71, 119.62],
    },
    index=pd.to_datetime(
        [
            "1993-01-29",
            "1993-02-01",
            "1993-02-02",
            "1993-02-03",
            "1993-02-04",
            "2024-02-05",
            "2024-02-06",
            "2024-02-07",
            "2024-02-08",
            "2024-02-09",
        ]
    ),
)

# find the highest price and corresponding date for each column
high_price = output_dataframe.max() # get the maximum value for each column
high_price_date = output_dataframe.idxmax() # get the index of the maximum value for each column

# create a DataFrame to store the results
high_price_df = pd.DataFrame(
    {
        "Ticker": output_dataframe.columns,
        "High Price": high_price,
        "High Price Date": high_price_date,
    }
)

# display the DataFrame
print(high_price_df)

# find the lowest price and corresponding date after the high price date for each column
# create an empty list to store the results
low_price_list = []

# loop through each column
for ticker in output_dataframe.columns:
    # get the high price date for the current column
    high_date = high_price_date[ticker]
    # get the subset of the output_dataframe after the high price date
    sub_df = output_dataframe.loc[high_date:, ticker]
    # get the minimum value and the index of the minimum value for the subset
    low_price = sub_df.min()
    low_price_date = sub_df.idxmin()
    # append the results to the list
    low_price_list.append([ticker, low_price, low_price_date])

# create a DataFrame from the list
low_price_df = pd.DataFrame(
    low_price_list, columns=["Ticker", "Low Price", "Low Price Date"]
)

# display the DataFrame
print(low_price_df)


     Ticker  High Price High Price Date
QQQ     QQQ      437.05      2024-02-09
TQQQ   TQQQ       60.27      2024-02-09
QLD     QLD       85.68      2024-02-09
SPY     SPY      501.20      2024-02-09
UPRO   UPRO       62.97      2024-02-09
SSO     SSO       71.67      2024-02-09
SPXL   SPXL      119.62      2024-02-09
  Ticker  Low Price Low Price Date
0    QQQ     437.05     2024-02-09
1   TQQQ      60.27     2024-02-09
2    QLD      85.68     2024-02-09
3    SPY     501.20     2024-02-09
4   UPRO      62.97     2024-02-09
5    SSO      71.67     2024-02-09
6   SPXL     119.62     2024-02-09


In [ ]:
output_dataframe is a Pandas DataFrame in python, where the index is dates.  The names of Columns are stock tickers.
Each column contains historic daily close prices for a ticker.
print( output_dataframe) has the following result:

Ticker         QQQ   TQQQ    QLD     SPY   UPRO    SSO    SPXL
1993-01-29     NaN    NaN    NaN   24.84    NaN    NaN     NaN
1993-02-01     NaN    NaN    NaN   25.02    NaN    NaN     NaN
1993-02-02     NaN    NaN    NaN   25.07    NaN    NaN     NaN
1993-02-03     NaN    NaN    NaN   25.34    NaN    NaN     NaN
1993-02-04     NaN    NaN    NaN   25.44    NaN    NaN     NaN
...            ...    ...    ...     ...    ...    ...     ...
2024-02-05  428.45  56.95  82.44  492.55  59.91  69.29  113.82
2024-02-06  427.59  56.62  82.14  493.98  60.40  69.66  114.74
2024-02-07  431.99  58.31  83.79  498.10  61.88  70.80  117.57
2024-02-08  432.79  58.55  84.06  498.32  61.95  70.86  117.71
2024-02-09  437.05  60.27  85.68  501.20  62.97  71.67  119.62

May you help me to provide a script:
1. find the highest price and corresponding date (called high_price_date) for each column
2. find the lowest price and corresponding date (called maximum_drop_date) after the date (high_price_date) found
   in the previous step for for each column


SyntaxError: leading zeros in decimal integer literals are not permitted; use an 0o prefix for octal integers (<ipython-input-5-1ac6a6f9bc9e>, line 4)

# yahooquery

In [ ]:
!pip install yahooquery
import pandas as pd
import yahooquery as yq

def get_symbol(query, preferred_exchange='AMS'):
    try:
        data = yq.search(query)
    except ValueError: # Will catch JSONDecodeError
        print(query)
    else:
        quotes = data['quotes']
        if len(quotes) == 0:
            return 'No Symbol Found'

        symbol = quotes[0]['symbol']
        for quote in quotes:
            if quote['exchange'] == preferred_exchange:
                symbol = quote['symbol']
                break
        return symbol

companies = ['Abbott Laboratories', 'ABBVIE', 'Abercrombie', 'Abiomed', 'Accenture Plc']
df = pd.DataFrame({'Company name': companies})
df['Company symbol'] = df.apply(lambda x: get_symbol(x['Company name']), axis=1)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.7/52.7 kB 603.1 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.0/143.0 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 24.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 346.6/346.6 kB 23.0 MB/s eta 0:00:00
  Attempting uninstall: beautifulsoup4
    Found existing installation: beautifulsoup4 4.11.2
    Uninstalling beautifulsoup4-4.11.2:
      Successfully uninstalled beautifulsoup4-4.11.2
  Attempting uninstall: pandas
    Found existing installation: pandas 1.5.3
    Uninstalling pandas-1.5.3:
      Successfully uninstalled pandas-1.5.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
lida 0.0.10 requires fastapi, which is not installed.
lida 0.0.10 requires kaleido, which is not installed.
lida 0.0.10 requires python-multipart, which is no

# test

In [ ]:
 CODE: ###
 yearly_data[['DivRatio','Return', 'TotalReturn', 'CalReturn']] = yearly_data[['DivRatio','Return', 'TotalReturn', 'CalReturn']].applymap("{:.2f}%".format)
 ###
 In the above  CODE, yearly_data ia Pandas DataFrame variable. The code is to add % to every values in the columns of
 'DivRatio','Return', 'TotalReturn', 'CalReturn'.   Even when the value is NaN,  % is still added resulting NaN%.
 Please modify the code to add % to all vaules except for NaN.




SyntaxError: leading zeros in decimal integer literals are not permitted; use an 0o prefix for octal integers (<ipython-input-5-ca93643a7020>, line 22)

In [ ]:
It